In [1]:
import requests # to get image from the web
import shutil # to save it locally
import os
import pandas as pd
from multiprocessing import Pool
from datetime import datetime
import time
from itertools import repeat

In [2]:
img_df = pd.read_csv('/Users/travistang/Documents/TorchScene/data/csv/Travis Fake_Merchant_Tagging_2022 - Combined Set.csv')
img_df

,source,entity_id,saudagar_id,last_action,last_rule,last_sanction_datetime,outlet_photo_url,bank_acc_name,bank_acc_no,cnt_diff_entity_shared_bank_acc,...,flag_fake_merchant,prediction_score,victor_label_inappropriate_flag,restaurant_cart_inappropriateness_label,random,set,restaurant_store_cart_false_positive,restaurant_store_cart_false_negative,indoor/outdoor,other labels
0,data,0012s00000NXGa6AAH,G285231232,SuspendMerchantPayout,IBT-M52-009-10A,2022-07-12 9:33:43,https://api.gobiz.co.id/v1/onboardings/data/SF...,MOHAMAD RIDWAN,8.330079e+09,302.0,...,Fake,0.000354,1,1.0,0.904060,test,0.0,0.0,NaN,NaN
1,data,0012s00000FLXXuAAP,G003365482,SuspendMerchantPayout,IBT-M52-019-02,2022-08-24 6:10:52,https://api.gobiz.co.id/v1/onboardings/data/SF...,MOHAMAD RIDWAN,8.330079e+09,302.0,...,Fake,0.000036,1,1.0,0.655615,test,0.0,0.0,NaN,NaN
2,data,0012s00000OCFBRAA5,G446076816,SuspendMerchantGoResto,IBT-M52-017A,2022-08-24 19:43:15,https://api.gobiz.co.id/v1/onboardings/data/SF...,MOHAMAD RIDWAN,8.330079e+09,302.0,...,Fake,0.987236,1,0.0,0.953858,test,0.0,1.0,NaN,NaN
3,data,0012s00000OEObcAAH,G552360890,SuspendMerchantGoResto,IBT-M52-017A,2022-09-07 13:07:06,https://api.gobiz.co.id/v1/onboardings/data/SF...,MOHAMAD RIDWAN,8.330079e+09,302.0,...,Fake,0.993513,1,0.0,0.896300,test,0.0,1.0,NaN,NaN
4,data,0012s00000ODxbsAAD,G189308019,SuspendMerchantPayout,IBT-M60-006-02A,2022-09-21 10:38:10,https://api.gobiz.co.id/v1/onboardings/data/SF...,MOHAMAD RIDWAN,8.330079e+09,302.0,...,Fake,0.000002,1,1.0,0.858019,test,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4324,good_photos,NaN,G552008576,NaN,NaN,NaN,https://api.gobiz.co.id/v1/onboardings/data/SF...,NaN,NaN,NaN,...,Fake,0.999985,0,0.0,0.508325,test,0.0,0.0,NaN,NaN
4325,good_photos,NaN,G481206228,NaN,NaN,NaN,https://api.gobiz.co.id/v1/onboardings/data/SF...,NaN,NaN,NaN,...,Fake,0.764269,0,0.0,0.488000,val,0.0,0.0,NaN,NaN
4326,good_photos,NaN,G409804302,NaN,NaN,NaN,https://api.gobiz.co.id/v1/onboardings/data/SF...,NaN,NaN,NaN,...,Fake,0.999978,1,0.0,0.648707,test,0.0,1.0,NaN,NaN
4327,good_photos,NaN,G061053355,NaN,NaN,NaN,https://api.gobiz.co.id/v1/onboardings/data/SF...,NaN,NaN,NaN,...,Fake,0.999999,0,0.0,0.398159,val,0.0,0.0,NaN,NaN


In [3]:
DATA_PATH = '/Users/travistang/Documents/TorchScene/'

def download_image(url, merchant_id, folder_name):
    
    global numberImages 
    
    ## Set up the image URL and filename
    image_url = url

    new_path = f"{DATA_PATH}/{folder_name}/"
    file_path = new_path + merchant_id + '.jpeg'

    if os.path.exists(file_path):
        # print("Exists: ", file_name, '\n')
        return

    try:
        # Open the url image, set stream to True, this will return the stream content.
        r = requests.get(image_url, stream = True)

        # Check if the image was retrieved successfully
        if r.status_code == 200:
            # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            r.raw.decode_content = True

            if not os.path.exists(new_path):
                os.makedirs(new_path)

            # Open a local file with wb ( write binary ) permission.
            with open(file_path,'wb') as f:
                shutil.copyfileobj(r.raw, f)
                numberImages += 1

            #sleep for half second to prevent getting bumped out
            time.sleep(0.5)

            #  print('Downloaded: ',file_name, '\n')
        else:
              print('Image Couldn\'t be retreived')
                
    except Exception as e:
        print('Error Type: {0}\n'.format(e.__class__.__name__))
        print('Error Message: {0}\n'.format(e))
        
# delete images that do not exist
def test_and_delete(path):
    try:
        image = Image.open(path)
    
    except:
        os.remove(path)  
        print(f'\n{path} not valid and deleted.\n')
        

In [4]:

if __name__ == '__main__':
    numberImages = 0
    n_process = 4
    pool = Pool(n_process)
    start = datetime.now()

    url_sel_list = img_df[0:2]['outlet_photo_url']
    merchant_sel_list = img_df[0:2]['saudagar_id']
    folder_name = "victor_set"

    pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, repeat(folder_name)))
    duration = datetime.now() - start
    print(f'Trend calculation for {len(merchant_sel_list)} outlets completed in {duration}')

    pool.close()
    pool.join()

Process SpawnPoolWorker-2:
Process SpawnPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/travistang/opt/anaconda3/envs/torch-scene/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/travistang/opt/anaconda3/envs/torch-scene/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/travistang/opt/anaconda3/envs/torch-scene/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/travistang/opt/anaconda3/envs/torch-scene/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'download_image' on <module '__main__' (built-in)>
  File "/Users/travistang/opt/anaconda3/envs/torch-scene/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/travistang/opt/anaconda3/envs/torch-scen

KeyboardInterrupt: 

In [9]:
# no need to check fo image valididty for now
# # testing and deleting
# try:
#     # test if we can open a test image
#     Image.open('/home/jovyan/data-vol-1/photo/test.png')
#     n_process = 1
    
#     # if we can open the test image, we start deletion.
#     pool = Pool(n_process)
#     image_list = glob.glob('/home/jovyan/data-vol-1/similarity-project/data/maf/*')
#     pool.map(test_and_delete, image_list)
# except:
#     print("no images deleted. Check code for error.")

no images deleted. Check code for error.


In [4]:
#Load and show an image with Pillow
from PIL import Image

#Load the image
img = Image.open('/home/jovyan/data-vol-1/similarity-project/data/maf/G000006123/2c6c2e29-4323-4783-af19-e850bddfe736_GoFood-F-1433823873-1635682618302.jpeg')

#Get basic details about the image
print(img.format)
print(img.mode)
print(img.size)

#show the image
img.show()

JPEG
RGB
(500, 500)


In [ ]:
# read from CSV

In [6]:
df = pd.read_csv(f'{DATA_PATH}/maf/maf-data.csv')

In [7]:
df = df.drop_duplicates()
df['file_name'] = df['img'].str.split('/').str[-1]
df['item_uuid'] = df['file_name'].str[0:36]

In [8]:
import pickle
with open('/home/jovyan/data-vol-1/similarity-project/result/intermediate/resnet-transfer-learning-2/chosen-merchants.pickle', 'rb') as handle:
    selected_merchants = pickle.load(handle)

In [11]:
len(selected_merchants)

4255

In [12]:
df_selected_merchants

,saudagar_id,event_timestamp,order_number,img,file_name,item_uuid
0,G000006123,2021-10-31 12:15:47.986 UTC,F-1433816614,https://i.gojekapi.com/darkroom/food-merchant-...,07df8166-30d8-443f-a926-d83032497562_GoFood-F-...,07df8166-30d8-443f-a926-d83032497562
1,G000006123,2021-10-31 12:17:30.239 UTC,F-1433823873,https://i.gojekapi.com/darkroom/food-merchant-...,2c6c2e29-4323-4783-af19-e850bddfe736_GoFood-F-...,2c6c2e29-4323-4783-af19-e850bddfe736
2,G000006123,2021-11-14 14:37:57.515 UTC,F-1454006555,https://i.gojekapi.com/darkroom/food-merchant-...,6951b4e7-ed5c-4912-b88c-33b3e146d75c_GoFood-F-...,6951b4e7-ed5c-4912-b88c-33b3e146d75c
3,G000006123,2022-01-10 13:01:43.263 UTC,F-1538407704,https://i.gojekapi.com/darkroom/food-merchant-...,b45d7408-560d-48f6-b092-301e1c5250ea_GoFood-F-...,b45d7408-560d-48f6-b092-301e1c5250ea
4,G000006123,2022-01-10 13:01:43.263 UTC,F-1538407704,https://i.gojekapi.com/darkroom/food-merchant-...,389d7b31-d352-448a-ab60-25468b2a3cf4_GoFood-F-...,389d7b31-d352-448a-ab60-25468b2a3cf4
...,...,...,...,...,...,...
108803,G091235871,2021-11-25 15:37:58.076 UTC,F-1469028190,https://i.gojekapi.com/darkroom/food-merchant-...,0793ae26-0b8c-450f-83eb-798c5baab79c_GoFood-F-...,0793ae26-0b8c-450f-83eb-798c5baab79c
108804,G091235871,2021-12-03 15:34:02.053 UTC,F-1481221434,https://i.gojekapi.com/darkroom/food-merchant-...,ccdb8d2c-1901-405e-b464-db061c11fc26_GoFood-F-...,ccdb8d2c-1901-405e-b464-db061c11fc26
108805,G091235871,2021-12-03 15:57:12.018 UTC,F-1481227546,https://i.gojekapi.com/darkroom/food-merchant-...,6c55f2c0-6f65-490c-a2fd-76052d536e1b_GoFood-F-...,6c55f2c0-6f65-490c-a2fd-76052d536e1b
108806,G091235871,2021-12-10 14:41:04.97 UTC,F-1492327557,https://i.gojekapi.com/darkroom/food-merchant-...,299514e3-442a-4533-87f0-48b4cd2622f4_GoFood-F-...,299514e3-442a-4533-87f0-48b4cd2622f4


In [18]:
import glob
len(glob.glob("/home/jovyan/data-vol-1/similarity-project/data/maf/**/**"))

91832

In [9]:
df_selected_merchants = df[df['saudagar_id'].isin(selected_merchants)]
df_merchants = df[~df['saudagar_id'].isin(selected_merchants)]

In [10]:
url_sel_list = df_selected_merchants['img']
merchant_sel_list = df_selected_merchants['saudagar_id']
file_name_sel_list = df_selected_merchants['file_name']

In [ ]:
# Using multiprocessing starmap
# https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments#5443941

numberImages = 0

n_process = 4

pool = Pool(n_process)

start = datetime.now() 

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_ids)} outlets completed in {duration}')

pool.close()
pool.join()

In [58]:
numberImages

0

In [69]:
# download one missed image
row = df[df['file_name'] == "df1fcaaf-b115-4bbb-8bf8-90522d12371d_GoFood-F-1484638350-1638720952129.jpeg"]

In [71]:
df[df['file_name'] == "df1fcaaf-b115-4bbb-8bf8-90522d12371d_GoFood-F-1484638350-1638720952129.jpeg"].\
apply(lambda row: download_image(row['img'], row['saudagar_id'], row['file_name']), axis = 1)

8216    None
dtype: object

In [72]:
df[df['file_name'] == "df1fcaaf-b115-4bbb-8bf8-90522d12371d_GoFood-F-1484638350-1638720952129.jpeg"]

,saudagar_id,event_timestamp,order_number,img,file_name,item_uuid
8216,G005654386,2021-12-05 16:16:05.587 UTC,F-1484638350,https://i.gojekapi.com/darkroom/food-merchant-...,df1fcaaf-b115-4bbb-8bf8-90522d12371d_GoFood-F-...,df1fcaaf-b115-4bbb-8bf8-90522d12371d


In [63]:
df

,saudagar_id,event_timestamp,order_number,img,file_name,item_uuid
0,G000006123,2021-10-31 12:15:47.986 UTC,F-1433816614,https://i.gojekapi.com/darkroom/food-merchant-...,07df8166-30d8-443f-a926-d83032497562_GoFood-F-...,07df8166-30d8-443f-a926-d83032497562
1,G000006123,2021-10-31 12:17:30.239 UTC,F-1433823873,https://i.gojekapi.com/darkroom/food-merchant-...,2c6c2e29-4323-4783-af19-e850bddfe736_GoFood-F-...,2c6c2e29-4323-4783-af19-e850bddfe736
2,G000006123,2021-11-14 14:37:57.515 UTC,F-1454006555,https://i.gojekapi.com/darkroom/food-merchant-...,6951b4e7-ed5c-4912-b88c-33b3e146d75c_GoFood-F-...,6951b4e7-ed5c-4912-b88c-33b3e146d75c
3,G000006123,2022-01-10 13:01:43.263 UTC,F-1538407704,https://i.gojekapi.com/darkroom/food-merchant-...,b45d7408-560d-48f6-b092-301e1c5250ea_GoFood-F-...,b45d7408-560d-48f6-b092-301e1c5250ea
4,G000006123,2022-01-10 13:01:43.263 UTC,F-1538407704,https://i.gojekapi.com/darkroom/food-merchant-...,389d7b31-d352-448a-ab60-25468b2a3cf4_GoFood-F-...,389d7b31-d352-448a-ab60-25468b2a3cf4
...,...,...,...,...,...,...
1334702,M336978,2021-11-05 05:30:11.59 UTC,F-1440185692,https://i.gojekapi.com/darkroom/food-merchant-...,9235ed9c-c280-4405-b6ee-3cb055d9a6bf_GoFood-F-...,9235ed9c-c280-4405-b6ee-3cb055d9a6bf
1334703,M336981,2021-10-31 10:29:42.395 UTC,F-1433586927,https://i.gojekapi.com/darkroom/food-merchant-...,a0bcef5e-1945-4c1d-ad16-6f59aef75d7a_GoFood-F-...,a0bcef5e-1945-4c1d-ad16-6f59aef75d7a
1334704,M336984,2021-12-05 11:29:05.021 UTC,F-1484151491,https://i.gojekapi.com/darkroom/food-merchant-...,f641ce96-ea80-428f-9282-a8c7045019de_GoFood-F-...,f641ce96-ea80-428f-9282-a8c7045019de
1334705,M336984,2021-12-05 11:29:05.021 UTC,F-1484151491,https://i.gojekapi.com/darkroom/food-merchant-...,bf6a0c45-eaa4-4a96-bea1-c7e0115ad4b4_GoFood-F-...,bf6a0c45-eaa4-4a96-bea1-c7e0115ad4b4


In [77]:
# Downloading maf-approved-images
df_maf_approved = pd.read_csv(f'{DATA_PATH}/maf-approved-220122/maf-approved-imgs.csv')

In [78]:
df_maf_approved

,saudagar_id,event_timestamp,order_number,claim_status_name,claim_amount,img
0,G595385334,2022-01-13 00:35:45.515 UTC,F-1541342461,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...
1,G595385334,2022-01-13 15:54:48.032 UTC,F-1542586181,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...
2,G595385334,2022-01-13 17:58:37.674 UTC,F-1542618161,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...
3,G595385334,2022-01-13 19:50:23.607 UTC,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...
4,G595385334,2022-01-13 19:50:23.611 UTC,F-1542634453,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...
...,...,...,...,...,...,...
1550,M186207,2022-01-21 06:26:21.116 UTC,F-1552193381,APPROVED,10400,https://i.gojekapi.com/darkroom/food-merchant-...
1551,M186207,2022-01-21 06:27:31.106 UTC,F-1552196504,APPROVED,12400,https://i.gojekapi.com/darkroom/food-merchant-...
1552,M186207,2022-01-21 06:35:32.099 UTC,F-1552205591,APPROVED,42320,https://i.gojekapi.com/darkroom/food-merchant-...
1553,M186207,2022-01-21 06:38:43.099 UTC,F-1552211447,APPROVED,18360,https://i.gojekapi.com/darkroom/food-merchant-...


In [84]:
df_maf_approved = df_maf_approved.drop_duplicates()
df_maf_approved['file_name'] = df_maf_approved['img'].str.split('/').str[-1]
df_maf_approved['item_uuid'] = df_maf_approved['file_name'].str[0:36]

url_list = df_maf_approved['img']
merchant_list = df_maf_approved['saudagar_id']
file_name_list = df_maf_approved['file_name']
folder_name = "maf-approved-220122"

numberImages = 0
n_process = 4
pool = Pool(n_process)
start = datetime.now()

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_list)} outlets completed in {duration}')

pool.close()
pool.join()

Image Couldn't be retreived


NameError: name 'merchant_ids' is not defined

In [85]:
print(f'Trend calculation for {len(merchant_list)} outlets completed in {duration}')

Trend calculation for 1555 outlets completed in 0:04:43.547865


In [ ]:
url_list = df_merchants['img']
merchant_list = df_merchants['saudagar_id']
file_name_list = df_merchants['file_name']

In [ ]:
# Using multiprocessing starmap
# https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments#5443941

numberImages = 0

n_process = 4

pool = Pool(n_process)

start = datetime.now() 

pool.starmap(download_image, zip(url_list, merchant_list, file_name_list))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_ids)} outlets completed in {duration}')

pool.close()
pool.join()

In [ ]:
#download resnet-transfer-learning-5/merchants_with_most_images_df

In [37]:
import pickle
with open('/home/jovyan/data-vol-1/similarity-project/result/intermediate/resnet-transfer-learning-5/merchants_with_most_images.pickle', 'rb') as handle:
    merchants_with_most_images = pickle.load(handle)

In [38]:
len(merchants_with_most_images)

50

In [39]:
df

,saudagar_id,event_timestamp,order_number,img,file_name,item_uuid
0,G000006123,2021-10-31 12:15:47.986 UTC,F-1433816614,https://i.gojekapi.com/darkroom/food-merchant-...,07df8166-30d8-443f-a926-d83032497562_GoFood-F-...,07df8166-30d8-443f-a926-d83032497562
1,G000006123,2021-10-31 12:17:30.239 UTC,F-1433823873,https://i.gojekapi.com/darkroom/food-merchant-...,2c6c2e29-4323-4783-af19-e850bddfe736_GoFood-F-...,2c6c2e29-4323-4783-af19-e850bddfe736
2,G000006123,2021-11-14 14:37:57.515 UTC,F-1454006555,https://i.gojekapi.com/darkroom/food-merchant-...,6951b4e7-ed5c-4912-b88c-33b3e146d75c_GoFood-F-...,6951b4e7-ed5c-4912-b88c-33b3e146d75c
3,G000006123,2022-01-10 13:01:43.263 UTC,F-1538407704,https://i.gojekapi.com/darkroom/food-merchant-...,b45d7408-560d-48f6-b092-301e1c5250ea_GoFood-F-...,b45d7408-560d-48f6-b092-301e1c5250ea
4,G000006123,2022-01-10 13:01:43.263 UTC,F-1538407704,https://i.gojekapi.com/darkroom/food-merchant-...,389d7b31-d352-448a-ab60-25468b2a3cf4_GoFood-F-...,389d7b31-d352-448a-ab60-25468b2a3cf4
...,...,...,...,...,...,...
1334702,M336978,2021-11-05 05:30:11.59 UTC,F-1440185692,https://i.gojekapi.com/darkroom/food-merchant-...,9235ed9c-c280-4405-b6ee-3cb055d9a6bf_GoFood-F-...,9235ed9c-c280-4405-b6ee-3cb055d9a6bf
1334703,M336981,2021-10-31 10:29:42.395 UTC,F-1433586927,https://i.gojekapi.com/darkroom/food-merchant-...,a0bcef5e-1945-4c1d-ad16-6f59aef75d7a_GoFood-F-...,a0bcef5e-1945-4c1d-ad16-6f59aef75d7a
1334704,M336984,2021-12-05 11:29:05.021 UTC,F-1484151491,https://i.gojekapi.com/darkroom/food-merchant-...,f641ce96-ea80-428f-9282-a8c7045019de_GoFood-F-...,f641ce96-ea80-428f-9282-a8c7045019de
1334705,M336984,2021-12-05 11:29:05.021 UTC,F-1484151491,https://i.gojekapi.com/darkroom/food-merchant-...,bf6a0c45-eaa4-4a96-bea1-c7e0115ad4b4_GoFood-F-...,bf6a0c45-eaa4-4a96-bea1-c7e0115ad4b4


In [40]:
df_merchants_with_most_images_df = df[df['saudagar_id'].isin(merchants_with_most_images)]
df_non_merchants_with_most_images_df = df[~df['saudagar_id'].isin(merchants_with_most_images)]


In [41]:
df_merchants_with_most_images_df

,saudagar_id,event_timestamp,order_number,img,file_name,item_uuid
118061,G099056459,2021-08-18 20:15:46.014 UTC,F-1343552558,https://i.gojekapi.com/darkroom/food-merchant-...,6f399412-b7d7-42c5-9e0c-6cd1d61ecb7f_GoFood-F-...,6f399412-b7d7-42c5-9e0c-6cd1d61ecb7f
118062,G099056459,2021-08-18 20:56:12.35 UTC,F-1343540834,https://i.gojekapi.com/darkroom/food-merchant-...,e288135e-cbca-4c86-b2ef-ab24c7f26495_GoFood-F-...,e288135e-cbca-4c86-b2ef-ab24c7f26495
118063,G099056459,2021-08-19 10:46:33.01 UTC,F-1344158718,https://i.gojekapi.com/darkroom/food-merchant-...,e2415e08-ae70-480a-9cef-305579713e5c_GoFood-F-...,e2415e08-ae70-480a-9cef-305579713e5c
118064,G099056459,2021-08-19 10:47:37.468 UTC,F-1344187394,https://i.gojekapi.com/darkroom/food-merchant-...,d0152295-8bf9-4ac6-86a1-fcf8b76b052e_GoFood-F-...,d0152295-8bf9-4ac6-86a1-fcf8b76b052e
118065,G099056459,2021-08-19 10:47:37.468 UTC,F-1344187394,https://i.gojekapi.com/darkroom/food-merchant-...,cf77a894-6f45-4494-9a43-d2fe713e3e6c_GoFood-F-...,cf77a894-6f45-4494-9a43-d2fe713e3e6c
...,...,...,...,...,...,...
1327827,M316515,2022-01-07 20:52:59.374 UTC,F-1534364489,https://i.gojekapi.com/darkroom/food-merchant-...,f76f8a23-3935-4ca7-b0cb-cb5d571c22b1_GoFood-F-...,f76f8a23-3935-4ca7-b0cb-cb5d571c22b1
1327828,M316515,2022-01-08 21:29:01.201 UTC,F-1534772677,https://i.gojekapi.com/darkroom/food-merchant-...,1e833826-649e-4d26-8cb8-0aee83688e46_GoFood-F-...,1e833826-649e-4d26-8cb8-0aee83688e46
1327829,M316515,2022-01-08 21:29:22.102 UTC,F-1534368438,https://i.gojekapi.com/darkroom/food-merchant-...,ab3694df-d56a-46e0-85af-9221f9675150_GoFood-F-...,ab3694df-d56a-46e0-85af-9221f9675150
1327830,M316515,2022-01-11 22:51:55.357 UTC,F-1539976480,https://i.gojekapi.com/darkroom/food-merchant-...,2f20544c-c1d5-4882-951e-e0f085d5a859_GoFood-F-...,2f20544c-c1d5-4882-951e-e0f085d5a859


In [44]:
merchant_sel_list

Series([], Name: saudagar_id, dtype: object)

In [ ]:
url_sel_list = df_merchants_with_most_images_df['img']
merchant_sel_list = df_merchants_with_most_images_df['saudagar_id']
file_name_sel_list = df_merchants_with_most_images_df['file_name']
folder_name = "merchants-with-most-images-220207"

# Using multiprocessing starmap
# https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments#5443941

numberImages = 0

n_process = 4

pool = Pool(n_process)

start = datetime.now() 

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_sel_list)} outlets completed in {duration}')

pool.close()
pool.join()

Image Couldn't be retreived
Image Couldn't be retreived
Image Couldn't be retreived
Image Couldn't be retreived
Image Couldn't be retreived
Image Couldn't be retreived
Image Couldn't be retreived
Image Couldn't be retreived
Image Couldn't be retreived
Image Couldn't be retreived
Image Couldn't be retreived
Image Couldn't be retreived


In [134]:
# retrieving the images for Traditional Food Knowledge of Indonesia

import zipfile
cropped_file_path = '/home/jovyan/data-vol-1/similarity-project/data/fasilkim-lab-1231/dados/'

In [ ]:
with zipfile.ZipFile(cropped_file_path + "fasilkim-lab-1231_cropped_220414_1205.zip","r") as zip_ref:
    zip_ref.extractall(cropped_file_path)

In [135]:
import os
cropped_file_list = []
for dirname, subdirs, files in os.walk(cropped_file_path):     
    for filename in files:
        if '_crop.jpg' in filename: 
            cropped_file_list.append(dirname+'/'+filename)
        else:
            print('2.'+dirname+'/'+filename)

2./home/jovyan/data-vol-1/similarity-project/data/fasilkim-lab-1231/dados/data/fasilkim-lab-1231/dados/trad_food_sampled_12k_df_new.csv
2./home/jovyan/data-vol-1/similarity-project/data/fasilkim-lab-1231/dados/data/fasilkim-lab-1231/dados/_categories.csv
2./home/jovyan/data-vol-1/similarity-project/data/fasilkim-lab-1231/dados/data/fasilkim-lab-1231/dados/_categories.


# Generating fasilkim-lab-1231

In [171]:
import pandas as pd
fasilkim_df = pd.DataFrame(cropped_file_list, columns = ['filepath'])

In [172]:
fasilkim_df['cat'] = fasilkim_df['filepath'].str.split(pat='/',n=12,expand=True)[11]

In [173]:
unique_cat_len = (len(fasilkim_df['cat'].drop_duplicates()))

valid_cat = fasilkim_df['cat'].drop_duplicates().sample(int(unique_cat_len*0.1), random_state = 5)

fasilkim_df.loc[:,'category'] = 'train'
fasilkim_df.loc[fasilkim_df['cat'].isin(valid_cat),'category'] = 'validation'

DATA_PATH_TRAD_FOOD = f'{DATA_PATH}/fasilkim-lab-1231/dados/data/fasilkim-lab-1231/dados/'
fasilkim_df.to_csv(f'{DATA_PATH_TRAD_FOOD}_categories.csv')
fasilkim_df


,filepath,cat,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,Kunyit asam-20190514T093022Z-001,train
1,/home/jovyan/data-vol-1/similarity-project/dat...,Kunyit asam-20190514T093022Z-001,train
2,/home/jovyan/data-vol-1/similarity-project/dat...,Kunyit asam-20190514T093022Z-001,train
3,/home/jovyan/data-vol-1/similarity-project/dat...,Kunyit asam-20190514T093022Z-001,train
4,/home/jovyan/data-vol-1/similarity-project/dat...,Kunyit asam-20190514T093022Z-001,train
...,...,...,...
3036,/home/jovyan/data-vol-1/similarity-project/dat...,Mie Aceh-20190514T093609Z-001,train
3037,/home/jovyan/data-vol-1/similarity-project/dat...,Mie Aceh-20190514T093609Z-001,train
3038,/home/jovyan/data-vol-1/similarity-project/dat...,Mie Aceh-20190514T093609Z-001,train
3039,/home/jovyan/data-vol-1/similarity-project/dat...,Mie Aceh-20190514T093609Z-001,train


In [174]:
fasilkim_df.query('category == "validation"')

,filepath,cat,category
160,/home/jovyan/data-vol-1/similarity-project/dat...,kolak-20190514T092953Z-001,validation
161,/home/jovyan/data-vol-1/similarity-project/dat...,kolak-20190514T092953Z-001,validation
162,/home/jovyan/data-vol-1/similarity-project/dat...,kolak-20190514T092953Z-001,validation
163,/home/jovyan/data-vol-1/similarity-project/dat...,kolak-20190514T092953Z-001,validation
164,/home/jovyan/data-vol-1/similarity-project/dat...,kolak-20190514T092953Z-001,validation
...,...,...,...
2747,/home/jovyan/data-vol-1/similarity-project/dat...,bir pletok-20190514T071510Z-001,validation
2748,/home/jovyan/data-vol-1/similarity-project/dat...,bir pletok-20190514T071510Z-001,validation
2749,/home/jovyan/data-vol-1/similarity-project/dat...,bir pletok-20190514T071510Z-001,validation
2750,/home/jovyan/data-vol-1/similarity-project/dat...,bir pletok-20190514T071510Z-001,validation


In [153]:
fasilkim_df = fasilkim_df.drop(columns = ['category'])

In [10]:
fasilkim_df_pairs = fasilkim_df.merge(fasilkim_df, how = 'left', on = 'cat')

In [11]:
# need to deduplicate xy and yx
fasilkim_df_pairs['dedup_set'] = fasilkim_df_pairs.apply(lambda row: {row['filepath_x'], row['filepath_y']},axis=1)

In [15]:
fasilkim_df_pairs['filepath_x'][0]

'/home/jovyan/data-vol-1/similarity-project/data/fasilkim-lab-1231/dados/data/fasilkim-lab-1231/dados/Kunyit asam-20190514T093022Z-001/Kunyit asam/Camera/IMG_8595_crop.jpg'

In [16]:
# fasilkim_df_pairs['filepath_x'] = fasilkim_df_pairs.filepath_x.replace('../data/fasilkim-lab-1231/dados/', '', regex=True)

# the fasilkim_df_pairs has to be prepared by the image itself. 

KeyboardInterrupt: 

In [ ]:
# fasilkim_df_pairs['filepath_y'] = fasilkim_df_pairs.filepath_y.replace('../data/fasilkim-lab-1231/dados/', '', regex=True)

In [18]:
fasilkim_df_pairs_dedup = fasilkim_df_pairs['dedup_set'][~fasilkim_df_pairs['dedup_set'].astype(str).duplicated(keep='first')]

In [19]:
fasilkim_df_pairs_dedup_df = fasilkim_df_pairs.loc[fasilkim_df_pairs_dedup.index]

In [20]:
fasilkim_df_pairs_dedup_df = fasilkim_df_pairs_dedup_df.drop(columns=['dedup_set'])

In [22]:
fasilkim_df_pairs_dedup_df

,filepath_x,cat,filepath_y
0,/home/jovyan/data-vol-1/similarity-project/dat...,similarity-project,/home/jovyan/data-vol-1/similarity-project/dat...
1,/home/jovyan/data-vol-1/similarity-project/dat...,similarity-project,/home/jovyan/data-vol-1/similarity-project/dat...
2,/home/jovyan/data-vol-1/similarity-project/dat...,similarity-project,/home/jovyan/data-vol-1/similarity-project/dat...
3,/home/jovyan/data-vol-1/similarity-project/dat...,similarity-project,/home/jovyan/data-vol-1/similarity-project/dat...
4,/home/jovyan/data-vol-1/similarity-project/dat...,similarity-project,/home/jovyan/data-vol-1/similarity-project/dat...
...,...,...,...
9247642,/home/jovyan/data-vol-1/similarity-project/dat...,similarity-project,/home/jovyan/data-vol-1/similarity-project/dat...
9247666,/home/jovyan/data-vol-1/similarity-project/dat...,similarity-project,/home/jovyan/data-vol-1/similarity-project/dat...
9247673,/home/jovyan/data-vol-1/similarity-project/dat...,similarity-project,/home/jovyan/data-vol-1/similarity-project/dat...
9247674,/home/jovyan/data-vol-1/similarity-project/dat...,similarity-project,/home/jovyan/data-vol-1/similarity-project/dat...


In [57]:
fasilkim_df_pairs_dedup_df['filepath_x'][0]

NameError: name 'fasilkim_df_pairs_dedup_df' is not defined

In [101]:
PROJECT_ROOT_PATH = "/home/jovyan/data-vol-1/similarity-project"
DATA_PATH = f"{PROJECT_ROOT_PATH}/data"
DATA_PATH_TRAD_FOOD = f'{DATA_PATH}/fasilkim-lab-1231/dados/data/fasilkim-lab-1231/dados/'

trad_food_file_list = []
for dirname, subdirs, files in os.walk(DATA_PATH_TRAD_FOOD):     
    for filename in files:
        if '_crop.jpg' in filename: 
            trad_food_file_list.append(dirname+'/'+filename)
        else:
            print('2.'+dirname+'/'+filename)
            
fasilkim_df = pd.DataFrame(trad_food_file_list, columns = ['filepath'])
            
fasilkim_df['cat'] = fasilkim_df['filepath'].str.split(pat='/',n=12,expand=True)[11]
fasilkim_df_pairs = fasilkim_df.merge(fasilkim_df, how = 'left', on = 'cat')
# need to deduplicate xy and yx
fasilkim_df_pairs['dedup_set'] = fasilkim_df_pairs.apply(lambda row: {row['filepath_x'], row['filepath_y']},axis=1)
fasilkim_df_pairs_dedup = fasilkim_df_pairs['dedup_set'][~fasilkim_df_pairs['dedup_set'].astype(str).duplicated(keep='first')]
fasilkim_df_pairs_dedup_df = fasilkim_df_pairs.loc[fasilkim_df_pairs_dedup.index]
fasilkim_df_pairs_dedup_df = fasilkim_df_pairs_dedup_df.drop(columns=['dedup_set'])

fasilkim_df_pairs_dedup_df['label'] = True
trad_food_df = fasilkim_df_pairs_dedup_df

In [102]:
# stratified sampling based on categories
# We want ~ 5k pairs
trad_food_sampled_6k_df = trad_food_df.groupby('cat', group_keys=False).\
apply(lambda x: x.sample(min(len(x), 165), random_state = 5))
trad_food_sampled_6k_df

,filepath_x,cat,filepath_y,label
171438,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True
170074,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True
174064,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True
174738,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True
170397,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True
...,...,...,...,...
140736,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True
141999,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True
139851,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True
142000,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True


In [122]:
# finding negative pairs from fasilkim_df:
fasilkim_df['dummy'] = 1

In [123]:
fasilkim_df_neg_pairs = fasilkim_df.merge(fasilkim_df,on='dummy',suffixes =['_x','_y']).query('cat_x!=cat_y').drop(columns = ['dummy'])
fasilkim_df_neg_pairs['dedup_set'] = fasilkim_df_neg_pairs.apply(lambda row: {row['filepath_x'], row['filepath_y']},axis=1)
fasilkim_df_neg_pairs_dedup = fasilkim_df_neg_pairs['dedup_set'][~fasilkim_df_neg_pairs['dedup_set'].astype(str).duplicated(keep='first')]

In [124]:
fasilkim_df_neg_pairs_dedup_df = fasilkim_df_neg_pairs.loc[fasilkim_df_neg_pairs_dedup.index]
fasilkim_df_neg_pairs_dedup_df = fasilkim_df_neg_pairs_dedup_df.drop(columns=['dedup_set'])
fasilkim_df_neg_pairs_dedup_df['label'] = False
trad_food_neg_df = fasilkim_df_neg_pairs_dedup_df


In [125]:
trad_food_neg_sampled_6k_df = trad_food_neg_df.groupby('cat_x', group_keys=False).\
apply(lambda x: x.sample(min(len(x), 165), random_state = 5))
trad_food_neg_sampled_6k_df

,filepath_x,cat_x,filepath_y,cat_y,label
6258180,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Kue Adee-20190721T230259Z-001,False
6315431,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Gulai Ikan Mas Papua-20190721T225613Z-001,False
6181467,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Keladi Tumbuk-20190721T230048Z-001,False
6243002,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Otak Otak-20190721T233847Z-001,False
6224325,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Sayur Daun Singkong-20190721T230804Z-001,False
...,...,...,...,...,...
4992013,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Gado-Gado-20190514T093012Z-001,False
5007608,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Keladi Tumbuk-20190721T230048Z-001,False
5166489,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Otak Otak-20190721T233847Z-001,False
5019630,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Sate Lilit-20190514T093843Z-001,False


In [126]:
test_foods = ['Kue Adee-20190721T230259Z-001','Tekwan-20190721T224514Z-001','kerak telor-20190514T093015Z-001']

In [127]:
trad_food_sampled_6k_df['source'] = 'trad_food_sampled_6k'

# from sklearn.model_selection import train_test_split
# trad_food_sampled_train_df, trad_food_sampled_valid_df = train_test_split(trad_food_sampled_df, stratify = trad_food_sampled_df['cat'], test_size=0.07, random_state = 5)

#400 in valid
#5300 in train 

trad_food_neg_sampled_6k_df['source'] = 'trad_food_neg_sampled_6k'

In [129]:
trad_food_sampled_6k_df = trad_food_sampled_6k_df.rename(columns={'cat':'cat_x'})
trad_food_sampled_6k_df['cat_y']= trad_food_sampled_6k_df['cat_x']

In [133]:
trad_food_sampled_6k_df.loc[~trad_food_sampled_6k_df.cat_x.isin(test_foods),'category'] ='train'
trad_food_sampled_6k_df.loc[trad_food_sampled_6k_df.cat_x.isin(test_foods),'category'] ='validation'

trad_food_neg_sampled_6k_df.loc[:,'category'] ='train'

trad_food_neg_sampled_6k_df.loc[(trad_food_neg_sampled_6k_df['cat_x'].isin(test_foods))|
                                (trad_food_neg_sampled_6k_df['cat_y'].isin(test_foods)),
                                'category'] = 'validation'

In [134]:
trad_food_sampled_6k_df.columns = ['file_path_x','cat_x','file_path_y','label','source','category','cat_y']
trad_food_neg_sampled_6k_df.columns = ['file_path_x','cat_x','file_path_y','cat_y','label','source','category']

In [136]:
# check if cat_x is correctly labeled in train or validation
trad_food_neg_sampled_6k_df.groupby(['cat_x','category']).count()

file_path_x  \
cat_x                                     category                  
Ayam Bumbu Rujak-20190514T070822Z-001     train               150   
                                          validation           15   
Ayam Goreng Lengkuas-20190514T070827Z-001 train               158   
                                          validation            7   
Bika Ambon-20190514T071140Z-001           train               150   
...                                                           ...   
kerak telor-20190514T093015Z-001          validation          165   
kolak-20190514T092953Z-001                train               157   
                                          validation            8   
nasi goreng kampung-20190514T093636Z-001  train               150   
                                          validation           15   

                                                      file_path_y  cat_y  \
cat_x                                     category                         
Ayam Bumbu Rujak-20190514T070822Z-001     train               150    150   
                                          validation           15     15   
Ayam Goreng Lengkuas-20190514T070827Z-001 train               158    158   
                                          validation            7      7   
Bika Ambon-20190514T071140Z-001           train               150    150   
...                                                           ...    ...   
kerak telor-20190514T093015Z-001          validation          165    165   
kolak-20190514T092953Z-001                train               157    157   
                                          validation            8      8   
nasi goreng kampung-20190514T093636Z-001  train               150    150   
                                          validation           15     15   

                                                      label  source  
cat_x                                     category                   
Ayam Bumbu Rujak-20190514T070822Z-001     train         150     150  
                                          validation     15      15  
Ayam Goreng Lengkuas-20190514T070827Z-001 train         158     158  
                                          validation      7       7  
Bika Ambon-20190514T071140Z-001           train         150     150  
...                                                     ...     ...  
kerak telor-20190514T093015Z-001          validation    165     165  
kolak-20190514T092953Z-001                train         157     157  
                                          validation      8       8  
nasi goreng kampung-20190514T093636Z-001  train         150     150  
                                          validation     15      15  

[73 rows x 5 columns]

In [137]:
# test that categories that are from test are only in  validation
trad_food_neg_sampled_6k_df.loc[(trad_food_neg_sampled_6k_df['cat_x'].isin(test_foods)|
                                trad_food_neg_sampled_6k_df['cat_y'].isin(test_foods)),'category'].unique()
    

array(['validation'], dtype=object)

In [138]:
trad_food_neg_sampled_6k_df

,file_path_x,cat_x,file_path_y,cat_y,label,source,category
6258180,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Kue Adee-20190721T230259Z-001,False,trad_food_neg_sampled_6k,validation
6315431,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Gulai Ikan Mas Papua-20190721T225613Z-001,False,trad_food_neg_sampled_6k,train
6181467,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Keladi Tumbuk-20190721T230048Z-001,False,trad_food_neg_sampled_6k,train
6243002,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Otak Otak-20190721T233847Z-001,False,trad_food_neg_sampled_6k,train
6224325,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Sayur Daun Singkong-20190721T230804Z-001,False,trad_food_neg_sampled_6k,train
...,...,...,...,...,...,...,...
4992013,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Gado-Gado-20190514T093012Z-001,False,trad_food_neg_sampled_6k,train
5007608,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Keladi Tumbuk-20190721T230048Z-001,False,trad_food_neg_sampled_6k,train
5166489,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Otak Otak-20190721T233847Z-001,False,trad_food_neg_sampled_6k,train
5019630,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,Sate Lilit-20190514T093843Z-001,False,trad_food_neg_sampled_6k,train


In [139]:
trad_food_sampled_12k_df = pd.concat([trad_food_sampled_6k_df,trad_food_neg_sampled_6k_df])

In [140]:
trad_food_sampled_12k_df

,file_path_x,cat_x,file_path_y,label,source,category,cat_y
171438,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True,trad_food_sampled_6k,train,Ayam Bumbu Rujak-20190514T070822Z-001
170074,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True,trad_food_sampled_6k,train,Ayam Bumbu Rujak-20190514T070822Z-001
174064,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True,trad_food_sampled_6k,train,Ayam Bumbu Rujak-20190514T070822Z-001
174738,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True,trad_food_sampled_6k,train,Ayam Bumbu Rujak-20190514T070822Z-001
170397,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Bumbu Rujak-20190514T070822Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True,trad_food_sampled_6k,train,Ayam Bumbu Rujak-20190514T070822Z-001
...,...,...,...,...,...,...,...
4992013,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,False,trad_food_neg_sampled_6k,train,Gado-Gado-20190514T093012Z-001
5007608,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,False,trad_food_neg_sampled_6k,train,Keladi Tumbuk-20190721T230048Z-001
5166489,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,False,trad_food_neg_sampled_6k,train,Otak Otak-20190721T233847Z-001
5019630,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,False,trad_food_neg_sampled_6k,train,Sate Lilit-20190514T093843Z-001


In [142]:
trad_food_sampled_12k_df.to_csv(f'{DATA_PATH_TRAD_FOOD}/trad_food_sampled_12k_df_new.csv')


In [143]:
trad_food_sampled_12k_df.sample(10)

,file_path_x,cat_x,file_path_y,label,source,category,cat_y
4343,/home/jovyan/data-vol-1/similarity-project/dat...,Ayam Goreng Lengkuas-20190514T070827Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True,trad_food_sampled_6k,train,Ayam Goreng Lengkuas-20190514T070827Z-001
2739823,/home/jovyan/data-vol-1/similarity-project/dat...,Gudeg-20190721T223732Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,False,trad_food_neg_sampled_6k,train,Otak Otak-20190721T233847Z-001
4743901,/home/jovyan/data-vol-1/similarity-project/dat...,Cendol-20190514T075002Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,False,trad_food_neg_sampled_6k,train,Mie Aceh-20190514T093609Z-001
8298696,/home/jovyan/data-vol-1/similarity-project/dat...,bir pletok-20190514T071510Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,False,trad_food_neg_sampled_6k,validation,Kue Adee-20190721T230259Z-001
6614092,/home/jovyan/data-vol-1/similarity-project/dat...,Keladi Tumbuk-20190721T230048Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,False,trad_food_neg_sampled_6k,train,Mie Aceh-20190514T093609Z-001
5100534,/home/jovyan/data-vol-1/similarity-project/dat...,nasi goreng kampung-20190514T093636Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,False,trad_food_neg_sampled_6k,validation,Tekwan-20190721T224514Z-001
159851,/home/jovyan/data-vol-1/similarity-project/dat...,Es Dawet Ayu-20190514T092956Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,True,trad_food_sampled_6k,train,Es Dawet Ayu-20190514T092956Z-001
5196021,/home/jovyan/data-vol-1/similarity-project/dat...,Gado-Gado-20190514T093012Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,False,trad_food_neg_sampled_6k,train,Sate Lilit-20190514T093843Z-001
2964,/home/jovyan/data-vol-1/similarity-project/dat...,Papeda + Gulai Ikan Mas Papua-20190721T225613Z...,/home/jovyan/data-vol-1/similarity-project/dat...,True,trad_food_sampled_6k,train,Papeda + Gulai Ikan Mas Papua-20190721T225613Z...
7939244,/home/jovyan/data-vol-1/similarity-project/dat...,Sate Maranggi-20190721T224022Z-001,/home/jovyan/data-vol-1/similarity-project/dat...,False,trad_food_neg_sampled_6k,train,Gulai Ikan Mas Papua-20190721T225613Z-001


## Creating Sim4kv2-LowHash

In [ ]:
# Gathering data in the MAF Approved Set that were removed because they were highly similar.

In [35]:
# generated in resnet-transfer-learning-6
NOTEBOOK_NAME = "resnet-transfer-learning-6"
PROJECT_ROOT_PATH = "/home/jovyan/data-vol-1/similarity-project"
INTERMEDIATE_RESULTS_PATH = f"{PROJECT_ROOT_PATH}/result/intermediate/{NOTEBOOK_NAME}"

df_maf_approved_similar_dedup_df_w_identical = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH}/df_maf_approved_similar_dedup_df_w_identical.csv', index_col = 0)

In [36]:
df_maf_approved_similar_dedup_df_w_identical

,saudagar_id,event_timestamp_x,order_number_x,claim_status_name_x,claim_amount_x,img_x,file_name_x,item_uuid_x,file_path_x,hash_x,...,order_number_y,claim_status_name_y,claim_amount_y,img_y,file_name_y,item_uuid_y,file_path_y,hash_y,dedup_set,hash_distance
777,G595385334,2022-01-13 15:54:48.032000+00:00,F-1542586181,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,2861bfd1-81c6-48c9-a887-a2d3686f6d3c_GoFood-F-...,2861bfd1-81c6-48c9-a887-a2d3686f6d3c,/home/jovyan/data-vol-1/similarity-project/dat...,f2660c373116e1bd,...,F-1542618161,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,927d7ba1-954f-48b4-b475-1cd824b478d0_GoFood-F-...,927d7ba1-954f-48b4-b475-1cd824b478d0,/home/jovyan/data-vol-1/similarity-project/dat...,e1a133e09bd8c5b3,{'https://i.gojekapi.com/darkroom/food-merchan...,34
778,G595385334,2022-01-13 15:54:48.032000+00:00,F-1542586181,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,2861bfd1-81c6-48c9-a887-a2d3686f6d3c_GoFood-F-...,2861bfd1-81c6-48c9-a887-a2d3686f6d3c,/home/jovyan/data-vol-1/similarity-project/dat...,f2660c373116e1bd,...,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,e34ccce0-fb4f-4996-8167-e584d732ae93_GoFood-F-...,e34ccce0-fb4f-4996-8167-e584d732ae93,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6489c6fa516c2,{'https://i.gojekapi.com/darkroom/food-merchan...,32
779,G595385334,2022-01-13 15:54:48.032000+00:00,F-1542586181,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,2861bfd1-81c6-48c9-a887-a2d3686f6d3c_GoFood-F-...,2861bfd1-81c6-48c9-a887-a2d3686f6d3c,/home/jovyan/data-vol-1/similarity-project/dat...,f2660c373116e1bd,...,F-1542634453,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,7c6b8809-2f65-4099-8244-5bd1127bdd2c_GoFood-F-...,7c6b8809-2f65-4099-8244-5bd1127bdd2c,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6488c7fa1174a,{'https://i.gojekapi.com/darkroom/food-merchan...,32
780,G595385334,2022-01-13 15:54:48.032000+00:00,F-1542586181,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,2861bfd1-81c6-48c9-a887-a2d3686f6d3c_GoFood-F-...,2861bfd1-81c6-48c9-a887-a2d3686f6d3c,/home/jovyan/data-vol-1/similarity-project/dat...,f2660c373116e1bd,...,F-1542632399,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,6bc4aeea-bb5d-467b-a8ab-5d816a616a80_GoFood-F-...,6bc4aeea-bb5d-467b-a8ab-5d816a616a80,/home/jovyan/data-vol-1/similarity-project/dat...,f266489e639556b1,{'https://i.gojekapi.com/darkroom/food-merchan...,20
781,G595385334,2022-01-13 15:54:48.032000+00:00,F-1542586181,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,2861bfd1-81c6-48c9-a887-a2d3686f6d3c_GoFood-F-...,2861bfd1-81c6-48c9-a887-a2d3686f6d3c,/home/jovyan/data-vol-1/similarity-project/dat...,f2660c373116e1bd,...,F-1542633276,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,1259842b-8b49-4ff0-acef-bf2b50c78be7_GoFood-F-...,1259842b-8b49-4ff0-acef-bf2b50c78be7,/home/jovyan/data-vol-1/similarity-project/dat...,f266489e639556b1,{'https://i.gojekapi.com/darkroom/food-merchan...,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775984,M186207,2022-01-21 06:35:32.099000+00:00,F-1552205591,APPROVED,42320,https://i.gojekapi.com/darkroom/food-merchant-...,b26db808-c73d-4d5a-9e52-6574232ef8f8_GoFood-F-...,b26db808-c73d-4d5a-9e52-6574232ef8f8,/home/jovyan/data-vol-1/similarity-project/dat...,86523f3ce3726945,...,F-1552211447,APPROVED,18360,https://i.gojekapi.com/darkroom/food-merchant-...,396acc86-f275-411f-80e1-e96f2d0b4521_GoFood-F-...,396acc86-f275-411f-80e1-e96f2d0b4521,/home/jovyan/data-vol-1/similarity-project/dat...,a1056f3ce3bb848e,{'https://i.gojekapi.com/darkroom/food-merchan...,26
775985,M186207,2022-01-21 06:35:32.099000+00:00,F-1552205591,APPROVED,42320,https://i.gojekapi.com/darkroom/food-merchant-...,b26db808-c73d-4d5a-9e52-6574232ef8f8_GoFood-F-...,b26db808-c73d-4d5a-9e52-6574232ef8f8,/home/jovyan/data-vol-1/simi

In [37]:
df_maf_approved_similar_dedup_df_identical = df_maf_approved_similar_dedup_df_w_identical.query('hash_distance < 12')

In [41]:
df_maf_approved_similar_dedup_df_identical.to_csv(f'{INTERMEDIATE_RESULTS_PATH}/df_maf_approved_similar_dedup_df_w_only_identical.csv', index = True)

In [42]:
pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH}/df_maf_approved_similar_dedup_df_w_only_identical.csv')

,Unnamed: 0,saudagar_id,event_timestamp_x,order_number_x,claim_status_name_x,claim_amount_x,img_x,file_name_x,item_uuid_x,file_path_x,...,order_number_y,claim_status_name_y,claim_amount_y,img_y,file_name_y,item_uuid_y,file_path_y,hash_y,dedup_set,hash_distance
0,2329,G595385334,2022-01-13 19:50:23.607000+00:00,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,e34ccce0-fb4f-4996-8167-e584d732ae93_GoFood-F-...,e34ccce0-fb4f-4996-8167-e584d732ae93,/home/jovyan/data-vol-1/similarity-project/dat...,...,F-1542634453,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,7c6b8809-2f65-4099-8244-5bd1127bdd2c_GoFood-F-...,7c6b8809-2f65-4099-8244-5bd1127bdd2c,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6488c7fa1174a,{'https://i.gojekapi.com/darkroom/food-merchan...,6
1,2333,G595385334,2022-01-13 19:50:23.607000+00:00,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,e34ccce0-fb4f-4996-8167-e584d732ae93_GoFood-F-...,e34ccce0-fb4f-4996-8167-e584d732ae93,/home/jovyan/data-vol-1/similarity-project/dat...,...,F-1542635011,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,13520b11-c263-4d4a-98c7-1124493a8933_GoFood-F-...,13520b11-c263-4d4a-98c7-1124493a8933,/home/jovyan/data-vol-1/similarity-project/dat...,f2e4488c7fa1374a,{'https://i.gojekapi.com/darkroom/food-merchan...,8
2,2334,G595385334,2022-01-13 19:50:23.607000+00:00,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,e34ccce0-fb4f-4996-8167-e584d732ae93_GoFood-F-...,e34ccce0-fb4f-4996-8167-e584d732ae93,/home/jovyan/data-vol-1/similarity-project/dat...,...,F-1542631614,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,7a1384b7-c081-471d-ac30-60210b448673_GoFood-F-...,7a1384b7-c081-471d-ac30-60210b448673,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6488c7fa1174a,{'https://i.gojekapi.com/darkroom/food-merchan...,6
3,2336,G595385334,2022-01-13 19:50:23.607000+00:00,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,e34ccce0-fb4f-4996-8167-e584d732ae93_GoFood-F-...,e34ccce0-fb4f-4996-8167-e584d732ae93,/home/jovyan/data-vol-1/similarity-project/dat...,...,F-1542634488,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,166f6203-5848-48f6-910b-e82ad15ea082_GoFood-F-...,166f6203-5848-48f6-910b-e82ad15ea082,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6488c7fa1174a,{'https://i.gojekapi.com/darkroom/food-merchan...,6
4,2337,G595385334,2022-01-13 19:50:23.607000+00:00,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,e34ccce0-fb4f-4996-8167-e584d732ae93_GoFood-F-...,e34ccce0-fb4f-4996-8167-e584d732ae93,/home/jovyan/data-vol-1/similarity-project/dat...,...,F-1542634091,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,9f65a101-70a0-40f3-8ed7-df0209bfb6a0_GoFood-F-...,9f65a101-70a0-40f3-8ed7-df0209bfb6a0,/home/jovyan/data-vol-1/similarity-project/dat...,f2e4488c7fa1374a,{'https://i.gojekapi.com/darkroom/food-merchan...,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117748,754547,G985182270,2022-01-21 12:20:35.099000+00:00,F-1551939337,APPROVED,18880,https://i.gojekapi.com/darkroom/food-merchant-...,697e138b-d483-4cd8-ab30-ead02fc8036d_GoFood-F-...,697e138b-d483-4cd8-ab30-ead02fc8036d,/home/jovyan/data-vol-1/similarity-project/dat...,...,F-1551911745,APPROVED,9400,https://i.gojekapi.com/darkroom/food-merchant-...,39c64271-7131-4968-b05e-d783cf978e60_GoFood-F-...,39c64271-7131-4968-b05e-d783cf978e60,/home/jovyan/data-vol-1/similarity-project/dat...,a8f74b98b56d4330,{'https://i.gojekapi.com/darkroom/food-merchan...,8
117749,754550,G985182270,2022-01-21 12:20:35.099000+00:00,F-1551939337,APPROVED,18880,https://i.gojekapi.com/darkroom/food-merchant-...,697e138b-d483-4cd8-ab30-ead02fc8036d_GoFood-F-...,697e138b-d483-4cd8-ab30-ead02fc8036d,/home/jovyan/data-vol-1/similarity-project/dat...,...,F-1551917764,APPROVED,11400,https://i.gojeka

In [40]:
df_maf_approved_similar_dedup_df_identical

,saudagar_id,event_timestamp_x,order_number_x,claim_status_name_x,claim_amount_x,img_x,file_name_x,item_uuid_x,file_path_x,hash_x,...,order_number_y,claim_status_name_y,claim_amount_y,img_y,file_name_y,item_uuid_y,file_path_y,hash_y,dedup_set,hash_distance
2329,G595385334,2022-01-13 19:50:23.607000+00:00,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,e34ccce0-fb4f-4996-8167-e584d732ae93_GoFood-F-...,e34ccce0-fb4f-4996-8167-e584d732ae93,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6489c6fa516c2,...,F-1542634453,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,7c6b8809-2f65-4099-8244-5bd1127bdd2c_GoFood-F-...,7c6b8809-2f65-4099-8244-5bd1127bdd2c,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6488c7fa1174a,{'https://i.gojekapi.com/darkroom/food-merchan...,6
2333,G595385334,2022-01-13 19:50:23.607000+00:00,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,e34ccce0-fb4f-4996-8167-e584d732ae93_GoFood-F-...,e34ccce0-fb4f-4996-8167-e584d732ae93,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6489c6fa516c2,...,F-1542635011,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,13520b11-c263-4d4a-98c7-1124493a8933_GoFood-F-...,13520b11-c263-4d4a-98c7-1124493a8933,/home/jovyan/data-vol-1/similarity-project/dat...,f2e4488c7fa1374a,{'https://i.gojekapi.com/darkroom/food-merchan...,8
2334,G595385334,2022-01-13 19:50:23.607000+00:00,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,e34ccce0-fb4f-4996-8167-e584d732ae93_GoFood-F-...,e34ccce0-fb4f-4996-8167-e584d732ae93,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6489c6fa516c2,...,F-1542631614,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,7a1384b7-c081-471d-ac30-60210b448673_GoFood-F-...,7a1384b7-c081-471d-ac30-60210b448673,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6488c7fa1174a,{'https://i.gojekapi.com/darkroom/food-merchan...,6
2336,G595385334,2022-01-13 19:50:23.607000+00:00,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,e34ccce0-fb4f-4996-8167-e584d732ae93_GoFood-F-...,e34ccce0-fb4f-4996-8167-e584d732ae93,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6489c6fa516c2,...,F-1542634488,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,166f6203-5848-48f6-910b-e82ad15ea082_GoFood-F-...,166f6203-5848-48f6-910b-e82ad15ea082,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6488c7fa1174a,{'https://i.gojekapi.com/darkroom/food-merchan...,6
2337,G595385334,2022-01-13 19:50:23.607000+00:00,F-1542631380,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,e34ccce0-fb4f-4996-8167-e584d732ae93_GoFood-F-...,e34ccce0-fb4f-4996-8167-e584d732ae93,/home/jovyan/data-vol-1/similarity-project/dat...,f2e6489c6fa516c2,...,F-1542634091,APPROVED,80000,https://i.gojekapi.com/darkroom/food-merchant-...,9f65a101-70a0-40f3-8ed7-df0209bfb6a0_GoFood-F-...,9f65a101-70a0-40f3-8ed7-df0209bfb6a0,/home/jovyan/data-vol-1/similarity-project/dat...,f2e4488c7fa1374a,{'https://i.gojekapi.com/darkroom/food-merchan...,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754547,G985182270,2022-01-21 12:20:35.099000+00:00,F-1551939337,APPROVED,18880,https://i.gojekapi.com/darkroom/food-merchant-...,697e138b-d483-4cd8-ab30-ead02fc8036d_GoFood-F-...,697e138b-d483-4cd8-ab30-ead02fc8036d,/home/jovyan/data-vol-1/similarity-project/dat...,a0f74b98756d3234,...,F-1551911745,APPROVED,9400,https://i.gojekapi.com/darkroom/food-merchant-...,39c64271-7131-4968-b05e-d783cf978e60_GoFood-F-...,39c64271-7131-4968-b05e-d783cf978e60,/home/jovyan/data-vol-1/similarity-project/dat...,a8f74b98b56d4330,{'https://i.gojekapi.com/darkroom/food-merchan...,8
754550,G985182270,2022-01-21 12:20:35.099000+00:00,F-1551939337,APPROVED,18880,https://i.gojekapi.com/darkroom/food-merchant-...,697e138b-d483-4cd8-ab30-ead02fc8036d_GoFood-F-...,697e138b-d483-4cd8-ab30-ead02fc8036d,/home/jovyan/data-vol-1/

# sim4kv1-lowhash

In [ ]:
# Reusing images that are downloaded but were never used in the train/test set

In [47]:
import pickle
def pickle_data(file_path, file, read_or_write):
  if read_or_write == "write":
    # Store data (serialize)
    with open(file_path, 'wb') as handle:
        pickle.dump(file, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Load data (deserialize)
    with open(file_path, 'rb') as handle:
        unserialized_data = pickle.load(handle)

    assert(file == unserialized_data)

  if read_or_write == "read":
    with open(file_path, 'rb') as handle:
        unserialized_data = pickle.load(handle)
    return unserialized_data

In [48]:
# NOTEBOOK_NAME = "resnet-transfer-learning-6"
# PROJECT_ROOT_PATH = "/home/jovyan/data-vol-1/similarity-project"
# INTERMEDIATE_RESULTS_PATH_READ = f"{PROJECT_ROOT_PATH}/result/intermediate/resnet-transfer-learning-5"
# chosen_merchant = pickle_data(f'{INTERMEDIATE_RESULTS_PATH_READ}/chosen-merchants.pickle', None, 'read')
# subset_chosen_merchant = pickle_data(f'{INTERMEDIATE_RESULTS_PATH_READ}/subset_chosen_merchant.pickle',None,'read')

In [51]:
# extracted from resnet-transfer-learning-6
unused_merchant_similar_dedup_df = pickle_data(f'{INTERMEDIATE_RESULTS_PATH}/unused_merchant_similar_dedup_df.pickle', None, 'read')

In [68]:
unused_merchant_similar_dedup_low_hash_df = unused_merchant_similar_dedup_df.query("hash_distance < 14")

In [69]:
unused_merchant_similar_dedup_low_hash_df

,saudagar_id,event_timestamp_x,order_number_x,img_x,file_name_x,item_uuid_x,file_path_x,hash_x,event_timestamp_y,order_number_y,img_y,file_name_y,item_uuid_y,file_path_y,hash_y,hash_distance,dedup_set
2556,G000038994,2021-09-10 09:47:57.240000+00:00,F-1367436053,https://i.gojekapi.com/darkroom/food-merchant-...,8d4887b5-47f9-428d-a7dc-0a5812b78701_GoFood-F-...,8d4887b5-47f9-428d-a7dc-0a5812b78701,/home/jovyan/data-vol-1/similarity-project/dat...,85f7902c2fe8bc85,2021-11-07 06:58:15.026000+00:00,F-1443289162,https://i.gojekapi.com/darkroom/food-merchant-...,d1deba0d-b0f8-4ca5-9279-90f78d42705f_GoFood-F-...,d1deba0d-b0f8-4ca5-9279-90f78d42705f,/home/jovyan/data-vol-1/similarity-project/dat...,857f83a82fc87495,12,{https://i.gojekapi.com/darkroom/food-merchant...
5017,G000038994,2021-10-31 13:13:45.647000+00:00,F-1433707643,https://i.gojekapi.com/darkroom/food-merchant-...,da286aae-9b77-477b-af6d-a67fa24b2c97_GoFood-F-...,da286aae-9b77-477b-af6d-a67fa24b2c97,/home/jovyan/data-vol-1/similarity-project/dat...,ede6874113425afa,2021-12-10 11:20:35.723000+00:00,F-1490668703,https://i.gojekapi.com/darkroom/food-merchant-...,579cbed1-ae26-4c35-9bc5-b6fc6009fb8d_GoFood-F-...,579cbed1-ae26-4c35-9bc5-b6fc6009fb8d,/home/jovyan/data-vol-1/similarity-project/dat...,af66a6c1b1431a7a,12,{https://i.gojekapi.com/darkroom/food-merchant...
11544,G000038994,2021-11-12 04:19:45.223000+00:00,F-1449766131,https://i.gojekapi.com/darkroom/food-merchant-...,8c05086c-66b0-45ca-8ae7-21eeb4b4d666_GoFood-F-...,8c05086c-66b0-45ca-8ae7-21eeb4b4d666,/home/jovyan/data-vol-1/similarity-project/dat...,e5dee1ed486886a8,2021-11-12 04:20:42.076000+00:00,F-1449767011,https://i.gojekapi.com/darkroom/food-merchant-...,ed34106a-6a2d-4265-9d2b-1636bc930292_GoFood-F-...,ed34106a-6a2d-4265-9d2b-1636bc930292,/home/jovyan/data-vol-1/similarity-project/dat...,c5dee1cd486cc6a8,4,{https://i.gojekapi.com/darkroom/food-merchant...
20624,G000164234,2021-11-10 12:25:13.246000+00:00,F-1447945627,https://i.gojekapi.com/darkroom/food-merchant-...,a40dff7b-8ba9-437c-97cc-c400a03c7dd2_GoFood-F-...,a40dff7b-8ba9-437c-97cc-c400a03c7dd2,/home/jovyan/data-vol-1/similarity-project/dat...,93e92407f144f769,2021-11-10 12:25:13.246000+00:00,F-1447945627,https://i.gojekapi.com/darkroom/food-merchant-...,fe9c3594-d65e-4b94-bd53-a8b80a9d62ce_GoFood-F-...,fe9c3594-d65e-4b94-bd53-a8b80a9d62ce,/home/jovyan/data-vol-1/similarity-project/dat...,9149b403f844ff75,12,{https://i.gojekapi.com/darkroom/food-merchant...
26816,G000164960,2021-12-04 22:25:43.859000+00:00,F-1482967197,https://i.gojekapi.com/darkroom/food-merchant-...,afc4d223-13b7-4bae-8cb3-da4f5ecfbae0_GoFood-F-...,afc4d223-13b7-4bae-8cb3-da4f5ecfbae0,/home/jovyan/data-vol-1/similarity-project/dat...,ab8db151d94a8b4e,2021-12-04 22:27:10.138000+00:00,F-1482961513,https://i.gojekapi.com/darkroom/food-merchant-...,c59ab864-c2bf-42b6-874b-80fb759588a3_GoFood-F-...,c59ab864-c2bf-42b6-874b-80fb759588a3,/home/jovyan/data-vol-1/similarity-project/dat...,8b8d3d5ddb4a8382,12,{https://i.gojekapi.com/darkroom/food-merchant...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5376467,G091015137,2021-12-20 14:05:02.332000+00:00,F-1507908673,https://i.gojekapi.com/darkroom/food-merchant-...,6b5f0ccc-0994-44e4-855a-7edc2f99e5d6_GoFood-F-...,6b5f0ccc-0994-44e4-855a-7edc2f99e5d6,/home/jovyan/data-vol-1/similarity-project/dat...,c0177bf3023d1e74,2021-12-20 14:05:02.332000+00:00,F-1507908673,https://i.gojekapi.com/darkroom/food-merchant-...,5e5a8326-5973-45d4-8c48-1cbdeaac5ce5_GoFood-F-...,5e5a8326-5973-45d4-8c48-1cbdeaac5ce5,/home/jovyan/data-vol-1/similarity-project/dat...,c0137df2263e5ce4,12,{https://i.gojekapi.com/darkroom/food-merchant...
5376497,G091015137,2021-12-20 14:51:50.677000+00:00,F-1507916193,https://i.gojekapi.com/darkroom/food-merchant-...,40c44651-8654-4567-a4d2-8d9ae8a49dcc_GoFood-F-...,40c44651-8654-4567-a4d2-8d9ae8a49dcc,/home/jovyan/data-vol-1/similarity-project/dat...,bdc18f4f243619ac,2021-12-20 14:52:42.806000+00:00,F-1507

In [70]:
unused_merchant_similar_dedup_low_hash_df.to_csv(f'{INTERMEDIATE_RESULTS_PATH}/unused_merchant_similar_dedup_low_hash_df.csv', index = True)

In [71]:
print(unused_merchant_similar_dedup_low_hash_df[['img_x','img_y']].to_string())

                                                                                                                                                          img_x                                                                                                                                                   img_y
2556     https://i.gojekapi.com/darkroom/food-merchant-order-inbox/v2/images/public/8d4887b5-47f9-428d-a7dc-0a5812b78701_GoFood-F-1367436053-1631267223649.jpeg  https://i.gojekapi.com/darkroom/food-merchant-order-inbox/v2/images/public/d1deba0d-b0f8-4ca5-9279-90f78d42705f_GoFood-F-1443289162-1636268264936.jpeg
5017     https://i.gojekapi.com/darkroom/food-merchant-order-inbox/v2/images/public/da286aae-9b77-477b-af6d-a67fa24b2c97_GoFood-F-1433707643-1635686010703.jpeg  https://i.gojekapi.com/darkroom/food-merchant-order-inbox/v2/images/public/579cbed1-ae26-4c35-9bc5-b6fc6009fb8d_GoFood-F-1490668703-1639135227821.jpeg
11544    https://i.gojekapi.com/darkroom/food-merchant-order-inb

In [65]:
set(df_maf_approved_similar_dedup_df_identical.saudagar_id)

{'G595385334', 'G605675970', 'G651008492', 'G985182270', 'M186207'}

In [67]:
set(unused_merchant_similar_dedup_df.saudagar_id)

{'G049497724',
 'G088657260',
 'G018051633',
 'G034336141',
 'G031675922',
 'G064748945',
 'G023604905',
 'G064112143',
 'G007089882',
 'G046071642',
 'G025992667',
 'G025751775',
 'G064106289',
 'G012226381',
 'G010365494',
 'G005602446',
 'G012680303',
 'G061260574',
 'G027076548',
 'G008970524',
 'G083450691',
 'G060252884',
 'G002854725',
 'G013258337',
 'G089266344',
 'G089523915',
 'G022706810',
 'G079171704',
 'G055877745',
 'G017261046',
 'G078621778',
 'G077704376',
 'G047596970',
 'G081918065',
 'G085151289',
 'G057797671',
 'G032864508',
 'G011109711',
 'G036561462',
 'G011015934',
 'G060821186',
 'G026511693',
 'G001093415',
 'G024003889',
 'G049933291',
 'G006918629',
 'G000219262',
 'G014998337',
 'G070748579',
 'G039309719',
 'G073151348',
 'G029757356',
 'G034718951',
 'G057805266',
 'G065373370',
 'G086311951',
 'G086366969',
 'G068078726',
 'G067367011',
 'G025341221',
 'G062058156',
 'G033048190',
 'G016083483',
 'G046255767',
 'G047271192',
 'G053898995',
 'G0043628

# Creating PFID_Lab_Stillsv2


In [ ]:
# note that we completely removed PFID_Lab_Stills_croppedv1 since the cropping was poor in v1. 

In [123]:
import zipfile
import os
import glob

original_file_path = 'home/jovyan/data-vol-1/similarity-project/data/data/pfid/PFID_Lab_Stills/**'
cropped_file_path = '/home/jovyan/data-vol-1/similarity-project/data/'


In [27]:
#start with a clean slate. remove all v2 files
for file in glob.glob('/home/jovyan/data-vol-1/similarity-project/data/data/pfid/PFID_Lab_Stills/**/*_cropv2.jpg', recursive=True):
    os.remove(file)
    
for file in glob.glob('/home/jovyan/data-vol-1/similarity-project/data/PFID_Lab_Stills/**/*_cropv2.jpg', recursive=True):
    os.remove(file) 
    
#ectract zip
with zipfile.ZipFile(cropped_file_path + "PFID_Lab_Stills_croppedv2.zip","r") as zip_ref:
    zip_ref.extractall(cropped_file_path)
    
#Moved the extracted data to '/home/jovyan/data-vol-1/similarity-project/data/PFID_Lab_Stills'
for file in glob.glob('/home/jovyan/data-vol-1/similarity-project/data/data/pfid/PFID_Lab_Stills/**/*_cropv2.jpg', recursive=True):
    os.rename(file, file.replace('data/pfid/',''))

In [124]:
#get a list of files
cropped_file_list = []
for dirname, subdirs, files in os.walk(cropped_file_path + 'PFID_Lab_Stills/'):     
    for filename in files:
        if '_cropv2.jpg' in filename: 
                        cropped_file_list.append(dirname+'/'+filename)
#         else:
#             print('2.'+dirname+'/'+filename)

In [125]:
import pandas as pd
pfid_lab_df = pd.DataFrame(cropped_file_list, columns = ['file_path'])

In [126]:
pfid_lab_df.iloc[0,0]

'/home/jovyan/data-vol-1/similarity-project/data/PFID_Lab_Stills/3/29/images/still/inst 1/img_3289_cropv2.jpg'

In [127]:
pfid_lab_split_path_df = pfid_lab_df['file_path'].str.split(pat='/',n=12,expand=True)[[6,7,8,11]]

In [128]:
pfid_lab_split_path_df['cat'] = \
pfid_lab_split_path_df[6]+'_'+\
pfid_lab_split_path_df[7].astype(str)+'_'+\
pfid_lab_split_path_df[8].astype(str)+'_'+\
pfid_lab_split_path_df[11].astype(str)

In [129]:
#replace category that has the word "img in it"
pfid_lab_split_path_df['cat'] = pfid_lab_split_path_df['cat'].str.replace('img[\w\W]*','',regex=True)
pfid_lab_split_path_df['cat'] = pfid_lab_split_path_df['cat'].str.replace('IMG[\w\W]*','',regex=True)

In [130]:
pfid_lab_df['cat'] = pfid_lab_split_path_df['cat']

In [131]:
pfid_lab_df['cat'].unique()

array(['PFID_Lab_Stills_3_29_inst 1', 'PFID_Lab_Stills_3_29_inst 3',
       'PFID_Lab_Stills_3_29_inst 2', 'PFID_Lab_Stills_3_23_inst 1',
       'PFID_Lab_Stills_3_23_inst 3', 'PFID_Lab_Stills_3_23_inst 2',
       'PFID_Lab_Stills_3_33_inst 1', 'PFID_Lab_Stills_3_33_inst 3',
       'PFID_Lab_Stills_3_33_inst 2', 'PFID_Lab_Stills_3_32_inst 1',
       'PFID_Lab_Stills_3_32_inst 3', 'PFID_Lab_Stills_3_32_inst 2',
       'PFID_Lab_Stills_3_28_', 'PFID_Lab_Stills_3_22_inst 1',
       'PFID_Lab_Stills_3_22_inst 3', 'PFID_Lab_Stills_3_22_inst 2',
       'PFID_Lab_Stills_3_27_inst 1', 'PFID_Lab_Stills_3_27_inst 3',
       'PFID_Lab_Stills_3_27_inst 2', 'PFID_Lab_Stills_3_21_inst 1',
       'PFID_Lab_Stills_3_21_inst 3', 'PFID_Lab_Stills_3_21_inst 2',
       'PFID_Lab_Stills_3_30_inst 4', 'PFID_Lab_Stills_3_30_inst 1',
       'PFID_Lab_Stills_3_30_inst 3', 'PFID_Lab_Stills_3_30_inst 2',
       'PFID_Lab_Stills_3_31_inst 4', 'PFID_Lab_Stills_3_31_inst 1',
       'PFID_Lab_Stills_3_31_inst 3', 'P

In [132]:
pfid_lab_df

,file_path,cat
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1
4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1
...,...,...
2169,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2
2170,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2
2171,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2
2172,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2


In [157]:
unique_cat_len = (len(pfid_lab_df['cat'].drop_duplicates()))

valid_cat = pfid_lab_df['cat'].drop_duplicates().sample(int(unique_cat_len*0.1), random_state = 5)

pfid_lab_df.loc[:,'category'] = 'train'
pfid_lab_df.loc[pfid_lab_df['cat'].isin(valid_cat),'category'] = 'validation'

pfid_lab_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/PFID_Lab_Stills/pfid_lab_categories.csv')

pfid_lab_df

,file_path,cat,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,train
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,train
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,train
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,train
4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,train
...,...,...,...
2169,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2,train
2170,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2,train
2171,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2,train
2172,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2,train


In [155]:
pfid_lab_df.query("category == 'validation'")

,file_path,cat,category
60,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_32_inst 3,validation
61,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_32_inst 3,validation
62,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_32_inst 3,validation
63,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_32_inst 3,validation
64,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_32_inst 3,validation
...,...,...,...
1881,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_12_98_inst 1,validation
1882,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_12_98_inst 1,validation
1883,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_12_98_inst 1,validation
1884,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_12_98_inst 1,validation


In [156]:

pfid_lab_df = pfid_lab_df.drop(columns = ['category'])

In [83]:
# Get negative pairs
pfid_lab_df['key'] = 1

pfid_lab_pairs_neg = pfid_lab_df.merge(pfid_lab_df, how = 'left', on = 'key')

pfid_lab_pairs_neg = pfid_lab_pairs_neg.query('cat_x != cat_y')

pfid_lab_pairs_neg['dedup_set'] = pfid_lab_pairs_neg.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

pfid_lab_pairs_neg_dedup = pfid_lab_pairs_neg['dedup_set'][~pfid_lab_pairs_neg['dedup_set'].astype(str).duplicated(keep='first')]

pfid_lab_pairs_neg_dedup_df = pfid_lab_pairs_neg.loc[pfid_lab_pairs_neg_dedup.index]

pfid_lab_pairs_neg_dedup_df['label'] = False

pfid_lab_pairs_neg_dedup_df = pfid_lab_pairs_neg_dedup_df.drop(columns=['dedup_set'])

In [84]:
pfid_lab_pairs_neg_dedup_sampled_df = pfid_lab_pairs_neg_dedup_df.groupby('cat_x', group_keys=False).\
apply(lambda x: x.sample(min(len(x), 25), random_state = 5)) 

# a higher number of negative samples than positive samples to make up

In [85]:
pfid_lab_pairs_neg_dedup_sampled_df = pfid_lab_pairs_neg_dedup_sampled_df.drop(columns = ['key'])

In [86]:
pfid_lab_pairs_neg_dedup_sampled_df

,file_path_x,cat_x,file_path_y,cat_y,label
689063,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_10_78_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_7_57_inst 3,False
690511,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_10_78_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_6_46_inst 3,False
688669,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_10_78_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_13_102_inst 3,False
694592,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_10_78_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_11_90_inst 3,False
694510,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_10_78_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_11_89_inst 1,False
...,...,...,...,...,...
2839381,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_9_77_inst 4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_21_inst 1,False
2836427,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_9_77_inst 4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_2_18_inst 2,False
2828652,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_9_77_inst 4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_10_79_inst 2,False
2841131,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_9_77_inst 4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_12_100_inst 4,False


In [87]:
pfid_lab_pairs_neg_dedup_sampled_df.columns = ['file_path_x', 'cat_x', 'file_path_y','cat_y', 'label']

In [88]:
pfid_lab_pairs_pos = pfid_lab_df.merge(pfid_lab_df, how = 'left', on = 'cat')

pfid_lab_pairs_pos = pfid_lab_pairs_pos.drop(columns=['key_x', 'key_y'])

pfid_lab_pairs_pos['dedup_set'] = pfid_lab_pairs_pos.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

pfid_lab_pairs_pos_dedup = pfid_lab_pairs_pos['dedup_set'][~pfid_lab_pairs_pos['dedup_set'].astype(str).duplicated(keep='first')]

pfid_lab_pairs_pos_dedup_df = pfid_lab_pairs_pos.loc[pfid_lab_pairs_pos_dedup.index]

pfid_lab_pairs_pos_dedup_df['label'] = True

pfid_lab_pairs_pos_dedup_df = pfid_lab_pairs_pos_dedup_df.drop(columns=['dedup_set'])

In [91]:
pfid_lab_pairs_pos_dedup_df = pfid_lab_pairs_pos_dedup_df.rename(columns = {'cat':'cat_x'})
pfid_lab_pairs_pos_dedup_df['cat_y'] = pfid_lab_pairs_pos_dedup_df['cat_x']

In [92]:
pfid_lab_pairs_dedup_v2_df = pd.concat([pfid_lab_pairs_pos_dedup_df,pfid_lab_pairs_neg_dedup_sampled_df])

In [93]:
pfid_lab_pairs_dedup_v2_df

,file_path_x,cat_x,file_path_y,label,cat_y
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_3_29_inst 1
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_3_29_inst 1
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_3_29_inst 1
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_3_29_inst 1
4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_3_29_inst 1
...,...,...,...,...,...
2839381,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_9_77_inst 4,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_3_21_inst 1
2836427,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_9_77_inst 4,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_2_18_inst 2
2828652,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_9_77_inst 4,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_10_79_inst 2
2841131,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_9_77_inst 4,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_12_100_inst 4


In [17]:
# need to deduplicate xy and yx
# pfid_lab_pairs['dedup_set'] = pfid_lab_pairs.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

In [ ]:
# perform some random matches to generate random negative pairs...

In [18]:
# pfid_lab_pairs['file_path_x'][0]

'/home/jovyan/data-vol-1/similarity-project/data/PFID_Lab_Stills/3/29/images/still/inst 1/img_3286_crop.jpg'

In [19]:
# pfid_lab_pairs_dedup = pfid_lab_pairs['dedup_set'][~pfid_lab_pairs['dedup_set'].astype(str).duplicated(keep='first')]

In [20]:
# pfid_lab_pairs_dedup_df = pfid_lab_pairs.loc[pfid_lab_pairs_dedup.index]

In [21]:
# pfid_lab_pairs_dedup_df = pfid_lab_pairs_dedup_df.drop(columns=['dedup_set'])

In [22]:
# pfid_lab_pairs_dedup_df

,file_path_x,cat,file_path_y
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...
4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_29_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...
...,...,...,...
17300,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...
17305,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...
17306,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...
17307,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_4_34_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...


In [95]:
pfid_lab_pairs_dedup_v2_df['cat_x'].drop_duplicates().sample(frac=1)

2919        PFID_Lab_Stills_1_3_inst 2
5529       PFID_Lab_Stills_8_61_inst 3
6761      PFID_Lab_Stills_11_95_inst 3
4349       PFID_Lab_Stills_5_38_inst 2
10612    PFID_Lab_Stills_13_102_inst 1
                     ...              
3508       PFID_Lab_Stills_1_11_inst 3
12258      PFID_Lab_Stills_7_51_inst 3
4806       PFID_Lab_Stills_5_45_inst 3
13598      PFID_Lab_Stills_4_34_inst 3
12294      PFID_Lab_Stills_7_51_inst 2
Name: cat_x, Length: 355, dtype: object

In [97]:
# length = len(pfid_lab_pairs_dedup_v2_df) # 16102

# pfid_lab_pairs_dedup_v2_df['source'] = 'pfid_restaurant_2k'
# pfid_lab_pairs_dedup_v2_df['category'] = 'train'
# pfid_lab_pairs_dedup_v2_df = pfid_lab_pairs_dedup_v2_df.sample(frac=1, random_state = 5).reset_index(drop=True)

# pfid_lab_pairs_dedup_v2_df.loc[0:12880,'category'] = 'train'
# pfid_lab_pairs_dedup_v2_df.loc[12881:14491,'category'] = 'validation'
# pfid_lab_pairs_dedup_v2_df.loc[14492:16101,'category'] = 'test'

# to be replaced by this:

train_cat = pfid_lab_pairs_dedup_v2_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[0:284]
test_cat = pfid_lab_pairs_dedup_v2_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[284:319]
valid_cat = pfid_lab_pairs_dedup_v2_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[319:]

pfid_lab_pairs_dedup_v2_df['source'] = 'pfid_lab_16k'
pfid_lab_pairs_dedup_v2_df = pfid_lab_pairs_dedup_v2_df.sample(frac=1, random_state = 5).reset_index(drop=True)
    
pfid_lab_pairs_dedup_v2_df.loc[:,'category'] = 'train'

pfid_lab_pairs_dedup_v2_df.loc[(pfid_lab_pairs_dedup_v2_df['cat_x'].isin(test_cat)|
                                      pfid_lab_pairs_dedup_v2_df['cat_y'].isin(test_cat)),'category'] = 'test'
    
pfid_lab_pairs_dedup_v2_df.loc[(pfid_lab_pairs_dedup_v2_df['cat_x'].isin(valid_cat)|
                                      pfid_lab_pairs_dedup_v2_df['cat_y'].isin(valid_cat)),'category'] = 'validation'


In [98]:
pfid_lab_pairs_dedup_v2_df

,file_path_x,cat_x,file_path_y,label,cat_y,source,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_6_48_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_12_96_inst 5,pfid_lab_16k,train
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_5_43_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_8_60_inst 2,pfid_lab_16k,validation
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_5_35_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_58_inst 2,pfid_lab_16k,validation
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_7_58_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_58_inst 2,pfid_lab_16k,test
4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_8_66_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_2_15_inst 2,pfid_lab_16k,train
...,...,...,...,...,...,...,...
17323,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_21_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_3_21_inst 1,pfid_lab_16k,train
17324,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_26_inst 3,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_3_26_inst 3,pfid_lab_16k,test
17325,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_6_48_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_6_48_inst 2,pfid_lab_16k,validation
17326,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_5_36_inst 3,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_5_36_inst 3,pfid_lab_16k,train


In [99]:
pfid_lab_pairs_dedup_v2_df.groupby(['category','label']).count()


file_path_x  cat_x  file_path_y  cat_y  source
category   label                                                
test       False         1570   1570         1570   1570    1570
           True           782    782          782    782     782
train      False         5849   5849         5849   5849    5849
           True          6855   6855         6855   6855    6855
validation False         1456   1456         1456   1456    1456
           True           816    816          816    816     816

In [100]:
pfid_lab_pairs_dedup_v2_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/PFID_Lab_Stills/pfid_lab_pairs_dedup_v2_new_df.csv')
# pfid_lab_pairs_dedup_v2_new_df is the corrected version of pfid_lab_pairs_dedup_v2_df

In [181]:
# There's a probem with v2: images in test set appear in train before.

pfid_lab_pairs_dedup_v2_df = pd.read_csv('/home/jovyan/data-vol-1/similarity-project/data/PFID_Lab_Stills/pfid_lab_pairs_dedup_v2_new_df.csv', index_col = [0])
# pfid_lab_pairs_dedup_v2_new_df is the corrected version of pfid_lab_pairs_dedup_v2_df

unique_cat = pd.concat([pfid_lab_pairs_dedup_v2_df['cat_x'],
                        pfid_lab_pairs_dedup_v2_df['cat_y']]).drop_duplicates()

train_cat = unique_cat.sample(frac=1, random_state = 5)[0:319]
# test_cat = unique_cat.sample(frac=1, random_state = 5)[284:319]
valid_cat = unique_cat.sample(frac=1, random_state = 5)[319:]

pfid_lab_pairs_dedup_v2_df['source'] = 'pfid_lab_16k'
pfid_lab_pairs_dedup_v2_df = pfid_lab_pairs_dedup_v2_df.sample(frac=1, random_state = 5).reset_index(drop=True)
    
# pfid_lab_pairs_dedup_v2_df.loc[(pfid_lab_pairs_dedup_v2_df['cat_x'].isin(test_cat)|
#                                       pfid_lab_pairs_dedup_v2_df['cat_y'].isin(test_cat)),'category'] = 'test'
    
# pfid_lab_pairs_dedup_v2_df.loc[(pfid_lab_pairs_dedup_v2_df['cat_x'].isin(train_cat)|
#                                 pfid_lab_pairs_dedup_v2_df['cat_y'].isin(train_cat)),'category'] = 'train'
pfid_lab_pairs_dedup_v2_df['category'] = 'train'

pfid_lab_pairs_dedup_v2_df.loc[(pfid_lab_pairs_dedup_v2_df['cat_x'].isin(valid_cat)&
                                pfid_lab_pairs_dedup_v2_df['cat_y'].isin(valid_cat)),'category'] = 'validation'

pfid_lab_pairs_dedup_v2_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/PFID_Lab_Stills/pfid_lab_pairs_dedup_v3_new_df.csv')
# pfid_lab_pairs_dedup_v2_new_df is the corrected version of pfid_lab_pairs_dedup_v2_df

In [177]:
train_cat[train_cat == "PFID_Lab_Stills_7_52_inst 6"]

Series([], dtype: object)

In [182]:
valid_cat[valid_cat == "PFID_Lab_Stills_7_52_inst 6"]

336    PFID_Lab_Stills_7_52_inst 6
dtype: object

In [183]:
pfid_lab_pairs_dedup_v2_df[pfid_lab_pairs_dedup_v2_df['file_path_y'] == "/home/jovyan/data-vol-1/similarity-project/data/PFID_Lab_Stills/7/52/images/still/inst 6/img_1733_cropv2.jpg"]

,file_path_x,cat_x,file_path_y,label,cat_y,source,category
916,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_12_96_inst 7,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_52_inst 6,pfid_lab_16k,train
2224,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_12_100_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_52_inst 6,pfid_lab_16k,train
3870,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_2_18_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_52_inst 6,pfid_lab_16k,train
4092,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_9_69_inst 3,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_52_inst 6,pfid_lab_16k,train
4196,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_7_52_inst 6,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_7_52_inst 6,pfid_lab_16k,validation
4250,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_13_106_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_52_inst 6,pfid_lab_16k,train
5857,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_13_102_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_52_inst 6,pfid_lab_16k,train
7217,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_2_17_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_52_inst 6,pfid_lab_16k,train
9274,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_9_70_inst 3,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_52_inst 6,pfid_lab_16k,train
9530,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_3_26_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_52_inst 6,pfid_lab_16k,train


In [119]:
pfid_lab_pairs_dedup_v2_df

,file_path_x,cat_x,file_path_y,label,cat_y,source,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_8_60_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_8_60_inst 1,pfid_lab_16k,train
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_11_88_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_59_inst 2,pfid_lab_16k,train
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_5_43_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_5_43_inst 2,pfid_lab_16k,train
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_12_100_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_52_inst 6,pfid_lab_16k,validation
4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_11_92_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_11_92_inst 1,pfid_lab_16k,train
...,...,...,...,...,...,...,...
17323,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_7_50_inst 3,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Lab_Stills_7_50_inst 3,pfid_lab_16k,train
17324,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_8_60_inst 4,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_7_59_inst 3,pfid_lab_16k,train
17325,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_13_107_inst 3,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_13_106_inst 3,pfid_lab_16k,train
17326,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Lab_Stills_1_10_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Lab_Stills_9_75_inst 1,pfid_lab_16k,train


In [120]:
pfid_lab_pairs_dedup_v2_df.groupby(['category','label']).count()

file_path_x  cat_x  file_path_y  cat_y  source
category   label                                                
train      False         7227   7227         7227   7227    7227
           True          7606   7606         7606   7606    7606
validation False         1648   1648         1648   1648    1648
           True           847    847          847    847     847

# Create PFID Restaurant Set

In [112]:
import zipfile
import os

cropped_file_path = '/home/jovyan/data-vol-1/similarity-project/data/'

In [ ]:
with zipfile.ZipFile(cropped_file_path + "PFID_Restaurant_Stills_cropped.zip","r") as zip_ref:
    zip_ref.extractall(cropped_file_path)

In [113]:
cropped_file_list = []
#MANUAL ACTION NEEDED: #Moved the extracted data to '/home/jovyan/data-vol-1/similarity-project/data/PFID_Lab_Stills'
for dirname, subdirs, files in os.walk(cropped_file_path + 'PFID_Restaurant_Stills/'):     
    for filename in files:
        if '_crop.jpg' in filename: 
            cropped_file_list.append(dirname+'/'+filename)
#         else:
#             print('2.'+dirname+'/'+filename)

In [114]:
import pandas as pd
pfid_restaurant_df = pd.DataFrame(cropped_file_list, columns = ['file_path'])


In [115]:
pfid_restaurant_df

,file_path
0,/home/jovyan/data-vol-1/similarity-project/dat...
1,/home/jovyan/data-vol-1/similarity-project/dat...
2,/home/jovyan/data-vol-1/similarity-project/dat...
3,/home/jovyan/data-vol-1/similarity-project/dat...
4,/home/jovyan/data-vol-1/similarity-project/dat...
...,...
491,/home/jovyan/data-vol-1/similarity-project/dat...
492,/home/jovyan/data-vol-1/similarity-project/dat...
493,/home/jovyan/data-vol-1/similarity-project/dat...
494,/home/jovyan/data-vol-1/similarity-project/dat...


In [116]:
pfid_restaurant_split_path_df = pfid_restaurant_df['file_path'].str.split(pat='/',n=12,expand=True)[[6,7,8,11]]

In [117]:
pfid_restaurant_split_path_df['cat'] = \
pfid_restaurant_split_path_df[6]+'_'+\
pfid_restaurant_split_path_df[7].astype(str)+'_'+\
pfid_restaurant_split_path_df[8].astype(str)+'_'+\
pfid_restaurant_split_path_df[11].astype(str)

In [118]:
#replace category that has the word "img in it"
pfid_restaurant_split_path_df['cat'] = pfid_restaurant_split_path_df['cat'].str.replace('img[\w\W]*','',regex=True)
pfid_restaurant_split_path_df['cat'] = pfid_restaurant_split_path_df['cat'].str.replace('IMG[\w\W]*','',regex=True)

In [119]:
pfid_restaurant_split_path_df['cat'].unique()

array(['PFID_Restaurant_Stills_10_79_inst 1',
       'PFID_Restaurant_Stills_10_79_inst 3',
       'PFID_Restaurant_Stills_10_79_inst 2',
       'PFID_Restaurant_Stills_10_81_inst 1',
       'PFID_Restaurant_Stills_10_81_inst 3',
       'PFID_Restaurant_Stills_10_81_inst 2',
       'PFID_Restaurant_Stills_10_84_inst 1',
       'PFID_Restaurant_Stills_10_84_inst 3',
       'PFID_Restaurant_Stills_10_84_inst 2',
       'PFID_Restaurant_Stills_10_78_inst 1',
       'PFID_Restaurant_Stills_10_85_inst 1',
       'PFID_Restaurant_Stills_10_85_inst 3',
       'PFID_Restaurant_Stills_10_85_inst 2',
       'PFID_Restaurant_Stills_10_80_inst 1',
       'PFID_Restaurant_Stills_10_80_inst 3',
       'PFID_Restaurant_Stills_10_80_inst 2',
       'PFID_Restaurant_Stills_10_82_inst 1',
       'PFID_Restaurant_Stills_10_82_inst 3',
       'PFID_Restaurant_Stills_10_82_inst 2',
       'PFID_Restaurant_Stills_10_83_inst 1',
       'PFID_Restaurant_Stills_10_83_inst 3',
       'PFID_Restaurant_Stills_10_

In [120]:
pfid_restaurant_df['cat'] = pfid_restaurant_split_path_df['cat']

In [121]:
pfid_restaurant_df

,file_path,cat
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1
4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 3
...,...,...
491,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 2
492,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5
493,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5
494,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5


In [158]:
unique_cat_len = (len(pfid_restaurant_df['cat'].drop_duplicates()))

valid_cat = pfid_restaurant_df['cat'].drop_duplicates().sample(int(unique_cat_len*0.1), random_state = 5)

# pfid_restaurant_df['source'] = 'pfid_restaurant'

pfid_restaurant_df.loc[:,'category'] = 'train'
pfid_restaurant_df.loc[pfid_restaurant_df['cat'].isin(valid_cat),'category'] = 'validation'

pfid_restaurant_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/PFID_Restaurant_Stills/pfid_restaurant_categories.csv')

pfid_restaurant_df



,file_path,cat,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,train
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,train
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,train
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,train
4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 3,train
...,...,...,...
491,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 2,train
492,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5,train
493,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5,train
494,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5,train


In [159]:

pfid_restaurant_df.query('category == "validation"')


,file_path,cat,category
32,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_84_inst 1,validation
33,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_84_inst 1,validation
34,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_84_inst 1,validation
35,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_84_inst 1,validation
48,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_85_inst 1,validation
49,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_85_inst 1,validation
50,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_85_inst 1,validation
51,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_85_inst 1,validation
76,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_82_inst 3,validation
77,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_82_inst 3,validation


In [160]:

pfid_restaurant_df = pfid_restaurant_df.drop(columns = ['category'])


In [53]:
# Get positive pairs
pfid_restaurant_pairs_pos = pfid_restaurant_df.merge(pfid_restaurant_df, how = 'left', on = 'cat')

pfid_restaurant_pairs_pos['dedup_set'] = pfid_restaurant_pairs_pos.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

pfid_restaurant_pairs_pos_dedup = pfid_restaurant_pairs_pos['dedup_set'][~pfid_restaurant_pairs_pos['dedup_set'].astype(str).duplicated(keep='first')]

pfid_restaurant_pairs_pos_dedup_df = pfid_restaurant_pairs_pos.loc[pfid_restaurant_pairs_pos_dedup.index]

pfid_restaurant_pairs_pos_dedup_df['label'] = True

pfid_restaurant_pairs_pos_dedup_df = pfid_restaurant_pairs_pos_dedup_df.drop(columns=['dedup_set'])


In [54]:
pfid_restaurant_pairs_pos_dedup_df['cat_y'] = pfid_restaurant_pairs_pos_dedup_df['cat']
pfid_restaurant_pairs_pos_dedup_df = pfid_restaurant_pairs_pos_dedup_df.rename(columns = {'cat':'cat_x'})

In [55]:
pfid_restaurant_pairs_pos_dedup_df

,file_path_x,cat_x,file_path_y,label,cat_y
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_10_79_inst 1
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_10_79_inst 1
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_10_79_inst 1
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_10_79_inst 1
5,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_10_79_inst 1
...,...,...,...,...,...
2416,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_12_99_inst 5
2417,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_12_99_inst 5
2420,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_12_99_inst 5
2421,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_12_99_inst 5


In [56]:
# Get negative pairs
pfid_restaurant_df['key'] = 1

pfid_restaurant_pairs_neg = pfid_restaurant_df.merge(pfid_restaurant_df, how = 'left', on = 'key')

pfid_restaurant_pairs_neg = pfid_restaurant_pairs_neg.query('cat_x != cat_y')

pfid_restaurant_pairs_neg['dedup_set'] = pfid_restaurant_pairs_neg.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

pfid_restaurant_pairs_neg_dedup = pfid_restaurant_pairs_neg['dedup_set'][~pfid_restaurant_pairs_neg['dedup_set'].astype(str).duplicated(keep='first')]

pfid_restaurant_pairs_neg_dedup_df = pfid_restaurant_pairs_neg.loc[pfid_restaurant_pairs_neg_dedup.index]

pfid_restaurant_pairs_neg_dedup_df['label'] = False

pfid_restaurant_pairs_neg_dedup_df = pfid_restaurant_pairs_neg_dedup_df.drop(columns=['dedup_set'])



In [57]:
pfid_restaurant_pairs_neg_dedup_sampled_df = pfid_restaurant_pairs_neg_dedup_df.groupby('cat_x', group_keys=False).\
apply(lambda x: x.sample(min(len(x), 11), random_state = 5))

pfid_restaurant_pairs_neg_dedup_sampled_df = pfid_restaurant_pairs_neg_dedup_sampled_df.drop(columns = ['key'])

pfid_restaurant_pairs_neg_dedup_sampled_df

pfid_restaurant_pairs_neg_dedup_sampled_df.columns = ['file_path_x', 'cat_x', 'file_path_y', 'cat_y','label']


In [58]:
pfid_restaurant_pairs_neg_dedup_sampled_df

,file_path_x,cat_x,file_path_y,cat_y,label
21987,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_78_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_1_8_inst 2,False
21955,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_78_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_1_1_inst 1,False
21902,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_78_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_82_inst 3,False
22254,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_78_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_96_inst 6 (open),False
22532,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_78_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_1_7_inst 2,False
...,...,...,...,...,...
120017,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_5_43_,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 4,False
120503,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_5_43_,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_100_inst 5,False
115369,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_5_43_,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_11_89_inst 2,False
119491,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_5_43_,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_98_inst 1,False


In [59]:
pfid_restaurant_pairs_dedup_v2_df = pd.concat([pfid_restaurant_pairs_pos_dedup_df,pfid_restaurant_pairs_neg_dedup_sampled_df])

In [60]:
pfid_restaurant_pairs_dedup_v2_df

,file_path_x,cat_x,file_path_y,label,cat_y
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_10_79_inst 1
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_10_79_inst 1
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_10_79_inst 1
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_10_79_inst 1
5,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_10_79_inst 1
...,...,...,...,...,...
120017,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_5_43_,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Restaurant_Stills_12_99_inst 4
120503,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_5_43_,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Restaurant_Stills_12_100_inst 5
115369,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_5_43_,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Restaurant_Stills_11_89_inst 2
119491,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_5_43_,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Restaurant_Stills_12_98_inst 1


In [36]:
# pfid_restaurant_pairs = pfid_restaurant_df.merge(pfid_restaurant_df, how = 'left', on = 'cat')

In [39]:
# # need to deduplicate xy and yx
# pfid_restaurant_pairs['dedup_set'] = pfid_restaurant_pairs.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

In [41]:
# pfid_restaurant_pairs_dedup = pfid_restaurant_pairs['dedup_set'][~pfid_restaurant_pairs['dedup_set'].astype(str).duplicated(keep='first')]

In [42]:
# pfid_restaurant_pairs_dedup_df = pfid_restaurant_pairs.loc[pfid_restaurant_pairs_dedup.index]

In [43]:
# pfid_restaurant_pairs_dedup_df = pfid_restaurant_pairs_dedup_df.drop(columns=['dedup_set'])

In [44]:
# pfid_restaurant_pairs_dedup_df

,file_path_x,cat,file_path_y
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...
5,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_79_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...
...,...,...,...
2176,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5,/home/jovyan/data-vol-1/similarity-project/dat...
2177,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5,/home/jovyan/data-vol-1/similarity-project/dat...
2180,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5,/home/jovyan/data-vol-1/similarity-project/dat...
2181,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_99_inst 5,/home/jovyan/data-vol-1/similarity-project/dat...


In [64]:
pfid_restaurant_pairs_dedup_v2_df['cat_x'].drop_duplicates()

0      PFID_Restaurant_Stills_11_95_inst 3
1      PFID_Restaurant_Stills_11_90_inst 2
2      PFID_Restaurant_Stills_10_81_inst 2
3      PFID_Restaurant_Stills_10_79_inst 3
4        PFID_Restaurant_Stills_1_1_inst 2
                      ...                 
379    PFID_Restaurant_Stills_11_92_inst 3
427    PFID_Restaurant_Stills_10_83_inst 3
435    PFID_Restaurant_Stills_11_92_inst 1
498    PFID_Restaurant_Stills_11_92_inst 2
633    PFID_Restaurant_Stills_12_99_inst 1
Name: cat_x, Length: 113, dtype: object

In [65]:

# pfid_restaurant_pairs_dedup_v2_df['source'] = 'pfid_restaurant_2k'
# pfid_restaurant_pairs_dedup_v2_df['category'] = 'train'
# pfid_restaurant_pairs_dedup_v2_df = pfid_restaurant_pairs_dedup_v2_df.sample(frac=1, random_state = 5).reset_index(drop=True)

# length = len(pfid_restaurant_pairs_dedup_v2_df) # 1947
# pfid_restaurant_pairs_dedup_v2_df.loc[:1500,'category'] = 'train'
# pfid_restaurant_pairs_dedup_v2_df.loc[1501:1750,'category'] = 'validation'
# pfid_restaurant_pairs_dedup_v2_df.loc[1751:1947,'category'] = 'test'


# to be replaced by this:

train_cat = pfid_restaurant_pairs_dedup_v2_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[0:90]
test_cat = pfid_restaurant_pairs_dedup_v2_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[91:101]
valid_cat = pfid_restaurant_pairs_dedup_v2_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[101:113]

pfid_restaurant_pairs_dedup_v2_df['source'] = 'pfid_restaurant_2k'
pfid_restaurant_pairs_dedup_v2_df = pfid_restaurant_pairs_dedup_v2_df.sample(frac=1, random_state = 5).reset_index(drop=True)

pfid_restaurant_pairs_dedup_v2_df.loc[:,'category'] = 'train'

pfid_restaurant_pairs_dedup_v2_df.loc[(pfid_restaurant_pairs_dedup_v2_df['cat_x'].isin(test_cat)|
                                      pfid_restaurant_pairs_dedup_v2_df['cat_y'].isin(test_cat)),'category'] = 'test'
    
pfid_restaurant_pairs_dedup_v2_df.loc[(pfid_restaurant_pairs_dedup_v2_df['cat_x'].isin(valid_cat)|
                                      pfid_restaurant_pairs_dedup_v2_df['cat_y'].isin(valid_cat)),'category'] = 'validation'


In [66]:
pfid_restaurant_pairs_dedup_v2_df

,file_path_x,cat_x,file_path_y,label,cat_y,source,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_1_10_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_1_10_inst 2,pfid_restaurant_2k,train
1,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_11_87_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_11_87_inst 1,pfid_restaurant_2k,train
2,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_12_96_inst 7,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_12_96_inst 7,pfid_restaurant_2k,train
3,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_81_inst 3,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_10_81_inst 3,pfid_restaurant_2k,train
4,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_1_8_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Restaurant_Stills_12_100_inst 3,pfid_restaurant_2k,train
...,...,...,...,...,...,...,...
2794,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_82_inst 1,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Restaurant_Stills_1_5_inst 2,pfid_restaurant_2k,train
2795,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_83_inst 3,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Restaurant_Stills_11_92_inst 3,pfid_restaurant_2k,test
2796,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_11_88_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,True,PFID_Restaurant_Stills_11_88_inst 2,pfid_restaurant_2k,train
2797,/home/jovyan/data-vol-1/similarity-project/dat...,PFID_Restaurant_Stills_10_84_inst 2,/home/jovyan/data-vol-1/similarity-project/dat...,False,PFID_Restaurant_Stills_11_91_inst 1,pfid_restaurant_2k,train


In [67]:
pfid_restaurant_pairs_dedup_v2_df.groupby(['category','label']).count()
# not an even split between test and validation, but good even

file_path_x  cat_x  file_path_y  cat_y  source
category   label                                                
test       False          141    141          141    141     141
           True           110    110          110    110     110
train      False          852    852          852    852     852
           True          1321   1321         1321   1321    1321
validation False          250    250          250    250     250
           True           125    125          125    125     125

In [69]:
# test that categories that are from test are only in test and validation
pfid_restaurant_pairs_dedup_v2_df.loc[(pfid_restaurant_pairs_dedup_v2_df['cat_x'].isin(test_cat)|
                                      pfid_restaurant_pairs_dedup_v2_df['cat_y'].isin(test_cat)),'category'].unique()
    

array(['test', 'validation'], dtype=object)

In [72]:
# test that categories that are from validation are only in validation
pfid_restaurant_pairs_dedup_v2_df.loc[(pfid_restaurant_pairs_dedup_v2_df['cat_x'].isin(valid_cat)|
                                      pfid_restaurant_pairs_dedup_v2_df['cat_y'].isin(valid_cat)),'category'].unique()
    

array(['validation'], dtype=object)

In [68]:
pfid_restaurant_pairs_dedup_v2_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/PFID_Restaurant_Stills/pfid_restaurant_pairs_dedup_v2_df_new.csv')

# pfid_restaurant_pairs_dedup_v2_df_new is a version of pfid_restaurant_pairs_dedup_v2_df that includes cat_y for neg pairs

In [174]:
# there is a problem with pfid_restaurant_pairs_dedup_v2_df
# images that appear in test set also appear in train set.
pfid_restaurant_pairs_dedup_v2_df = pd.read_csv('/home/jovyan/data-vol-1/similarity-project/data/PFID_Restaurant_Stills/pfid_restaurant_pairs_dedup_v2_df_new.csv', index_col = [0])

# to be replaced by this:

unique_cat = pd.concat([pfid_restaurant_pairs_dedup_v2_df['cat_x'],
                        pfid_restaurant_pairs_dedup_v2_df['cat_y']]).drop_duplicates()

train_cat = unique_cat.sample(frac=1, random_state = 5)[0:90]
# test_cat = unique_cat.sample(frac=1, random_state = 5)[91:101]
valid_cat = unique_cat.sample(frac=1, random_state = 5)[101:113]

pfid_restaurant_pairs_dedup_v2_df['source'] = 'pfid_restaurant_2k'
pfid_restaurant_pairs_dedup_v2_df = pfid_restaurant_pairs_dedup_v2_df.sample(frac=1, random_state = 5).reset_index(drop=True)

pfid_restaurant_pairs_dedup_v2_df.loc[:,'category'] = 'train'

# pfid_restaurant_pairs_dedup_v2_df.loc[(pfid_restaurant_pairs_dedup_v2_df['cat_x'].isin(test_cat)|
#                                       pfid_restaurant_pairs_dedup_v2_df['cat_y'].isin(test_cat)),'category'] = 'test'
    
pfid_restaurant_pairs_dedup_v2_df.loc[(pfid_restaurant_pairs_dedup_v2_df['cat_x'].isin(valid_cat)&
                                      pfid_restaurant_pairs_dedup_v2_df['cat_y'].isin(valid_cat)),'category'] = 'validation'

# pfid_restaurant_pairs_dedup_v2_df.loc[(pfid_restaurant_pairs_dedup_v2_df['cat_x'].isin(train_cat)|
#                                       pfid_restaurant_pairs_dedup_v2_df['cat_y'].isin(train_cat)),'category'] = 'train'

pfid_restaurant_pairs_dedup_v2_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/PFID_Restaurant_Stills/pfid_restaurant_pairs_dedup_v3_df_new.csv')


In [175]:
pfid_restaurant_pairs_dedup_v2_df.groupby(['category','label']).count()
# not an even split between test and validation, but good even

file_path_x  cat_x  file_path_y  cat_y  source
category   label                                                
train      False         1231   1231         1231   1231    1231
           True          1393   1393         1393   1393    1393
validation False           12     12           12     12      12
           True           163    163          163    163     163

# Unict-FD1200

In [102]:
import zipfile
import os
import glob

cropped_file_path = '/home/jovyan/data-vol-1/similarity-project/data/'
# with zipfile.ZipFile(cropped_file_path + "unict-fd1200_cropped.zip","r") as zip_ref:
#     zip_ref.extractall(cropped_file_path)
    
cropped_file_list = []


In [ ]:

#Moved the extracted data to '/home/jovyan/data-vol-1/similarity-project/data/PFID_Lab_Stills'
for file in glob.glob('/home/jovyan/data-vol-1/similarity-project/data/data/unict-fd1200/UNICT-FD1200_Small/*_crop.jpg', recursive=True):
    os.rename(file, file.replace('data/unict-fd1200/UNICT-FD1200_Small/','UNICT-FD1200/'))


In [103]:

for dirname, subdirs, files in os.walk(cropped_file_path + 'UNICT-FD1200'):     
    for filename in files:
        if '_crop.jpg' in filename: 
            cropped_file_list.append(dirname+'/'+filename)
#         else:
#             print('2.'+dirname+'/'+filename)

In [104]:
import pandas as pd
unict_df = pd.DataFrame(cropped_file_list, columns = ['file_path'])


In [105]:
unict_df

,file_path
0,/home/jovyan/data-vol-1/similarity-project/dat...
1,/home/jovyan/data-vol-1/similarity-project/dat...
2,/home/jovyan/data-vol-1/similarity-project/dat...
3,/home/jovyan/data-vol-1/similarity-project/dat...
4,/home/jovyan/data-vol-1/similarity-project/dat...
...,...
4745,/home/jovyan/data-vol-1/similarity-project/dat...
4746,/home/jovyan/data-vol-1/similarity-project/dat...
4747,/home/jovyan/data-vol-1/similarity-project/dat...
4748,/home/jovyan/data-vol-1/similarity-project/dat...


In [106]:
unict_df.loc[0,'file_path']

'/home/jovyan/data-vol-1/similarity-project/data/UNICT-FD1200/_Sample_0560_0002_crop.jpg'

In [107]:
unict_df['name'] = unict_df['file_path'].str.split(pat='/',n=7,expand=True)[7]

In [108]:
unict_df

,file_path,name
0,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0560_0002_crop.jpg
1,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0810_0004_crop.jpg
2,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0671_0002_crop.jpg
3,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0926_0002_crop.jpg
4,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_1191_0004_crop.jpg
...,...,...
4745,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0220_0001_crop.jpg
4746,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0839_0006_crop.jpg
4747,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0698_0004_crop.jpg
4748,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0643_0001_crop.jpg


In [109]:
unict_df['cat'] = unict_df['name'].str.split(pat = '_', n=3, expand=True)[2]

In [110]:
unict_df

,file_path,name,cat
0,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0560_0002_crop.jpg,0560
1,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0810_0004_crop.jpg,0810
2,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0671_0002_crop.jpg,0671
3,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0926_0002_crop.jpg,0926
4,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_1191_0004_crop.jpg,1191
...,...,...,...
4745,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0220_0001_crop.jpg,0220
4746,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0839_0006_crop.jpg,0839
4747,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0698_0004_crop.jpg,0698
4748,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0643_0001_crop.jpg,0643


In [162]:
unique_cat_len = (len(unict_df['cat'].drop_duplicates()))

valid_cat = unict_df['cat'].drop_duplicates().sample(int(unique_cat_len*0.1), random_state = 5)

# unict_df['source'] = 'unict'

# unict_df = unict_df.drop(columns = ['name'])
unict_df.loc[:,'category'] = 'train'
unict_df.loc[unict_df['cat'].isin(valid_cat),'category'] = 'validation'

unict_df.drop(columns = ['name']).to_csv('/home/jovyan/data-vol-1/similarity-project/data/UNICT-FD1200/unict_categories.csv') # for triplet

unict_df

,file_path,name,cat,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0560_0002_crop.jpg,0560,train
1,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0810_0004_crop.jpg,0810,train
2,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0671_0002_crop.jpg,0671,train
3,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0926_0002_crop.jpg,0926,validation
4,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_1191_0004_crop.jpg,1191,train
...,...,...,...,...
4745,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0220_0001_crop.jpg,0220,train
4746,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0839_0006_crop.jpg,0839,validation
4747,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0698_0004_crop.jpg,0698,train
4748,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0643_0001_crop.jpg,0643,train


In [163]:
unict_df.query('category == "validation"')

,file_path,name,cat,category
3,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0926_0002_crop.jpg,0926,validation
11,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0701_0002_crop.jpg,0701,validation
15,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0678_0002_crop.jpg,0678,validation
24,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0606_0003_crop.jpg,0606,validation
29,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_1193_0003_crop.jpg,1193,validation
...,...,...,...,...
4669,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0496_0002_crop.jpg,0496,validation
4670,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0392_0003_crop.jpg,0392,validation
4707,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0505_0001_crop.jpg,0505,validation
4720,/home/jovyan/data-vol-1/similarity-project/dat...,_Sample_0467_0006_crop.jpg,0467,validation


In [164]:
unict_df = unict_df.drop(columns = ['category'])

In [240]:
# Get positive pairs
unict_pairs_pos = unict_df.merge(unict_df, how = 'left', on = 'cat')

# unict_pairs_pos = unict_pairs_pos.drop(columns=['key_x', 'key_y'])

unict_pairs_pos['dedup_set'] = unict_pairs_pos.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

unict_pairs_pos_dedup = unict_pairs_pos['dedup_set'][~unict_pairs_pos['dedup_set'].astype(str).duplicated(keep='first')]

unict_pairs_pos_dedup_df = unict_pairs_pos.loc[unict_pairs_pos_dedup.index]

unict_pairs_pos_dedup_df['label'] = True

unict_pairs_pos_dedup_df = unict_pairs_pos_dedup_df.drop(columns=['dedup_set','name_x','name_y','key_x','key_y'])


In [241]:
unict_pairs_pos_dedup_df

,file_path_x,cat,file_path_y,label
0,/home/jovyan/data-vol-1/similarity-project/dat...,0560,/home/jovyan/data-vol-1/similarity-project/dat...,True
1,/home/jovyan/data-vol-1/similarity-project/dat...,0560,/home/jovyan/data-vol-1/similarity-project/dat...,True
2,/home/jovyan/data-vol-1/similarity-project/dat...,0560,/home/jovyan/data-vol-1/similarity-project/dat...,True
3,/home/jovyan/data-vol-1/similarity-project/dat...,0560,/home/jovyan/data-vol-1/similarity-project/dat...,True
4,/home/jovyan/data-vol-1/similarity-project/dat...,0560,/home/jovyan/data-vol-1/similarity-project/dat...,True
...,...,...,...,...
21644,/home/jovyan/data-vol-1/similarity-project/dat...,0220,/home/jovyan/data-vol-1/similarity-project/dat...,True
21650,/home/jovyan/data-vol-1/similarity-project/dat...,0839,/home/jovyan/data-vol-1/similarity-project/dat...,True
21655,/home/jovyan/data-vol-1/similarity-project/dat...,0698,/home/jovyan/data-vol-1/similarity-project/dat...,True
21659,/home/jovyan/data-vol-1/similarity-project/dat...,0643,/home/jovyan/data-vol-1/similarity-project/dat...,True


In [226]:
# Get negative pairs
unict_df['key'] = 1

unict_pairs_neg = unict_df.merge(unict_df, how = 'left', on = 'key')

unict_pairs_neg = unict_pairs_neg.query('cat_x != cat_y')

unict_pairs_neg['dedup_set'] = unict_pairs_neg.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

unict_pairs_neg_dedup = unict_pairs_neg['dedup_set'][~unict_pairs_neg['dedup_set'].astype(str).duplicated(keep='first')]

unict_pairs_neg_dedup_df = unict_pairs_neg.loc[unict_pairs_neg_dedup.index]

unict_pairs_neg_dedup_df['label'] = False

unict_pairs_neg_dedup_df = unict_pairs_neg_dedup_df.drop(columns=['dedup_set'])



In [229]:
unict_pairs_neg_dedup_sampled_df = unict_pairs_neg_dedup_df.groupby('cat_x', group_keys=False).\
apply(lambda x: x.sample(min(len(x), 11), random_state = 5))

unict_pairs_neg_dedup_sampled_df = unict_pairs_neg_dedup_sampled_df.drop(columns = ['key','cat_y','name_x','name_y'])

unict_pairs_neg_dedup_sampled_df

unict_pairs_neg_dedup_sampled_df.columns = ['file_path_x', 'cat', 'file_path_y', 'label']


In [233]:
unict_pairs_neg_dedup_sampled_df

,file_path_x,cat,file_path_y,label
12059782,/home/jovyan/data-vol-1/similarity-project/dat...,0000,/home/jovyan/data-vol-1/similarity-project/dat...,False
12059416,/home/jovyan/data-vol-1/similarity-project/dat...,0000,/home/jovyan/data-vol-1/similarity-project/dat...,False
12847883,/home/jovyan/data-vol-1/similarity-project/dat...,0000,/home/jovyan/data-vol-1/similarity-project/dat...,False
20647990,/home/jovyan/data-vol-1/similarity-project/dat...,0000,/home/jovyan/data-vol-1/similarity-project/dat...,False
12059830,/home/jovyan/data-vol-1/similarity-project/dat...,0000,/home/jovyan/data-vol-1/similarity-project/dat...,False
...,...,...,...,...
899248,/home/jovyan/data-vol-1/similarity-project/dat...,1199,/home/jovyan/data-vol-1/similarity-project/dat...,False
901845,/home/jovyan/data-vol-1/similarity-project/dat...,1199,/home/jovyan/data-vol-1/similarity-project/dat...,False
6438031,/home/jovyan/data-vol-1/similarity-project/dat...,1199,/home/jovyan/data-vol-1/similarity-project/dat...,False
899719,/home/jovyan/data-vol-1/similarity-project/dat...,1199,/home/jovyan/data-vol-1/similarity-project/dat...,False


In [242]:
unict_pairs_dedup_df = pd.concat([unict_pairs_pos_dedup_df,unict_pairs_neg_dedup_sampled_df])

In [243]:
unict_pairs_dedup_df

,file_path_x,cat,file_path_y,label
0,/home/jovyan/data-vol-1/similarity-project/dat...,0560,/home/jovyan/data-vol-1/similarity-project/dat...,True
1,/home/jovyan/data-vol-1/similarity-project/dat...,0560,/home/jovyan/data-vol-1/similarity-project/dat...,True
2,/home/jovyan/data-vol-1/similarity-project/dat...,0560,/home/jovyan/data-vol-1/similarity-project/dat...,True
3,/home/jovyan/data-vol-1/similarity-project/dat...,0560,/home/jovyan/data-vol-1/similarity-project/dat...,True
4,/home/jovyan/data-vol-1/similarity-project/dat...,0560,/home/jovyan/data-vol-1/similarity-project/dat...,True
...,...,...,...,...
899248,/home/jovyan/data-vol-1/similarity-project/dat...,1199,/home/jovyan/data-vol-1/similarity-project/dat...,False
901845,/home/jovyan/data-vol-1/similarity-project/dat...,1199,/home/jovyan/data-vol-1/similarity-project/dat...,False
6438031,/home/jovyan/data-vol-1/similarity-project/dat...,1199,/home/jovyan/data-vol-1/similarity-project/dat...,False
899719,/home/jovyan/data-vol-1/similarity-project/dat...,1199,/home/jovyan/data-vol-1/similarity-project/dat...,False


In [254]:
train_cat = unict_pairs_dedup_df['cat'].drop_duplicates().sample(frac=1, random_state = 5)[0:760]
test_cat = unict_pairs_dedup_df['cat'].drop_duplicates().sample(frac=1, random_state = 5)[761:857]
valid_cat = unict_pairs_dedup_df['cat'].drop_duplicates().sample(frac=1, random_state = 5)[858:959]

In [263]:
unict_pairs_dedup_df['source'] = 'unict_27k'
# unict_pairs_dedup_df['category'] = 'train'
unict_pairs_dedup_df = unict_pairs_dedup_df.sample(frac=1, random_state = 5).reset_index(drop=True)

length = len(unict_pairs_dedup_df) # 27145
unict_pairs_dedup_df.loc[unict_pairs_dedup_df['cat'].isin(train_cat),'category'] = 'train'
unict_pairs_dedup_df.loc[unict_pairs_dedup_df['cat'].isin(valid_cat),'category'] = 'validation'
unict_pairs_dedup_df.loc[unict_pairs_dedup_df['cat'].isin(test_cat),'category'] = 'test'
    

In [264]:
unict_pairs_dedup_df

,file_path_x,cat,file_path_y,label,source,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,0643,/home/jovyan/data-vol-1/similarity-project/dat...,False,unict_27k,train
1,/home/jovyan/data-vol-1/similarity-project/dat...,0652,/home/jovyan/data-vol-1/similarity-project/dat...,True,unict_27k,train
2,/home/jovyan/data-vol-1/similarity-project/dat...,0696,/home/jovyan/data-vol-1/similarity-project/dat...,False,unict_27k,train
3,/home/jovyan/data-vol-1/similarity-project/dat...,0452,/home/jovyan/data-vol-1/similarity-project/dat...,True,unict_27k,train
4,/home/jovyan/data-vol-1/similarity-project/dat...,0303,/home/jovyan/data-vol-1/similarity-project/dat...,True,unict_27k,train
...,...,...,...,...,...,...
27140,/home/jovyan/data-vol-1/similarity-project/dat...,0712,/home/jovyan/data-vol-1/similarity-project/dat...,True,unict_27k,train
27141,/home/jovyan/data-vol-1/similarity-project/dat...,1107,/home/jovyan/data-vol-1/similarity-project/dat...,False,unict_27k,train
27142,/home/jovyan/data-vol-1/similarity-project/dat...,0039,/home/jovyan/data-vol-1/similarity-project/dat...,True,unict_27k,train
27143,/home/jovyan/data-vol-1/similarity-project/dat...,0549,/home/jovyan/data-vol-1/similarity-project/dat...,True,unict_27k,train


In [265]:
unict_pairs_dedup_df.groupby(['category','label']).count()

file_path_x    cat  file_path_y  source
category   label                                         
test       False         1056   1056         1056    1056
           True          1291   1291         1291    1291
train      False        11022  11022        11022   11022
           True         11530  11530        11530   11530
validation False         1111   1111         1111    1111
           True          1135   1135         1135    1135

In [266]:
unict_pairs_dedup_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/UNICT-FD1200/unict_pairs_dedup_df.csv')

0       0643
1       0652
2       0696
3       0452
4       0303
        ... 
7636    0295
7674    0027
7751    0054
8368    0770
9905    0485
Length: 1199, dtype: object

In [171]:
unict_pairs_dedup_df = pd.read_csv('/home/jovyan/data-vol-1/similarity-project/data/UNICT-FD1200/unict_pairs_dedup_df.csv', index_col = [0])

#add cat_x and cat_y
unict_pairs_dedup_df['cat_x'] = unict_pairs_dedup_df['file_path_x'].str.split(pat='/',n=7,expand=True)[7].str.split(pat = '_', n=3, expand=True)[2]
unict_pairs_dedup_df['cat_y'] = unict_pairs_dedup_df['file_path_y'].str.split(pat='/',n=7,expand=True)[7].str.split(pat = '_', n=3, expand=True)[2]

unict_pairs_dedup_df = unict_pairs_dedup_df.drop(columns='cat')

unique_cats = pd.concat([unict_pairs_dedup_df['cat_x'],
          unict_pairs_dedup_df['cat_y']]).drop_duplicates()

train_cat = unique_cats.sample(frac=1, random_state = 5)[0:960]
# test_cat = unique_cats.sample(frac=1, random_state = 5)[761:857]
valid_cat = unique_cats.sample(frac=1, random_state = 5)[960:1199]

unict_pairs_dedup_df['source'] = 'unict_27k'
# unict_pairs_dedup_df['category'] = 'train'
unict_pairs_dedup_df = unict_pairs_dedup_df.sample(frac=1, random_state = 5).reset_index(drop=True)

length = len(unict_pairs_dedup_df) # 27145
# unict_pairs_dedup_df.loc[unict_pairs_dedup_df['cat'].isin(test_cat),'category'] = 'test'

unict_pairs_dedup_df['category'] = 'train'

unict_pairs_dedup_df.loc[unict_pairs_dedup_df['cat_x'].isin(valid_cat)&
                         unict_pairs_dedup_df['cat_y'].isin(valid_cat),'category'] = 'validation'
unict_pairs_dedup_df    

,file_path_x,file_path_y,label,source,category,cat_x,cat_y
0,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,False,unict_27k,train,0351,1120
1,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,False,unict_27k,train,0487,0321
2,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,True,unict_27k,train,1178,1178
3,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,True,unict_27k,train,1017,1017
4,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,False,unict_27k,train,1065,0830
...,...,...,...,...,...,...,...
27140,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,False,unict_27k,train,0006,0418
27141,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,True,unict_27k,train,0164,0164
27142,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,True,unict_27k,train,0348,0348
27143,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,True,unict_27k,train,1063,1063


In [172]:
unict_pairs_dedup_df.groupby('category').count()

,file_path_x,file_path_y,label,source,cat_x,cat_y
category,,,,,,
train,23792,23792,23792,23792,23792,23792
validation,3353,3353,3353,3353,3353,3353


In [173]:
unict_pairs_dedup_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/UNICT-FD1200/unict_pairs_dedup_df_v2.csv')

# Download Top 10 Percentile by Value Sampled

In [61]:
# From Trace Analysis notebook
top_10_percentile_by_value_sampled_df = pd.read_csv('/home/jovyan/data-vol-1/similarity-project/result/Trace analysis/top_10_percentile_by_value_sampled_df.CSV', index_col = [0])


In [62]:
top_10_percentile_dl_df = pd.concat([top_10_percentile_by_value_sampled_df[['saudagar_id','new_file_name','new_img']].set_axis(['saudagar_id','file_name','img'], axis = 1),
           top_10_percentile_by_value_sampled_df[['saudagar_id','old_file_name','old_img']].set_axis(['saudagar_id','file_name','img'], axis = 1)
          ]).drop_duplicates()

In [63]:
top_10_percentile_dl_df

,saudagar_id,file_name,img
4156,G921520109,39dbc414-c788-4b6d-b319-8963f3d1d5f3_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
12380,G929040839,b241fa45-c052-4a44-86ae-39478c42f42d_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
24207,G873416291,601885ec-6309-4a8a-b3fe-4aa45711a03c_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
22474,G617096834,101a12c0-b91a-42fb-b556-3047acc2f104_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
16576,G247887696,c19130e0-6021-42f2-9e86-25ec0e27b16e_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
...,...,...,...
7159,G880347190,362d9b8d-138a-4628-a399-8f62d68ae6e6_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
5938,G855256019,623bae77-79e2-4d9a-a418-4c3cd92473d3_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
5035,G184319913,ee0a45e4-0dca-4b33-9e5f-58e4d7a1a1e9_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
2900,G916865697,a9aaf426-f73a-497f-b530-b23d97d4fa8b_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...


In [64]:
url_sel_list = top_10_percentile_dl_df['img']
merchant_sel_list = top_10_percentile_dl_df['saudagar_id']
file_name_sel_list = top_10_percentile_dl_df['file_name'].astype(str)
folder_name = "maf-220523-220605"

In [65]:
# Using multiprocessing starmap
# https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments#5443941

numberImages = 0

n_process = 4

pool = Pool(n_process)

start = datetime.now() 

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(top_10_percentile_dl_df)} outlets completed in {duration}')

pool.close()
pool.join()

Trend calculation for 15861 outlets completed in 0:00:00.055165


In [69]:
# From Trace Analysis notebook
merchant_claim_suspicion_df_sampled = pd.read_csv('/home/jovyan/data-vol-1/similarity-project/result/Trace analysis/merchant_claim_suspicion_df_sampled.csv', index_col = [0])

dl_df = pd.concat([merchant_claim_suspicion_df_sampled[['saudagar_id','new_file_name','new_img']].set_axis(['saudagar_id','file_name','img'], axis = 1),
           merchant_claim_suspicion_df_sampled[['saudagar_id','old_file_name','old_img']].set_axis(['saudagar_id','file_name','img'], axis = 1)
          ]).drop_duplicates()

url_sel_list = dl_df['img']
merchant_sel_list = dl_df['saudagar_id']
file_name_sel_list = dl_df['file_name'].astype(str)
folder_name = "maf-220523-220605"

In [70]:
# Using multiprocessing starmap
# https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments#5443941

numberImages = 0

n_process = 4

pool = Pool(n_process)

start = datetime.now() 

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(top_10_percentile_dl_df)} outlets completed in {duration}')

pool.close()
pool.join()

Image Couldn't be retreived
Image Couldn't be retreived
Error Type: ConnectionError

Error Message: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

Image Couldn't be retreived
Trend calculation for 29595 outlets completed in 1:20:17.169406


In [71]:
# Rerunning
numberImages = 0

n_process = 4

pool = Pool(n_process)

start = datetime.now() 

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(top_10_percentile_dl_df)} outlets completed in {duration}')

pool.close()
pool.join()

Trend calculation for 29595 outlets completed in 0:00:01.477285


# Preparing LTR

In [95]:
import zipfile
import os

cropped_file_path = '/home/jovyan/data-vol-1/similarity-project/data/'



In [ ]:
with zipfile.ZipFile(cropped_file_path + "LTR_cropped.zip","r") as zip_ref:
    zip_ref.extractall(cropped_file_path)


In [96]:
#Moved the extracted data to '/home/jovyan/data-vol-1/similarity-project/data/LTR'

cropped_file_list = []

for dirname, subdirs, files in os.walk(cropped_file_path + 'LTR'):     
    for filename in files:
        if '_crop.jpg' in filename: 
#             cropped_file_list.append(dirname+'/'+filename)
            cropped_file_list.append(filename)
#         else:
#             print('2.'+dirname+'/'+filename)

In [97]:
cropped_file_list
#every item is different on this list. So we will make predictions on this

['minestrone_verdure3.1_crop.jpg',
 'fusilli_crudaiola1.2_crop.jpg',
 'involtini_prosciutto1.3_crop.jpg',
 'minestrone_verdure3.2_crop.jpg',
 'involtini_prosciutto2.2_crop.jpg',
 'insalata_orzo_verdura3.1_crop.jpg',
 'frittata_forno3.2_crop.jpg',
 'fusilli_crudaiola2.3_crop.jpg',
 'insalata_pasta_fredda3.1_crop.jpg',
 'insalata_orzo_verdura2.2_crop.jpg',
 'ditalini_lenticchie1.1_crop.jpg',
 'insalata_orzo_verdura1.3_crop.jpg',
 'insalata_orzo_verdura2.1_crop.jpg',
 'frittata_forno3.1_crop.jpg',
 'tiepida_mare1.3_crop.jpg',
 'frittata_forno2.1_crop.jpg',
 'minestrone_verdure2.3_crop.jpg',
 'insalata_orzo_verdura2.3_crop.jpg',
 'minestrone_verdure3.3_crop.jpg',
 'tiepida_mare3.3_crop.jpg',
 'frittata_forno2.2_crop.jpg',
 'frittata_forno1.1_crop.jpg',
 'celentani_pesto_di_melanzane2.3_crop.jpg',
 'mozzarelline_fritte3.1_crop.jpg',
 'celentani_pesto_di_melanzane3.1_crop.jpg',
 'involtini_prosciutto3.1_crop.jpg',
 'celentani_pesto_di_melanzane2.2_crop.jpg',
 'minestrone_verdure2.2_crop.jpg'

In [98]:
ltr_df = pd.DataFrame(cropped_file_list, columns = ['file_path'])

#Cat is the most granular category, with food name AND size
ltr_df['cat'] = ltr_df.apply(lambda row: row['file_path'].split('.')[0], axis =1)

#cat broad is a broader category, with food name only
ltr_df['cat_broad_x'] = ltr_df.apply(lambda row: row['file_path'].split('.')[0][:-1], axis =1)


In [99]:
ltr_df

,file_path,cat,cat_broad_x
0,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure
1,fusilli_crudaiola1.2_crop.jpg,fusilli_crudaiola1,fusilli_crudaiola
2,involtini_prosciutto1.3_crop.jpg,involtini_prosciutto1,involtini_prosciutto
3,minestrone_verdure3.2_crop.jpg,minestrone_verdure3,minestrone_verdure
4,involtini_prosciutto2.2_crop.jpg,involtini_prosciutto2,involtini_prosciutto
...,...,...,...
94,celentani_pesto_di_melanzane3.2_crop.jpg,celentani_pesto_di_melanzane3,celentani_pesto_di_melanzane
95,minestrone_verdure1.2_crop.jpg,minestrone_verdure1,minestrone_verdure
96,insalata_orzo_verdura1.2_crop.jpg,insalata_orzo_verdura1,insalata_orzo_verdura
97,ditalini_lenticchie3.3_crop.jpg,ditalini_lenticchie3,ditalini_lenticchie


In [165]:
unique_cat_len = (len(ltr_df['cat'].drop_duplicates()))

valid_cat = ltr_df['cat'].drop_duplicates().sample(int(unique_cat_len*0.1), random_state = 5)

# ltr_df['source'] = 'ltr'

ltr_df.loc[:,'category'] = 'train'
ltr_df.loc[ltr_df['cat'].isin(valid_cat),'category'] = 'validation'

ltr_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/LTR/ltr_categories.csv') # for triplet
ltr_df

,file_path,cat,cat_broad_x,category
0,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,train
1,fusilli_crudaiola1.2_crop.jpg,fusilli_crudaiola1,fusilli_crudaiola,train
2,involtini_prosciutto1.3_crop.jpg,involtini_prosciutto1,involtini_prosciutto,train
3,minestrone_verdure3.2_crop.jpg,minestrone_verdure3,minestrone_verdure,train
4,involtini_prosciutto2.2_crop.jpg,involtini_prosciutto2,involtini_prosciutto,validation
...,...,...,...,...
94,celentani_pesto_di_melanzane3.2_crop.jpg,celentani_pesto_di_melanzane3,celentani_pesto_di_melanzane,validation
95,minestrone_verdure1.2_crop.jpg,minestrone_verdure1,minestrone_verdure,train
96,insalata_orzo_verdura1.2_crop.jpg,insalata_orzo_verdura1,insalata_orzo_verdura,validation
97,ditalini_lenticchie3.3_crop.jpg,ditalini_lenticchie3,ditalini_lenticchie,train


In [166]:
ltr_df.query("category == 'validation'")

,file_path,cat,cat_broad_x,category
4,involtini_prosciutto2.2_crop.jpg,involtini_prosciutto2,involtini_prosciutto,validation
11,insalata_orzo_verdura1.3_crop.jpg,insalata_orzo_verdura1,insalata_orzo_verdura,validation
24,celentani_pesto_di_melanzane3.1_crop.jpg,celentani_pesto_di_melanzane3,celentani_pesto_di_melanzane,validation
41,involtini_prosciutto2.3_crop.jpg,involtini_prosciutto2,involtini_prosciutto,validation
53,insalata_orzo_verdura1.1_crop.jpg,insalata_orzo_verdura1,insalata_orzo_verdura,validation
56,involtini_prosciutto2.1_crop.jpg,involtini_prosciutto2,involtini_prosciutto,validation
63,celentani_pesto_di_melanzane3.3_crop.jpg,celentani_pesto_di_melanzane3,celentani_pesto_di_melanzane,validation
94,celentani_pesto_di_melanzane3.2_crop.jpg,celentani_pesto_di_melanzane3,celentani_pesto_di_melanzane,validation
96,insalata_orzo_verdura1.2_crop.jpg,insalata_orzo_verdura1,insalata_orzo_verdura,validation


In [167]:

ltr_df = ltr_df.drop(columns = ['category'])

In [20]:

# start merging based on catbroad
ltr_paired_df = ltr_df.merge(ltr_df, how = 'left', on ='cat_broad_x').query('file_path_x != file_path_y')

ltr_paired_df['dedup_set'] = ltr_paired_df.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

ltr_paired_df['cat_broad_y'] = ltr_paired_df['cat_broad_x']

#removing duplicates
ltr_paired_dedup_df = ltr_paired_df['dedup_set'][~ltr_paired_df['dedup_set'].astype(str).duplicated(keep='first')]

ltr_paired_dedup_df = ltr_paired_df.loc[ltr_paired_dedup_df.index]

ltr_paired_dedup_df['label'] = False

ltr_paired_dedup_df = ltr_paired_dedup_df.drop(columns=['dedup_set'])


In [107]:
# ltr_paired_dedup_df = pd.read_csv('/home/jovyan/data-vol-1/similarity-project/data/LTR/ltr_paired_dedup_df.csv', index_col = [0])

In [108]:
ltr_paired_dedup_df

,file_path_x,cat_x,cat_broad_x,file_path_y,cat_y,cat_broad_y,label
1,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure3.2_crop.jpg,minestrone_verdure3,minestrone_verdure,False
2,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure2.3_crop.jpg,minestrone_verdure2,minestrone_verdure,False
3,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure3.3_crop.jpg,minestrone_verdure3,minestrone_verdure,False
4,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure2.2_crop.jpg,minestrone_verdure2,minestrone_verdure,False
5,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure1.1_crop.jpg,minestrone_verdure1,minestrone_verdure,False
...,...,...,...,...,...,...,...
835,fusilli_crudaiola3.2_crop.jpg,fusilli_crudaiola3,fusilli_crudaiola,fusilli_crudaiola2.2_crop.jpg,fusilli_crudaiola2,fusilli_crudaiola,False
839,celentani_pesto_di_melanzane2.1_crop.jpg,celentani_pesto_di_melanzane2,celentani_pesto_di_melanzane,celentani_pesto_di_melanzane2.2_crop.jpg,celentani_pesto_di_melanzane2,celentani_pesto_di_melanzane,False
845,celentani_pesto_di_melanzane2.1_crop.jpg,celentani_pesto_di_melanzane2,celentani_pesto_di_melanzane,celentani_pesto_di_melanzane3.2_crop.jpg,celentani_pesto_di_melanzane3,celentani_pesto_di_melanzane,False
855,minestrone_verdure1.2_crop.jpg,minestrone_verdure1,minestrone_verdure,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,False


In [110]:
ltr_paired_dedup_df['cat_broad_x'].drop_duplicates().sample(2, random_state = 5)

73            insalata_pasta_fredda
199    celentani_pesto_di_melanzane
Name: cat_broad_x, dtype: object

In [115]:
valid_cat = ltr_paired_dedup_df['cat_broad_x'].drop_duplicates().sample(2, random_state = 5)

ltr_paired_dedup_df['source'] = 'ltr'
# unict_pairs_dedup_df['category'] = 'train'
# ltr_paired_dedup_df = ltr_paired_dedup_df.sample(frac=1, random_state = 5).reset_index(drop=True)

length = len(ltr_paired_dedup_df) # 27145
ltr_paired_dedup_df.loc[:,'category'] = 'train'
ltr_paired_dedup_df.loc[(ltr_paired_dedup_df['cat_broad_x'].isin(valid_cat))|
                        (ltr_paired_dedup_df['cat_broad_y'].isin(valid_cat)),'category'] = 'validation'

    

In [118]:
ltr_paired_dedup_df

,file_path_x,cat_x,cat_broad_x,file_path_y,cat_y,cat_broad_y,label,source,category
1,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure3.2_crop.jpg,minestrone_verdure3,minestrone_verdure,False,ltr,train
2,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure2.3_crop.jpg,minestrone_verdure2,minestrone_verdure,False,ltr,train
3,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure3.3_crop.jpg,minestrone_verdure3,minestrone_verdure,False,ltr,train
4,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure2.2_crop.jpg,minestrone_verdure2,minestrone_verdure,False,ltr,train
5,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure1.1_crop.jpg,minestrone_verdure1,minestrone_verdure,False,ltr,train
...,...,...,...,...,...,...,...,...,...
835,fusilli_crudaiola3.2_crop.jpg,fusilli_crudaiola3,fusilli_crudaiola,fusilli_crudaiola2.2_crop.jpg,fusilli_crudaiola2,fusilli_crudaiola,False,ltr,train
839,celentani_pesto_di_melanzane2.1_crop.jpg,celentani_pesto_di_melanzane2,celentani_pesto_di_melanzane,celentani_pesto_di_melanzane2.2_crop.jpg,celentani_pesto_di_melanzane2,celentani_pesto_di_melanzane,False,ltr,validation
845,celentani_pesto_di_melanzane2.1_crop.jpg,celentani_pesto_di_melanzane2,celentani_pesto_di_melanzane,celentani_pesto_di_melanzane3.2_crop.jpg,celentani_pesto_di_melanzane3,celentani_pesto_di_melanzane,False,ltr,validation
855,minestrone_verdure1.2_crop.jpg,minestrone_verdure1,minestrone_verdure,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,False,ltr,train


In [117]:
ltr_paired_dedup_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/LTR/ltr_paired_dedup_df.csv')

In [168]:
import pandas as pd
# there's a problem with ltr_paired_dedup_df
# images in test set appeared in train set before. this is unacceptable
ltr_paired_dedup_df = pd.read_csv('/home/jovyan/data-vol-1/similarity-project/data/LTR/ltr_paired_dedup_df.csv', index_col=[0])

unique_cat = pd.concat([ltr_paired_dedup_df['cat_broad_x'],
                        ltr_paired_dedup_df['cat_broad_y']]).drop_duplicates()
train_cat = unique_cat.sample(frac = 1, random_state = 5)[0:10] # select 9 out of 11
valid_cat = unique_cat.sample(frac = 1, random_state = 5)[10:] # select 9 out of 11

ltr_paired_dedup_df['source'] = 'ltr'
# unict_pairs_dedup_df['category'] = 'train'
# ltr_paired_dedup_df = ltr_paired_dedup_df.sample(frac=1, random_state = 5).reset_index(drop=True)

length = len(ltr_paired_dedup_df) # 27145
ltr_paired_dedup_df.loc[:,'category'] = 'train'
ltr_paired_dedup_df.loc[(ltr_paired_dedup_df['cat_broad_x'].isin(valid_cat))&
                        (ltr_paired_dedup_df['cat_broad_y'].isin(valid_cat)),'category'] = 'validation'


# ltr_paired_dedup_df.loc[(ltr_paired_dedup_df['cat_broad_x'].isin(train_cat)|
#                         ltr_paired_dedup_df['cat_broad_y'].isin(train_cat)),'category'] = 'train'

ltr_paired_dedup_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/LTR/ltr_paired_dedup_df_v2.csv')

In [169]:
ltr_paired_dedup_df

,file_path_x,cat_x,cat_broad_x,file_path_y,cat_y,cat_broad_y,label,source,category
1,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure3.2_crop.jpg,minestrone_verdure3,minestrone_verdure,False,ltr,train
2,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure2.3_crop.jpg,minestrone_verdure2,minestrone_verdure,False,ltr,train
3,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure3.3_crop.jpg,minestrone_verdure3,minestrone_verdure,False,ltr,train
4,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure2.2_crop.jpg,minestrone_verdure2,minestrone_verdure,False,ltr,train
5,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,minestrone_verdure1.1_crop.jpg,minestrone_verdure1,minestrone_verdure,False,ltr,train
...,...,...,...,...,...,...,...,...,...
835,fusilli_crudaiola3.2_crop.jpg,fusilli_crudaiola3,fusilli_crudaiola,fusilli_crudaiola2.2_crop.jpg,fusilli_crudaiola2,fusilli_crudaiola,False,ltr,train
839,celentani_pesto_di_melanzane2.1_crop.jpg,celentani_pesto_di_melanzane2,celentani_pesto_di_melanzane,celentani_pesto_di_melanzane2.2_crop.jpg,celentani_pesto_di_melanzane2,celentani_pesto_di_melanzane,False,ltr,train
845,celentani_pesto_di_melanzane2.1_crop.jpg,celentani_pesto_di_melanzane2,celentani_pesto_di_melanzane,celentani_pesto_di_melanzane3.2_crop.jpg,celentani_pesto_di_melanzane3,celentani_pesto_di_melanzane,False,ltr,train
855,minestrone_verdure1.2_crop.jpg,minestrone_verdure1,minestrone_verdure,minestrone_verdure3.1_crop.jpg,minestrone_verdure3,minestrone_verdure,False,ltr,train


In [170]:
ltr_paired_dedup_df.groupby('category').count()

,file_path_x,cat_x,cat_broad_x,file_path_y,cat_y,cat_broad_y,label,source
category,,,,,,,,
train,386,386,386,386,386,386,386,386
validation,37,37,37,37,37,37,37,37


# NuFood

In [51]:
import zipfile
import os

cropped_file_path = '/home/jovyan/data-vol-1/similarity-project/data/'

In [ ]:
with zipfile.ZipFile(cropped_file_path + "nufood_cropped.zip","r") as zip_ref:
    zip_ref.extractall(cropped_file_path)


In [52]:
#Moved the extracted data to '/home/jovyan/data-vol-1/similarity-project/data/LTR'

cropped_file_list = []

for dirname, subdirs, files in os.walk(cropped_file_path + 'nufood'):     
    for filename in files:
        if '_crop.jpg' in filename: 
            cropped_file_list.append(dirname+'/'+filename)
#             cropped_file_list.append(filename)
#         else:
#             print('2.'+dirname+'/'+filename)

In [53]:
cropped_file_list

['/home/jovyan/data-vol-1/similarity-project/data/nufood/02.curry_and_rice/orig/21_crop.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/nufood/02.curry_and_rice/orig/16_crop.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/nufood/02.curry_and_rice/orig/20_crop.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/nufood/02.curry_and_rice/orig/22_crop.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/nufood/02.curry_and_rice/orig/12_crop.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/nufood/02.curry_and_rice/orig/23_crop.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/nufood/02.curry_and_rice/orig/31_crop.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/nufood/02.curry_and_rice/orig/17_crop.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/nufood/02.curry_and_rice/orig/2_crop.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/nufood/02.curry_and_rice/orig/25_crop.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/nufood/02.c

In [54]:
nufood_df = pd.DataFrame(cropped_file_list, columns = ['file_path'])

#Cat is the most granular category, with food name AND size
# nufood_df['cat'] = nufood_df.apply(lambda row: row['file_path'].split('.')[0], axis =1)
nufood_df['cat'] = nufood_df['file_path'].str.split(pat='/',n=8,expand=True)[7]


In [55]:
nufood_df

,file_path,cat
0,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice
1,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice
2,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice
3,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice
4,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice
...,...,...
355,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl
356,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl
357,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl
358,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl


In [168]:
unique_cat_len = (len(nufood_df['cat'].drop_duplicates()))

valid_cat = nufood_df['cat'].drop_duplicates().sample(int(unique_cat_len*0.1), random_state = 5)

# ltr_df['source'] = 'ltr'

nufood_df.loc[:,'category'] = 'train'
nufood_df.loc[nufood_df['cat'].isin(valid_cat),'category'] = 'validation'

nufood_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/nufood/nufood_df_categories.csv')

nufood_df

,file_path,cat,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice,train
1,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice,train
2,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice,train
3,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice,train
4,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice,train
...,...,...,...
355,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
356,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
357,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
358,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation


In [169]:

nufood_df.query("category == 'validation'")

,file_path,cat,category
324,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
325,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
326,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
327,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
328,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
329,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
330,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
331,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
332,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation
333,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,validation


In [170]:

nufood_df = nufood_df.drop(columns = ['category'])

In [38]:

# start merging based on catbroad
nufood_paired_df = nufood_df.merge(nufood_df, how = 'left', on ='cat').query('file_path_x != file_path_y')

nufood_paired_df['dedup_set'] = nufood_paired_df.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

nufood_paired_df['cat_y'] = nufood_paired_df['cat']
nufood_paired_df['cat_x'] = nufood_paired_df['cat']
nufood_paired_df=nufood_paired_df.drop(columns=['cat'])

#removing duplicates
nufood_paired_dedup_df = nufood_paired_df['dedup_set'][~nufood_paired_df['dedup_set'].astype(str).duplicated(keep='first')]

nufood_paired_dedup_df = nufood_paired_df.loc[nufood_paired_dedup_df.index]

nufood_paired_dedup_df['label'] = True

nufood_paired_dedup_df = nufood_paired_dedup_df.drop(columns=['dedup_set'])


In [39]:
nufood_paired_df

,file_path_x,file_path_y,dedup_set,cat_y,cat_x
1,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,{/home/jovyan/data-vol-1/similarity-project/da...,02.curry_and_rice,02.curry_and_rice
2,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,{/home/jovyan/data-vol-1/similarity-project/da...,02.curry_and_rice,02.curry_and_rice
3,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,{/home/jovyan/data-vol-1/similarity-project/da...,02.curry_and_rice,02.curry_and_rice
4,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,{/home/jovyan/data-vol-1/similarity-project/da...,02.curry_and_rice,02.curry_and_rice
5,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,{/home/jovyan/data-vol-1/similarity-project/da...,02.curry_and_rice,02.curry_and_rice
...,...,...,...,...,...
12954,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,{/home/jovyan/data-vol-1/similarity-project/da...,06.tempura_rice-bowl,06.tempura_rice-bowl
12955,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,{/home/jovyan/data-vol-1/similarity-project/da...,06.tempura_rice-bowl,06.tempura_rice-bowl
12956,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,{/home/jovyan/data-vol-1/similarity-project/da...,06.tempura_rice-bowl,06.tempura_rice-bowl
12957,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,{/home/jovyan/data-vol-1/similarity-project/da...,06.tempura_rice-bowl,06.tempura_rice-bowl


In [42]:
# Get negative pairs
nufood_df['key'] = 1

nufood_pairs_neg = nufood_df.merge(nufood_df, how = 'left', on = 'key')

nufood_pairs_neg = nufood_pairs_neg.query('cat_x != cat_y')

nufood_pairs_neg['dedup_set'] = nufood_pairs_neg.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

nufood_pairs_neg_dedup = nufood_pairs_neg['dedup_set'][~nufood_pairs_neg['dedup_set'].astype(str).duplicated(keep='first')]

nufood_pairs_neg_dedup_df = nufood_pairs_neg.loc[nufood_pairs_neg_dedup.index]

nufood_pairs_neg_dedup_df['label'] = False

nufood_pairs_neg_dedup_df = nufood_pairs_neg_dedup_df.drop(columns=['dedup_set'])



In [54]:
nufood_pairs_neg_dedup_sampled_df = nufood_pairs_neg_dedup_df.groupby('cat_x', group_keys=False).\
apply(lambda x: x.sample(min(len(x), 700), random_state = 5))

nufood_pairs_neg_dedup_sampled_df = nufood_pairs_neg_dedup_sampled_df.drop(columns = ['key'])

nufood_pairs_neg_dedup_sampled_df.columns = ['file_path_x', 'cat_x', 'file_path_y', 'cat_y','label']

In [59]:
nufood_pairs_dedup_df = pd.concat([nufood_paired_dedup_df,nufood_pairs_neg_dedup_sampled_df])
nufood_pairs_dedup_df

,file_path_x,file_path_y,cat_y,cat_x,label
1,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice,02.curry_and_rice,True
2,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice,02.curry_and_rice,True
3,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice,02.curry_and_rice,True
4,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice,02.curry_and_rice,True
5,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,02.curry_and_rice,02.curry_and_rice,True
...,...,...,...,...,...
101301,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,03.eel_rice-bowl,10.fish_burger,False
91019,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,05.hamburger_steak,10.fish_burger,False
96444,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,10.fish_burger,False
98932,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,05.hamburger_steak,10.fish_burger,False


In [1]:
nufood_pairs_neg_dedup_df

NameError: name 'nufood_pairs_neg_dedup_df' is not defined

In [90]:
nufood_pairs_dedup_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)

11665       06.tempura_rice-bowl
6481                04.beef_stew
2593     07.fried_pork_rice-bowl
5185           08.tuna_rice-bowl
9073              10.fish_burger
1297                  01.sashimi
1              02.curry_and_rice
10369         05.hamburger_steak
7777            09.cheese_burger
3889            03.eel_rice-bowl
Name: cat_x, dtype: object

In [91]:
# to be replaced by this:

train_cat = nufood_pairs_dedup_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[0:8]
test_cat = nufood_pairs_dedup_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[8:9]
valid_cat = nufood_pairs_dedup_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[9:10]

nufood_pairs_dedup_df['source'] = 'nufood'
nufood_pairs_dedup_df = nufood_pairs_dedup_df.sample(frac=1, random_state = 5).reset_index(drop=True)
    
nufood_pairs_dedup_df.loc[:,'category'] = 'train'

nufood_pairs_dedup_df.loc[(nufood_pairs_dedup_df['cat_x'].isin(test_cat)|
                                      nufood_pairs_dedup_df['cat_y'].isin(test_cat)),'category'] = 'test'
    
nufood_pairs_dedup_df.loc[(nufood_pairs_dedup_df['cat_x'].isin(valid_cat)|
                                      nufood_pairs_dedup_df['cat_y'].isin(valid_cat)),'category'] = 'validation'


In [95]:
# test that categories that are from test are only in test and validation
nufood_pairs_dedup_df.loc[(nufood_pairs_dedup_df['cat_x'].isin(train_cat)|
                           nufood_pairs_dedup_df['cat_y'].isin(train_cat)),'category'].unique()
    

array(['train', 'validation', 'test'], dtype=object)

In [93]:
# test that categories that are from test are only in test and validation
nufood_pairs_dedup_df.loc[(nufood_pairs_dedup_df['cat_x'].isin(test_cat)|
                           nufood_pairs_dedup_df['cat_y'].isin(test_cat)),'category'].unique()
    

array(['validation', 'test'], dtype=object)

In [94]:
# test that categories that are from validation are only in validation
nufood_pairs_dedup_df.loc[(nufood_pairs_dedup_df['cat_x'].isin(valid_cat)|
                           nufood_pairs_dedup_df['cat_y'].isin(valid_cat)),'category'].unique()
    

array(['validation'], dtype=object)

In [97]:
nufood_pairs_dedup_df.groupby('category').count()

,file_path_x,file_path_y,cat_y,cat_x,label,source
category,,,,,,
test,2049,2049,2049,2049,2049,2049
train,10002,10002,10002,10002,10002,10002
validation,1790,1790,1790,1790,1790,1790


In [98]:
nufood_pairs_dedup_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/nufood/nufood_pairs_dedup_df.csv')

In [158]:
import pandas as pd
# there's a problem with nufood_pairs_dedup_df
# images in test set appeared in train set before. this is unacceptable
nufood_pairs_dedup_df = pd.read_csv('/home/jovyan/data-vol-1/similarity-project/data/nufood/nufood_pairs_dedup_df.csv', index_col=[0])

train_cat = nufood_pairs_dedup_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[0:8]
# test_cat = nufood_pairs_dedup_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[8:9]
valid_cat = nufood_pairs_dedup_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[9:10]

nufood_pairs_dedup_df['source'] = 'nufood'
nufood_pairs_dedup_df = nufood_pairs_dedup_df.sample(frac=1, random_state = 5).reset_index(drop=True)

# nufood_pairs_dedup_df.loc[(nufood_pairs_dedup_df['cat_x'].isin(test_cat)|
#                                       nufood_pairs_dedup_df['cat_y'].isin(test_cat)),'category'] = 'test'
    
nufood_pairs_dedup_df['category'] = 'train'

nufood_pairs_dedup_df.loc[(nufood_pairs_dedup_df['cat_x'].isin(valid_cat)&
                                      nufood_pairs_dedup_df['cat_y'].isin(valid_cat)),'category'] = 'validation'

# nufood_pairs_dedup_df.loc[(nufood_pairs_dedup_df['cat_x'].isin(train_cat)&
#                                       nufood_pairs_dedup_df['cat_y'].isin(train_cat)),'category'] = 'train'

nufood_pairs_dedup_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/nufood/nufood_pairs_dedup_df_v2.csv')

In [159]:
nufood_pairs_dedup_df[nufood_pairs_dedup_df['file_path_x'] == "/home/jovyan/data-vol-1/similarity-project/data/nufood/04.beef_stew/orig/4_crop.jpg"]

,file_path_x,file_path_y,cat_y,cat_x,label,source,category
344,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,04.beef_stew,04.beef_stew,True,nufood,train
847,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,04.beef_stew,04.beef_stew,True,nufood,train
933,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,04.beef_stew,04.beef_stew,True,nufood,train
1083,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,04.beef_stew,04.beef_stew,True,nufood,train
1405,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,09.cheese_burger,04.beef_stew,False,nufood,train
2234,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,04.beef_stew,04.beef_stew,True,nufood,train
2255,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,04.beef_stew,04.beef_stew,True,nufood,train
2522,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,04.beef_stew,False,nufood,train
2766,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,10.fish_burger,04.beef_stew,False,nufood,train
3020,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,04.beef_stew,04.beef_stew,True,nufood,train


In [161]:
valid_cat

4    10.fish_burger
Name: cat_x, dtype: object

In [162]:
nufood_pairs_dedup_df

,file_path_x,file_path_y,cat_y,cat_x,label,source,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,07.fried_pork_rice-bowl,02.curry_and_rice,False,nufood,train
1,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,05.hamburger_steak,06.tempura_rice-bowl,False,nufood,train
2,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,01.sashimi,10.fish_burger,False,nufood,train
3,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,06.tempura_rice-bowl,05.hamburger_steak,False,nufood,train
4,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,09.cheese_burger,06.tempura_rice-bowl,False,nufood,train
...,...,...,...,...,...,...,...
13836,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,08.tuna_rice-bowl,02.curry_and_rice,False,nufood,train
13837,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,03.eel_rice-bowl,03.eel_rice-bowl,True,nufood,train
13838,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,01.sashimi,05.hamburger_steak,False,nufood,train
13839,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,04.beef_stew,04.beef_stew,True,nufood,train


In [167]:
nufood_pairs_dedup_df[nufood_pairs_dedup_df['category'].isin(['validation'])]

,file_path_x,file_path_y,cat_y,cat_x,label,source,category
24,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,10.fish_burger,10.fish_burger,True,nufood,validation
28,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,10.fish_burger,10.fish_burger,True,nufood,validation
53,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,10.fish_burger,10.fish_burger,True,nufood,validation
60,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,10.fish_burger,10.fish_burger,True,nufood,validation
78,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,10.fish_burger,10.fish_burger,True,nufood,validation
...,...,...,...,...,...,...,...
13661,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,10.fish_burger,10.fish_burger,True,nufood,validation
13708,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,10.fish_burger,10.fish_burger,True,nufood,validation
13720,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,10.fish_burger,10.fish_burger,True,nufood,validation
13725,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,10.fish_burger,10.fish_burger,True,nufood,validation


In [163]:
nufood_pairs_dedup_df.groupby('category').count()

,file_path_x,file_path_y,cat_y,cat_x,label,source
category,,,,,,
train,13162,13162,13162,13162,13162,13162
validation,679,679,679,679,679,679


# TurkFood

In [63]:
import zipfile
import os

cropped_file_path = '/home/jovyan/data-vol-1/similarity-project/data/'
with zipfile.ZipFile(cropped_file_path + "turk-food.zip","r") as zip_ref:
    zip_ref.extractall(cropped_file_path)


In [66]:
#Moved the extracted data to '/home/jovyan/data-vol-1/similarity-project/data/LTR'

cropped_file_list = []

for dirname, subdirs, files in os.walk(cropped_file_path + 'Turk-15'):     
    for filename in files:
#         if '_crop.jpg' in filename:  # all files are included for turkfood
            cropped_file_list.append(dirname+'/'+filename)
#             cropped_file_list.append(filename)
#         else:
#             print('2.'+dirname+'/'+filename)
cropped_file_list

['/home/jovyan/data-vol-1/similarity-project/data/Turk-15/.DS_Store',
 '/home/jovyan/data-vol-1/similarity-project/data/Turk-15/yaprak_sarma/438.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/Turk-15/yaprak_sarma/165.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/Turk-15/yaprak_sarma/362.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/Turk-15/yaprak_sarma/460.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/Turk-15/yaprak_sarma/285.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/Turk-15/yaprak_sarma/223.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/Turk-15/yaprak_sarma/96.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/Turk-15/yaprak_sarma/276.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/Turk-15/yaprak_sarma/234.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/Turk-15/yaprak_sarma/262.jpg',
 '/home/jovyan/data-vol-1/similarity-project/data/Turk-15/yaprak_sarma/9.jpg',
 '/home/jovyan/data-vol-1/similarity-proje

In [69]:
turkfood_df = pd.DataFrame(cropped_file_list, columns = ['file_path'])

turkfood_df['cat'] = turkfood_df['file_path'].str.split(pat='/',n=8,expand=True)[7]

# start merging based on catbroad
turkfood_paired_df = turkfood_df.merge(turkfood_df, how = 'left', on ='cat').query('file_path_x != file_path_y')

turkfood_paired_df['dedup_set'] = turkfood_paired_df.apply(lambda row: {row['file_path_x'], row['file_path_y']},axis=1)

turkfood_paired_df['cat_y'] = turkfood_paired_df['cat']
turkfood_paired_df['cat_x'] = turkfood_paired_df['cat']
turkfood_paired_df=turkfood_paired_df.drop(columns=['cat'])

#removing duplicates
turkfood_paired_dedup_df = turkfood_paired_df['dedup_set'][~turkfood_paired_df['dedup_set'].astype(str).duplicated(keep='first')]

turkfood_paired_dedup_df = turkfood_paired_df.loc[turkfood_paired_dedup_df.index]

turkfood_paired_dedup_df['label'] = False

turkfood_paired_dedup_df = turkfood_paired_dedup_df.drop(columns=['dedup_set'])


In [70]:
turkfood_paired_dedup_df

,file_path_x,file_path_y,cat_y,cat_x,label
2,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,yaprak_sarma,yaprak_sarma,False
3,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,yaprak_sarma,yaprak_sarma,False
4,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,yaprak_sarma,yaprak_sarma,False
5,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,yaprak_sarma,yaprak_sarma,False
6,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,yaprak_sarma,yaprak_sarma,False
...,...,...,...,...,...
4052351,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,manti,manti,False
4052364,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,manti,manti,False
4052367,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,manti,manti,False
4052386,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,manti,manti,False


In [80]:
turkfood_paired_dedup_sampled_df = turkfood_paired_dedup_df.groupby('cat_x', group_keys=False).\
apply(lambda x: x.sample(min(len(x), 400), random_state = 5))

# turkfood_paired_dedup_sampled_df = turkfood_paired_dedup_sampled_df.drop(columns = ['key'])

# turkfood_paired_dedup_sampled_df.columns = ['file_path_x', 'cat_x', 'file_path_y', 'cat_y','label']

In [81]:
turkfood_paired_dedup_sampled_df

,file_path_x,file_path_y,cat_y,cat_x,label
1677322,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,biber_dolmasi,biber_dolmasi,False
1580811,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,biber_dolmasi,biber_dolmasi,False
1512759,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,biber_dolmasi,biber_dolmasi,False
1553779,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,biber_dolmasi,biber_dolmasi,False
1633707,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,biber_dolmasi,biber_dolmasi,False
...,...,...,...,...,...
115178,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,yaprak_sarma,yaprak_sarma,False
76098,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,yaprak_sarma,yaprak_sarma,False
229812,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,yaprak_sarma,yaprak_sarma,False
50280,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,yaprak_sarma,yaprak_sarma,False


In [102]:
turkfood_paired_dedup_sampled_df['cat_x'].drop_duplicates().sample(frac=1)

6      biber_dolmasi
34             kebap
22    hunkar_begendi
0              borek
10             lokum
28             simit
2              manti
11      yaprak_sarma
1         icli_kofte
14           enginar
8       kuru_fasulye
13             kisir
7            ispanak
9          cig_kofte
5              hamsi
Name: cat_x, dtype: object

In [103]:
# to be replaced by this:

train_cat = turkfood_paired_dedup_sampled_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[0:11]
test_cat = turkfood_paired_dedup_sampled_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[11:13]
valid_cat = turkfood_paired_dedup_sampled_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[13:15]

turkfood_paired_dedup_sampled_df['source'] = 'turkfood'
turkfood_paired_dedup_sampled_df = turkfood_paired_dedup_sampled_df.sample(frac=1, random_state = 5).reset_index(drop=True)
    
turkfood_paired_dedup_sampled_df.loc[:,'category'] = 'train'

turkfood_paired_dedup_sampled_df.loc[(turkfood_paired_dedup_sampled_df['cat_x'].isin(test_cat)|
                                      turkfood_paired_dedup_sampled_df['cat_y'].isin(test_cat)),'category'] = 'test'
    
turkfood_paired_dedup_sampled_df.loc[(turkfood_paired_dedup_sampled_df['cat_x'].isin(valid_cat)|
                                      turkfood_paired_dedup_sampled_df['cat_y'].isin(valid_cat)),'category'] = 'validation'


In [105]:
turkfood_paired_dedup_sampled_df

,file_path_x,file_path_y,cat_y,cat_x,label,source,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,kebap,kebap,False,turkfood,train
1,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,ispanak,ispanak,False,turkfood,train
2,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,ispanak,ispanak,False,turkfood,train
3,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,biber_dolmasi,biber_dolmasi,False,turkfood,train
4,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,lokum,lokum,False,turkfood,train
...,...,...,...,...,...,...,...
5995,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,yaprak_sarma,yaprak_sarma,False,turkfood,train
5996,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,icli_kofte,icli_kofte,False,turkfood,train
5997,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,borek,borek,False,turkfood,train
5998,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,borek,borek,False,turkfood,train


In [106]:
turkfood_paired_dedup_sampled_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/Turk-15/turkfood_paired_dedup_sampled_df.csv')

In [146]:
import pandas as pd
# there's a problem with turkfood_paired_dedup_sampled_df
# images in test set appeared in train set before. this is unacceptable
turkfood_paired_dedup_sampled_df = pd.read_csv('/home/jovyan/data-vol-1/similarity-project/data/Turk-15/turkfood_paired_dedup_sampled_df.csv', index_col = [0])

train_cat = turkfood_paired_dedup_sampled_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[0:13]
# test_cat = turkfood_paired_dedup_sampled_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[11:13]
valid_cat = turkfood_paired_dedup_sampled_df['cat_x'].drop_duplicates().sample(frac=1, random_state = 5)[13:]

turkfood_paired_dedup_sampled_df['source'] = 'turkfood'
turkfood_paired_dedup_sampled_df = turkfood_paired_dedup_sampled_df.sample(frac=1, random_state = 5).reset_index(drop=True)
    
# turkfood_paired_dedup_sampled_df.loc[(turkfood_paired_dedup_sampled_df['cat_x'].isin(test_cat)|
#                                       turkfood_paired_dedup_sampled_df['cat_y'].isin(test_cat)),'category'] = 'test'
    
turkfood_paired_dedup_sampled_df['category'] = 'train'

turkfood_paired_dedup_sampled_df.loc[(turkfood_paired_dedup_sampled_df['cat_x'].isin(valid_cat)|
                                      turkfood_paired_dedup_sampled_df['cat_y'].isin(valid_cat)),'category'] = 'validation'

# turkfood_paired_dedup_sampled_df.loc[(turkfood_paired_dedup_sampled_df['cat_x'].isin(train_cat)|
#                                       turkfood_paired_dedup_sampled_df['cat_y'].isin(train_cat)),'category'] = 'train'

turkfood_paired_dedup_sampled_df.to_csv('/home/jovyan/data-vol-1/similarity-project/data/Turk-15/turkfood_paired_dedup_sampled_df_v2.csv')

In [144]:
turkfood_paired_dedup_sampled_df

,file_path_x,file_path_y,cat_y,cat_x,label,source,category
0,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,lokum,lokum,False,turkfood,validation
1,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,hunkar_begendi,hunkar_begendi,False,turkfood,train
2,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,borek,borek,False,turkfood,train
3,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,kuru_fasulye,kuru_fasulye,False,turkfood,train
4,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,hunkar_begendi,hunkar_begendi,False,turkfood,train
...,...,...,...,...,...,...,...
5995,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,kuru_fasulye,kuru_fasulye,False,turkfood,train
5996,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,hunkar_begendi,hunkar_begendi,False,turkfood,train
5997,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,hamsi,hamsi,False,turkfood,train
5998,/home/jovyan/data-vol-1/similarity-project/dat...,/home/jovyan/data-vol-1/similarity-project/dat...,lokum,lokum,False,turkfood,validation


In [145]:
turkfood_paired_dedup_sampled_df.groupby('category').count()

,file_path_x,file_path_y,cat_y,cat_x,label,source
category,,,,,,
train,5200,5200,5200,5200,5200,5200
validation,800,800,800,800,800,800


# Preparing MAF Sim 5K

In [9]:
sim5k_df = pd.read_csv(f'{RESULT_PATH}/Trace analysis/220616_Sampled_Paired_Capped.csv', index_col = [0])

In [4]:
def download_image(url, merchant_id, file_name, folder_name):
    
    global numberImages 
    
    ## Set up the image URL and filename
    image_url = url

    new_path = f"{DATA_PATH}/{folder_name}/{merchant_id}/"
    file_path = new_path + file_name 

    if os.path.exists(file_path):
        # print("Exists: ", file_name, '\n')
        return

    try:
        # Open the url image, set stream to True, this will return the stream content.
        r = requests.get(image_url, stream = True)

        # Check if the image was retrieved successfully
        if r.status_code == 200:
            # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            r.raw.decode_content = True

            if not os.path.exists(new_path):
                os.makedirs(new_path)

            # Open a local file with wb ( write binary ) permission.
            with open(file_path,'wb') as f:
                shutil.copyfileobj(r.raw, f)
                numberImages += 1

            #sleep for half second to prevent getting bumped out
            time.sleep(0.5)

            #  print('Downloaded: ',file_name, '\n')
        else:
              print('Image Couldn\'t be retreived')
                
    except Exception as e:
        print('Error Type: {0}\n'.format(e.__class__.__name__))
        print('Error Message: {0}\n'.format(e))
        
# delete images that do not exist
def test_and_delete(path):
    try:
        image = Image.open(path)
    
    except:
        os.remove(path)  
        print(f'\n{path} not valid and deleted.\n')
        

In [11]:
sim5k_df = sim5k_df.drop_duplicates()
sim5k_df['file_name_new'] = sim5k_df['img_new'].str.split('/').str[-1]
sim5k_df['item_uuid_new'] = sim5k_df['file_name_new'].str[0:36]
sim5k_df['file_name_old'] = sim5k_df['img_old'].str.split('/').str[-1]
sim5k_df['item_uuid_old'] = sim5k_df['file_name_old'].str[0:36]

In [12]:
sim5k_df

,event_timestamp_new,saudagar_id,order_number_new,claim_amount_new,img_new,event_timestamp_old,order_number_old,claim_amount_old,img_old,time_diff,file_name_new,item_uuid_new,file_name_old,item_uuid_old
6645,2022-05-30 00:33:18.557000+00:00,G356674051,F-1713823348,14280.0,https://i.gojekapi.com/darkroom/food-merchant-...,2022-05-30 00:33:06.004000+00:00,F-1713828940,17600.0,https://i.gojekapi.com/darkroom/food-merchant-...,0 days 00:00:12.553000,06f7b9ec-b604-4143-b295-e5b098469574_GoFood-F-...,06f7b9ec-b604-4143-b295-e5b098469574,e61a0470-0dc4-4070-bbec-b366b35496c7_GoFood-F-...,e61a0470-0dc4-4070-bbec-b366b35496c7
6637,2022-05-30 00:33:18.557000+00:00,G356674051,F-1713823348,14280.0,https://i.gojekapi.com/darkroom/food-merchant-...,2022-05-30 00:32:52.208000+00:00,F-1713838377,12280.0,https://i.gojekapi.com/darkroom/food-merchant-...,0 days 00:00:26.349000,06f7b9ec-b604-4143-b295-e5b098469574_GoFood-F-...,06f7b9ec-b604-4143-b295-e5b098469574,969f3b0f-a66e-4cd5-959c-c17d4ff53a4b_GoFood-F-...,969f3b0f-a66e-4cd5-959c-c17d4ff53a4b
6636,2022-05-30 00:33:18.557000+00:00,G356674051,F-1713823348,14280.0,https://i.gojekapi.com/darkroom/food-merchant-...,2022-05-30 00:32:39.097000+00:00,F-1713839034,12280.0,https://i.gojekapi.com/darkroom/food-merchant-...,0 days 00:00:39.460000,06f7b9ec-b604-4143-b295-e5b098469574_GoFood-F-...,06f7b9ec-b604-4143-b295-e5b098469574,1b17f01d-45e5-46e1-bde2-4a439fa00be9_GoFood-F-...,1b17f01d-45e5-46e1-bde2-4a439fa00be9
6536,2022-05-30 02:34:47.034000+00:00,G305398741,F-1713840797,14040.0,https://i.gojekapi.com/darkroom/food-merchant-...,2022-05-29 02:42:44.657000+00:00,F-1712375334,18360.0,https://i.gojekapi.com/darkroom/food-merchant-...,0 days 23:52:02.377000,b19c898d-c95f-4b21-a150-a5e89cbc39d4_GoFood-F-...,b19c898d-c95f-4b21-a150-a5e89cbc39d4,b949b7b9-f446-4d3d-84f9-8f5865d23909_GoFood-F-...,b949b7b9-f446-4d3d-84f9-8f5865d23909
7356,2022-05-30 03:28:35.726000+00:00,G153430300,F-1713831107,15800.0,https://i.gojekapi.com/darkroom/food-merchant-...,2022-05-29 14:28:58.380000+00:00,F-1713744871,8800.0,https://i.gojekapi.com/darkroom/food-merchant-...,0 days 12:59:37.346000,cb348e92-72f5-4a41-b0f1-b111983dd792_GoFood-F-...,cb348e92-72f5-4a41-b0f1-b111983dd792,ff322b36-bb6c-4750-bba4-d43da5cd3719_GoFood-F-...,ff322b36-bb6c-4750-bba4-d43da5cd3719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7627,2022-06-05 22:36:51.059000+00:00,G586409043,F-1721211161,14720.0,https://i.gojekapi.com/darkroom/food-merchant-...,2022-06-05 22:35:12.368000+00:00,F-1721214552,14720.0,https://i.gojekapi.com/darkroom/food-merchant-...,0 days 00:01:38.691000,a3aa699c-31a7-4367-953d-16e218b7e4d8_GoFood-F-...,a3aa699c-31a7-4367-953d-16e218b7e4d8,16ad3cbf-faf5-49c7-a8a5-91dddd489063_GoFood-F-...,16ad3cbf-faf5-49c7-a8a5-91dddd489063
10449,2022-06-05 22:49:02.335000+00:00,G519707106,F-1722603696,5600.0,https://i.gojekapi.com/darkroom/food-merchant-...,2022-06-05 22:48:30.198000+00:00,F-1722605419,5600.0,https://i.gojekapi.com/darkroom/food-merchant-...,0 days 00:00:32.137000,24ce2bf5-a43c-45c1-8658-92ec1ea05f1f_GoFood-F-...,24ce2bf5-a43c-45c1-8658-92ec1ea05f1f,921aadc3-d0ee-495b-8b09-e8fc0f0ae7d1_GoFood-F-...,921aadc3-d0ee-495b-8b09-e8fc0f0ae7d1
10478,2022-06-05 22:49:02.335000+00:00,G519707106,F-1722603696,5600.0,https://i.gojekapi.com/darkroom/food-merchant-...,2022-06-05 22:48:30.198000+00:00,F-1722605419,5600.0,https://i.gojekapi.com/darkroom/food-merchant-...,0 days 00:00:32.137000,be6ec1fd-afd8-4f42-8dc4-32ce0ae9b6f3_GoFood-F-...,be6ec1fd-afd8-4f42-8dc4-32ce0ae9b6f3,921aadc3-d0ee-495b-8b09-e8fc0f0ae7d1_GoFood-F-...,921aadc3-d0ee-495b-8b09-e8fc0f0ae7d1
10472,2022-06-05 22:49:02.335000+00:00,G519707106,F-1722603696,5600.0,https://i.gojekapi.com/darkroom/food-merchant-...,2022-06-05 22:48:04.034000+00:00,F-1722229312,10000.0,https://i.gojekapi.com/darkroom/food-merchant-...,0 days 00:00:58.301000,24ce2bf5-a43c-45c1-8658-92ec1ea05f1f_GoFood-F-...,24ce2bf5-a43c-45c1-8658-92ec1ea05f1f,d997abe2-f0b1-4c3d-ae75-2934ceb00704_G

In [21]:
sim5k_df[['saudagar_id','file_name_old','img_old']].rename(columns = {'file_name_old': 'file_name',
                                                                       'img_old': 'img'})

,saudagar_id,file_name,img
6645,G356674051,e61a0470-0dc4-4070-bbec-b366b35496c7_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
6637,G356674051,969f3b0f-a66e-4cd5-959c-c17d4ff53a4b_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
6636,G356674051,1b17f01d-45e5-46e1-bde2-4a439fa00be9_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
6536,G305398741,b949b7b9-f446-4d3d-84f9-8f5865d23909_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
7356,G153430300,ff322b36-bb6c-4750-bba4-d43da5cd3719_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
...,...,...,...
7627,G586409043,16ad3cbf-faf5-49c7-a8a5-91dddd489063_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
10449,G519707106,921aadc3-d0ee-495b-8b09-e8fc0f0ae7d1_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
10478,G519707106,921aadc3-d0ee-495b-8b09-e8fc0f0ae7d1_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
10472,G519707106,d997abe2-f0b1-4c3d-ae75-2934ceb00704_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...


In [22]:
sim5k_dl_df = pd.concat([sim5k_df[['saudagar_id','file_name_new','img_new']].rename(columns = {'file_name_new': 'file_name',
                                                                                              'img_new': 'img'}),
                        sim5k_df[['saudagar_id','file_name_old','img_old']].rename(columns = {'file_name_old': 'file_name',
                                                                                              'img_old': 'img'})])

In [25]:
sim5k_dl_df = sim5k_dl_df.drop_duplicates()

In [26]:
sim5k_dl_df

,saudagar_id,file_name,img
6645,G356674051,06f7b9ec-b604-4143-b295-e5b098469574_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
6536,G305398741,b19c898d-c95f-4b21-a150-a5e89cbc39d4_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
7356,G153430300,cb348e92-72f5-4a41-b0f1-b111983dd792_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
7271,G040961099,fb7e7f3e-9fe4-467a-9215-4552c437bc88_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
6698,G856493449,fc66e1f9-14e7-41bf-b772-c354e2261bd8_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
...,...,...,...
8253,G210218430,903c70fb-dc77-4bf8-89b5-be791dbd67ad_GoFood-FP...,https://i.gojekapi.com/darkroom/food-merchant-...
7626,G586409043,5e46efa7-4e3c-46e0-bc91-b2e74286ac4f_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
7627,G586409043,16ad3cbf-faf5-49c7-a8a5-91dddd489063_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...
10449,G519707106,921aadc3-d0ee-495b-8b09-e8fc0f0ae7d1_GoFood-F-...,https://i.gojekapi.com/darkroom/food-merchant-...


In [30]:
url_sel_list = sim5k_dl_df['img']
merchant_sel_list = sim5k_dl_df['saudagar_id']
file_name_sel_list = sim5k_dl_df['file_name']
folder_name = "sim5k"

numberImages = 0
n_process = 4
pool = Pool(n_process)
start = datetime.now()

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_sel_list)} outlets completed in {duration}')

pool.close()
pool.join()

Trend calculation for 4608 outlets completed in 0:13:54.407540


# Preparing MAF Sim 5K v2

In [4]:
sim5k_df_2 = pd.read_csv(f'{RESULT_PATH}/Trace analysis/220702_Sampled_Paired_Capped.csv')

In [5]:
def download_image(url, merchant_id, file_name, folder_name):
    
    global numberImages 
    
    ## Set up the image URL and filename
    image_url = url

    new_path = f"{DATA_PATH}/{folder_name}/{merchant_id}/"
    file_path = new_path + file_name 

    if os.path.exists(file_path):
        # print("Exists: ", file_name, '\n')
        return

    try:
        # Open the url image, set stream to True, this will return the stream content.
        r = requests.get(image_url, stream = True)

        # Check if the image was retrieved successfully
        if r.status_code == 200:
            # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            r.raw.decode_content = True

            if not os.path.exists(new_path):
                os.makedirs(new_path)

            # Open a local file with wb ( write binary ) permission.
            with open(file_path,'wb') as f:
                shutil.copyfileobj(r.raw, f)
                numberImages += 1

            #sleep for half second to prevent getting bumped out
            time.sleep(0.5)

            #  print('Downloaded: ',file_name, '\n')
        else:
              print('Image Couldn\'t be retreived')
                
    except Exception as e:
        print('Error Type: {0}\n'.format(e.__class__.__name__))
        print('Error Message: {0}\n'.format(e))
        
# delete images that do not exist
def test_and_delete(path):
    try:
        image = Image.open(path)
    
    except:
        os.remove(path)  
        print(f'\n{path} not valid and deleted.\n')
        

In [7]:
sim5k_df_2 = sim5k_df_2.drop_duplicates()
sim5k_df_2['file_name_new'] = sim5k_df_2['img_new'].str.split('/').str[-1]
sim5k_df_2['item_uuid_new'] = sim5k_df_2['file_name_new'].str[0:36]
sim5k_df_2['file_name_old'] = sim5k_df_2['img_old'].str.split('/').str[-1]
sim5k_df_2['item_uuid_old'] = sim5k_df_2['file_name_old'].str[0:36]

In [8]:
sim5k_df_2[['saudagar_id','file_name_old','img_old']].rename(columns = {'file_name_old': 'file_name',
                                                                       'img_old': 'img'})

sim5k_dl_df_2 = pd.concat([sim5k_df_2[['saudagar_id','file_name_new','img_new']].rename(columns = {'file_name_new': 'file_name',
                                                                                              'img_new': 'img'}),
                        sim5k_df_2[['saudagar_id','file_name_old','img_old']].rename(columns = {'file_name_old': 'file_name',
                                                                                              'img_old': 'img'})])

sim5k_dl_df_2 = sim5k_dl_df_2.drop_duplicates()

url_sel_list = sim5k_dl_df_2['img']
merchant_sel_list = sim5k_dl_df_2['saudagar_id']
file_name_sel_list = sim5k_dl_df_2['file_name']
folder_name = "sim5k_2"

In [9]:
numberImages = 0
n_process = 4
pool = Pool(n_process)
start = datetime.now()

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_sel_list)} outlets completed in {duration}')

pool.close()
pool.join()

Error Type: ConnectionError

Error Message: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

Error Type: ConnectionError

Error Message: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

Error Type: ConnectionError
Error Type: ConnectionError

Error Type: ConnectionError
Error Type: ConnectionError

Error Message: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


Error Message: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

Error Message: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error Message: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))



Error Type: ConnectionError
Error Type: ConnectionError


Error Message: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error Message: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


Error Ty

In [10]:
numberImages = 0
n_process = 4
pool = Pool(n_process)
start = datetime.now()

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_sel_list)} outlets completed in {duration}')

pool.close()
pool.join()

Trend calculation for 3753 outlets completed in 0:00:05.518949


# Preparing prod_test_1_220703_220706_all_pairs_30mins_20cap

In [12]:
prod_test_1 = pd.read_csv(f'{RESULT_PATH}/Trace analysis/Production-Testing/prod_test_1_220703_220706_all_pairs_30mins_20cap.csv')

In [13]:
def download_image(url, merchant_id, file_name, folder_name):
    
    global numberImages 
    
    ## Set up the image URL and filename
    image_url = url

    new_path = f"{DATA_PATH}/{folder_name}/{merchant_id}/"
    file_path = new_path + file_name 

    if os.path.exists(file_path):
        # print("Exists: ", file_name, '\n')
        return

    try:
        # Open the url image, set stream to True, this will return the stream content.
        r = requests.get(image_url, stream = True)

        # Check if the image was retrieved successfully
        if r.status_code == 200:
            # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            r.raw.decode_content = True

            if not os.path.exists(new_path):
                os.makedirs(new_path)

            # Open a local file with wb ( write binary ) permission.
            with open(file_path,'wb') as f:
                shutil.copyfileobj(r.raw, f)
                numberImages += 1

            #sleep for half second to prevent getting bumped out
            time.sleep(0.5)

            #  print('Downloaded: ',file_name, '\n')
        else:
              print('Image Couldn\'t be retreived')
                
    except Exception as e:
        print('Error Type: {0}\n'.format(e.__class__.__name__))
        print('Error Message: {0}\n'.format(e))
        
# delete images that do not exist
def test_and_delete(path):
    try:
        image = Image.open(path)
    
    except:
        os.remove(path)  
        print(f'\n{path} not valid and deleted.\n')
        

In [14]:
prod_test_1 = prod_test_1.drop_duplicates()
prod_test_1['file_name_new'] = prod_test_1['img_new'].str.split('/').str[-1]
prod_test_1['item_uuid_new'] = prod_test_1['file_name_new'].str[0:36]
prod_test_1['file_name_old'] = prod_test_1['img_old'].str.split('/').str[-1]
prod_test_1['item_uuid_old'] = prod_test_1['file_name_old'].str[0:36]

In [15]:
prod_test_1[['saudagar_id','file_name_old','img_old']].rename(columns = {'file_name_old': 'file_name',
                                                                       'img_old': 'img'})

prod_test_dl_1 = pd.concat([prod_test_1[['saudagar_id','file_name_new','img_new']].rename(columns = {'file_name_new': 'file_name',
                                                                                              'img_new': 'img'}),
                        prod_test_1[['saudagar_id','file_name_old','img_old']].rename(columns = {'file_name_old': 'file_name',
                                                                                              'img_old': 'img'})])

prod_test_dl_1 = prod_test_dl_1.drop_duplicates()

url_sel_list = prod_test_dl_1['img']
merchant_sel_list = prod_test_dl_1['saudagar_id']
file_name_sel_list = prod_test_dl_1['file_name']
folder_name = "prod_test_1"

In [16]:
numberImages = 0
n_process = 4
pool = Pool(n_process)
start = datetime.now()

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_sel_list)} outlets completed in {duration}')

pool.close()
pool.join()

Trend calculation for 10686 outlets completed in 0:31:39.475293


In [22]:
import glob

for filename in glob.glob('/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/**/**.jpeg', recursive=True):
    if os.stat(filename).st_size==0:
        print(filename)
        os.remove(filename)

/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/G536814434/f74a4894-1ab8-4035-8e0d-4b0bc83713ef_GoFood-F-1755490588-1656811353201.jpeg
/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/G536814434/99eb132b-834c-41a9-b852-617599e95b25_GoFood-F-1755488979-1656811438529.jpeg
/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/G536814434/0ffd5492-51cd-44f2-9b83-472a316db60f_GoFood-F-1755490272-1656811384215.jpeg
/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/G536814434/9d42c60f-c15a-4373-8938-0bf35bba78a9_GoFood-F-1755489374-1656811409600.jpeg
/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/G536814434/05276920-d79e-43ad-ac34-2699b6209f88_GoFood-F-1755487223-1656811462715.jpeg


In [23]:
numberImages = 0
n_process = 4
pool = Pool(n_process)
start = datetime.now()

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_sel_list)} outlets completed in {duration}')

pool.close()
pool.join()

Trend calculation for 10686 outlets completed in 0:00:02.521497


In [24]:
import glob

for filename in glob.glob('/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/**/**.jpeg', recursive=True):
    if os.stat(filename).st_size==0:
        print(filename)
        os.remove(filename)

/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/G536814434/f74a4894-1ab8-4035-8e0d-4b0bc83713ef_GoFood-F-1755490588-1656811353201.jpeg
/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/G536814434/99eb132b-834c-41a9-b852-617599e95b25_GoFood-F-1755488979-1656811438529.jpeg
/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/G536814434/0ffd5492-51cd-44f2-9b83-472a316db60f_GoFood-F-1755490272-1656811384215.jpeg
/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/G536814434/9d42c60f-c15a-4373-8938-0bf35bba78a9_GoFood-F-1755489374-1656811409600.jpeg
/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/G536814434/05276920-d79e-43ad-ac34-2699b6209f88_GoFood-F-1755487223-1656811462715.jpeg


# Preparing unlikely_pairs_1_220620_220626_3500_96_20

In [25]:
unlikely_pairs_df_1 = pd.read_csv(f'{RESULT_PATH}/Trace analysis/Unlikely-Positive-Pairs/unlikely_pairs_1_220620_220626_3500_96_20.csv')
folder_name = "unlikely_pairs_1"

unlikely_pairs_df_1 = unlikely_pairs_df_1.drop_duplicates()
unlikely_pairs_df_1['file_name_new'] = unlikely_pairs_df_1['img_new'].str.split('/').str[-1]
unlikely_pairs_df_1['item_uuid_new'] = unlikely_pairs_df_1['file_name_new'].str[0:36]
unlikely_pairs_df_1['file_name_old'] = unlikely_pairs_df_1['img_old'].str.split('/').str[-1]
unlikely_pairs_df_1['item_uuid_old'] = unlikely_pairs_df_1['file_name_old'].str[0:36]

unlikely_pairs_df_1[['saudagar_id','file_name_old','img_old']].rename(columns = {'file_name_old': 'file_name',
                                                                       'img_old': 'img'})

unlikely_pairs_df_dl_1 = pd.concat([unlikely_pairs_df_1[['saudagar_id','file_name_new','img_new']].rename(columns = {'file_name_new': 'file_name',
                                                                                              'img_new': 'img'}),
                        unlikely_pairs_df_1[['saudagar_id','file_name_old','img_old']].rename(columns = {'file_name_old': 'file_name',
                                                                                              'img_old': 'img'})])

unlikely_pairs_df_dl_1 = unlikely_pairs_df_dl_1.drop_duplicates()

url_sel_list = unlikely_pairs_df_dl_1['img']
merchant_sel_list = unlikely_pairs_df_dl_1['saudagar_id']
file_name_sel_list = unlikely_pairs_df_dl_1['file_name']

numberImages = 0
n_process = 4
pool = Pool(n_process)
start = datetime.now()

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_sel_list)} outlets completed in {duration}')

pool.close()
pool.join()

Trend calculation for 6541 outlets completed in 0:19:42.447384


In [302]:

INTERMEDIATE_RESULTS_PATH_T = f"{PROJECT_ROOT_PATH}/result/intermediate/SimTrainingV0 - BaselineV5.9"
threshold_df = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_df4.csv')
threshold_df

,Unnamed: 0,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,overall_support,macro_avg_prec,...,true_f1,true_support,false_prec,false_rec,false_f1,false_support,weigh_avg_prec,weigh_avg_rec,weigh_avg_f1,acc
0,0,1,1,1,21600,0.01,0.01,0.1,762,0.820595,...,0.846743,464,0.879121,0.536913,0.666667,298,0.807845,0.790026,0.776320,0.790026
1,0,1,1,1,21600,0.01,0.21,0.1,762,0.790044,...,0.797189,416,0.895604,0.471098,0.617424,346,0.780346,0.734908,0.715563,0.734908
2,0,1,1,1,21600,0.01,0.41,0.1,762,0.724526,...,0.697826,340,0.895604,0.386256,0.539735,422,0.742936,0.635171,0.610274,0.635171
3,0,1,1,1,21600,0.01,0.61,0.1,762,0.670699,...,0.589349,265,0.912088,0.334004,0.488954,497,0.744193,0.544619,0.523869,0.544619
4,0,1,1,1,21600,0.01,0.81,0.1,762,0.650009,...,0.548112,241,0.912088,0.318618,0.472262,521,0.746311,0.513123,0.496251,0.513123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6070,0,3,3,3,21600,0.81,0.01,0.9,463,0.766626,...,0.265560,207,0.592075,0.992188,0.741606,256,0.748153,0.617711,0.528773,0.617711
6071,0,3,3,3,21600,0.81,0.21,0.9,463,0.777115,...,0.275862,198,0.613054,0.992453,0.757925,265,0.753374,0.637149,0.551773,0.637149
6072,0,3,3,3,21600,0.81,0.41,0.9,463,0.815577,...,0.321608,165,0.689977,0.993289,0.814305,298,0.779497,0.708423,0.638722,0.708423
6073,0,3,3,3,21600,0.81,0.61,0.9,463,0.847045,...,0.372093,138,0.752914,0.993846,0.856764,325,0.809027,0.766739,0.712305,0.766739


In [303]:
threshold_df.groupby('duration_threshold').count()

,Unnamed: 0,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,pred_threshold,truth_threshold,sim_threshold,overall_support,macro_avg_prec,macro_avg_rec,...,true_f1,true_support,false_prec,false_rec,false_f1,false_support,weigh_avg_prec,weigh_avg_rec,weigh_avg_f1,acc
duration_threshold,,,,,,,,,,,,,,,,,,,,,
21600,6075,6075,6075,6075,6075,6075,6075,6075,6075,6075,...,6075,6075,6075,6075,6075,6075,6075,6075,6075,6075


In [274]:
(86400+17180)/(6)*8


138106.66666666666

In [266]:
pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_df3.csv')

,Unnamed: 0,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,overall_support,macro_avg_prec,...,true_f1,true_support,false_prec,false_rec,false_f1,false_support,weigh_avg_prec,weigh_avg_rec,weigh_avg_f1,acc
0,0,0,0,0,86400,0.01,0.01,0.1,852,0.813372,...,0.829312,488,0.891403,0.541209,0.673504,364.0,0.802015,0.775822,0.762746,0.775822
1,0,0,0,0,86400,0.01,0.11,0.1,852,0.807366,...,0.812044,465,0.909502,0.519380,0.661184,387.0,0.798016,0.758216,0.743520,0.758216
2,0,0,0,0,86400,0.01,0.21,0.1,852,0.774648,...,0.760267,416,0.918552,0.465596,0.617960,436.0,0.778026,0.705399,0.687444,0.705399
3,0,0,0,0,86400,0.01,0.31,0.1,852,0.734122,...,0.693548,361,0.923077,0.415479,0.573034,491.0,0.762953,0.643192,0.624097,0.643192
4,0,0,0,0,86400,0.01,0.41,0.1,852,0.700049,...,0.634352,318,0.923077,0.382022,0.540397,534.0,0.756591,0.592723,0.575465,0.592723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25979,0,0,0,1,86400,0.91,0.91,0.9,852,0.885321,...,0.420664,212,0.804540,0.996875,0.890440,640.0,0.844741,0.815728,0.773547,0.815728
25980,0,0,0,2,86400,0.91,0.01,0.9,852,0.619157,...,0.188366,302,0.662043,0.954545,0.781832,550.0,0.631640,0.656103,0.571472,0.656103
25981,0,0,0,2,86400,0.91,0.11,0.9,852,0.621049,...,0.189944,299,0.665826,0.954792,0.784547,553.0,0.634398,0.659624,0.575878,0.659624
25982,0,0,0,2,86400,0.91,0.21,0.9,852,0.635550,...,0.202985,276,0.694830,0.956597,0.804967,576.0,0.656423,0.686620,0.609959,0.686620


In [244]:
threshold_df.columns

Index(['Unnamed: 0', 'overall_num_pairs_min', 'pred_pairs_min',
       'truth_pairs_min', 'duration_threshold', 'pred_threshold',
       'truth_threshold', 'sim_threshold', 'overall_support', 'macro_avg_prec',
       'macro_avg_rec', 'macro_avg_f1', 'true_prec', 'true_rec', 'true_f1',
       'true_support', 'false_prec', 'false_rec', 'false_f1', 'false_support',
       'weigh_avg_prec', 'weigh_avg_rec', 'weigh_avg_f1', 'acc'],
      dtype='object')

In [245]:
threshold_df.describe()

,Unnamed: 0,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,overall_support,macro_avg_prec,...,true_f1,true_support,false_prec,false_rec,false_f1,false_support,weigh_avg_prec,weigh_avg_rec,weigh_avg_f1,acc
count,87329.0,87329.000000,87329.000000,87329.000000,87329.000000,87329.000000,87329.000000,87329.000000,87329.000000,87329.000000,...,87329.000000,87329.000000,87329.000000,87329.000000,87329.000000,87329.000000,87329.000000,87329.000000,87329.000000,87329.000000
mean,0.0,0.989362,1.491578,1.499845,85940.441320,0.459504,0.459995,0.520479,793.595747,0.739208,...,0.523013,242.047911,0.802386,0.859472,0.811797,551.547836,0.793004,0.743170,0.729675,0.743170
std,0.0,0.818601,1.117348,1.118086,4431.925736,0.287330,0.287227,0.261682,82.059146,0.068680,...,0.125068,100.581742,0.129977,0.140637,0.089525,117.362988,0.054638,0.093607,0.104365,0.093607
min,0.0,0.000000,0.000000,0.000000,43200.000000,0.010000,0.010000,0.100000,677.000000,0.540661,...,0.089844,106.000000,0.410413,0.244718,0.392655,269.000000,0.603546,0.364845,0.295164,0.364845
25%,0.0,0.000000,0.000000,0.000000,86400.000000,0.210000,0.210000,0.300000,677.000000,0.687190,...,0.450479,166.000000,0.736170,0.798799,0.756256,466.000000,0.752795,0.687793,0.676935,0.687793
50%,0.0,1.000000,1.000000,1.000000,86400.000000,0.410000,0.410000,0.500000,852.000000,0.732410,...,0.530201,224.000000,0.830688,0.904494,0.832441,551.000000,0.793188,0.755869,0.745112,0.755869
75%,0.0,2.000000,2.000000,2.000000,86400.000000,0.710000,0.710000,0.750000,852.000000,0.792280,...,0.611940,299.000000,0.901961,0.965625,0.882526,636.000000,0.837587,0.815728,0.808933,0.815728
max,0.0,2.000000,3.000000,3.000000,86400.000000,0.910000,0.910000,0.950000,852.000000,0.927234,...,0.842904,488.000000,0.996552,0.998654,0.959842,743.000000,0.928335,0.928404,0.927748,0.928404


In [246]:
threshold_df

,Unnamed: 0,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,overall_support,macro_avg_prec,...,true_f1,true_support,false_prec,false_rec,false_f1,false_support,weigh_avg_prec,weigh_avg_rec,weigh_avg_f1,acc
0,0,0,0,0,86400,0.01,0.01,0.1,852,0.813372,...,0.829312,488,0.891403,0.541209,0.673504,364,0.802015,0.775822,0.762746,0.775822
1,0,0,0,0,86400,0.01,0.11,0.1,852,0.807366,...,0.812044,465,0.909502,0.519380,0.661184,387,0.798016,0.758216,0.743520,0.758216
2,0,0,0,0,86400,0.01,0.21,0.1,852,0.774648,...,0.760267,416,0.918552,0.465596,0.617960,436,0.778026,0.705399,0.687444,0.705399
3,0,0,0,0,86400,0.01,0.31,0.1,852,0.734122,...,0.693548,361,0.923077,0.415479,0.573034,491,0.762953,0.643192,0.624097,0.643192
4,0,0,0,0,86400,0.01,0.41,0.1,852,0.700049,...,0.634352,318,0.923077,0.382022,0.540397,534,0.756591,0.592723,0.575465,0.592723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87324,0,0,2,0,43200,0.31,0.41,0.1,787,0.645059,...,0.594354,328,0.710407,0.684096,0.697003,459,0.655936,0.653113,0.654222,0.653113
87325,0,0,2,0,43200,0.31,0.51,0.1,787,0.624559,...,0.538588,264,0.773756,0.653920,0.708808,523,0.673659,0.642948,0.651708,0.642948
87326,0,0,2,0,43200,0.31,0.61,0.1,787,0.610066,...,0.514190,254,0.773756,0.641651,0.701538,533,0.668096,0.630241,0.641073,0.630241
87327,0,0,2,0,43200,0.31,0.71,0.1,787,0.597023,...,0.491525,245,0.773756,0.630996,0.695122,542,0.663719,0.618806,0.631741,0.618806


In [270]:
threshold_df[['overall_num_pairs_min', 'pred_pairs_min',
       'truth_pairs_min', 'duration_threshold', 'pred_threshold',
       'truth_threshold', 'sim_threshold', 'overall_support', 
        'macro_avg_f1', 'true_prec', 'true_rec', 'true_support','false_support']].query("((true_prec >= 0.92) & (true_rec >= 0.4) & (true_support <= 250))")

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,overall_support,macro_avg_f1,true_prec,true_rec,true_support,false_support
20714,0,3,3,86400,0.71,0.41,0.70,852,0.751105,0.920000,0.415663,166,686
20715,0,3,3,86400,0.71,0.51,0.70,852,0.775540,0.920000,0.456954,151,701
21086,0,0,0,86400,0.71,0.61,0.75,852,0.732341,0.920354,0.421053,247,605
21096,0,0,1,86400,0.71,0.61,0.75,852,0.732341,0.920354,0.421053,247,605
21486,0,1,0,86400,0.71,0.61,0.75,852,0.732341,0.920354,0.421053,247,605
21496,0,1,1,86400,0.71,0.61,0.75,852,0.732341,0.920354,0.421053,247,605
22315,0,3,3,86400,0.71,0.51,0.75,852,0.758035,0.940299,0.417219,151,701
23425,0,2,2,86400,0.51,0.51,0.80,852,0.750274,0.925000,0.418079,177,675
23465,0,2,2,86400,0.61,0.51,0.80,852,0.745534,0.935065,0.406780,177,675
23835,0,3,3,86400,0.51,0.51,0.80,852,0.766933,0.929577,0.437086,151,701


In [225]:
threshold_df[['overall_num_pairs_min', 'pred_pairs_min',
       'truth_pairs_min', 'duration_threshold', 'pred_threshold',
       'truth_threshold', 'sim_threshold', 'overall_support', 
        'macro_avg_f1', 'true_prec', 'true_rec', 'true_support','false_support']].query("((duration_threshold !=300))")

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,overall_support,macro_avg_f1,true_prec,true_rec,true_support,false_support


In [ ]:
threshold_df

,Unnamed: 0,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,r1_overall_support,r1_macro_avg_prec,...,num_pairs_y,pct_similar_pairs_pred,order_level_similarity_pred,lenient_order_pred,moderate_order_pred,order_level_lenient_truth,order_level_moderate_truth,order_level_combined_moderate_truth,order_level_combined_lenient_truth,order_level_simple_or_pred
0,0,4,1,1,86400,0.01,0.01,0.1,497,0.832347,...,4770.0,248.162771,420.0,19.0,115.0,27.0,178.0,381.0,340.0,433.0
1,0,4,1,1,86400,0.01,0.21,0.1,497,0.809324,...,4770.0,248.162771,420.0,19.0,115.0,27.0,178.0,330.0,272.0,433.0
2,0,4,1,1,86400,0.01,0.41,0.1,497,0.740040,...,4770.0,248.162771,420.0,19.0,115.0,27.0,178.0,270.0,196.0,433.0
3,0,4,1,1,86400,0.01,0.61,0.1,497,0.699625,...,4770.0,248.162771,420.0,19.0,115.0,27.0,178.0,235.0,147.0,433.0
4,0,4,1,1,86400,0.01,0.81,0.1,497,0.686923,...,4770.0,248.162771,420.0,19.0,115.0,27.0,178.0,224.0,122.0,433.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4720,0,4,3,1,600,0.81,0.01,0.9,268,0.589005,...,2106.0,58.345621,30.0,11.0,61.0,16.0,97.0,234.0,218.0,77.0
4721,0,4,3,1,600,0.81,0.21,0.9,268,0.625654,...,2106.0,58.345621,30.0,11.0,61.0,16.0,97.0,220.0,199.0,77.0
4722,0,4,3,1,600,0.81,0.41,0.9,268,0.682090,...,2106.0,58.345621,30.0,11.0,61.0,16.0,97.0,188.0,163.0,77.0
4723,0,4,3,1,600,0.81,0.61,0.9,268,0.768478,...,2106.0,58.345621,30.0,11.0,61.0,16.0,97.0,155.0,123.0,77.0


In [342]:
threshold_expt_df = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_expt_df.csv')
threshold_expt_df_2 = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_expt_df_2.csv')
threshold_expt_df_3 = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_expt_df_3.csv')

threshold_expt_df = pd.concat([threshold_expt_df, threshold_expt_df_2,threshold_expt_df_3]).reset_index()
threshold_expt_df['recall_uplift'] = threshold_expt_df['r1_true_rec'] - threshold_expt_df['r2_true_rec']

selected_cols = ['overall_num_pairs_min','pred_pairs_min',
       'truth_pairs_min', 'duration_threshold', 'pred_threshold',
       'truth_threshold', 'sim_threshold','r1_true_prec', 'r1_true_rec','r1_macro_avg_f1','r1_true_f1',
        'r2_true_prec', 'r2_true_rec','r1_true_support','r1_false_support','recall_uplift']

In [343]:
threshold_expt_df[selected_cols].\
query('r1_true_prec > 0.95 & r1_true_rec > 0.5 & r1_true_support < 300 & truth_pairs_min == 1').\
sort_values('r1_true_rec', ascending=False).head(50)

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,r1_true_prec,r1_true_rec,r1_macro_avg_f1,r1_true_f1,r2_true_prec,r2_true_rec,r1_true_support,r1_false_support,recall_uplift
120885,3,2,1,600,0.01,0.01,0.4,0.950226,0.795455,0.664395,0.865979,1.000000,0.276515,264,39,0.518939
121485,3,1,1,600,0.01,0.01,0.7,0.958140,0.780303,0.666283,0.860125,1.000000,0.276515,264,39,0.503788
126660,4,1,1,600,0.01,0.01,0.7,0.952880,0.777778,0.653461,0.856471,1.000000,0.260684,234,34,0.517094
126535,4,2,1,600,0.01,0.01,0.5,0.951351,0.752137,0.633723,0.840095,1.000000,0.260684,234,34,0.491453
121110,3,2,1,600,0.01,0.01,0.5,0.956522,0.750000,0.642604,0.840764,1.000000,0.276515,264,39,0.473485
126335,4,3,1,600,0.01,0.01,0.2,0.955556,0.735043,0.628574,0.830918,1.000000,0.260684,234,34,0.474359
126340,4,3,1,600,0.21,0.01,0.2,0.955556,0.735043,0.628574,0.830918,1.000000,0.260684,234,34,0.474359
126540,4,2,1,600,0.21,0.01,0.5,0.950276,0.735043,0.621069,0.828916,1.000000,0.260684,234,34,0.474359
121125,3,2,1,600,0.21,0.01,0.5,0.955665,0.734848,0.631245,0.830835,1.000000,0.276515,264,39,0.458333
120510,3,3,1,600,0.01,0.01,0.2,0.960396,0.734848,0.637738,0.832618,1.000000,0.276515,264,39,0.458333


In [344]:
threshold_expt_df[selected_cols].\
query('r1_true_prec > 0.95 & r1_true_rec > 0.5 & r1_true_support < 400 & truth_pairs_min == 1 & recall_uplift >= 0.4').\
sort_values('r1_true_f1', ascending=False).head(50)

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,r1_true_prec,r1_true_rec,r1_macro_avg_f1,r1_true_f1,r2_true_prec,r2_true_rec,r1_true_support,r1_false_support,recall_uplift
120885,3,2,1,600,0.01,0.01,0.4,0.950226,0.795455,0.664395,0.865979,1.000000,0.276515,264,39,0.518939
121485,3,1,1,600,0.01,0.01,0.7,0.958140,0.780303,0.666283,0.860125,1.000000,0.276515,264,39,0.503788
126660,4,1,1,600,0.01,0.01,0.7,0.952880,0.777778,0.653461,0.856471,1.000000,0.260684,234,34,0.517094
109335,3,1,1,3600,0.01,0.01,0.7,0.953668,0.771875,0.706390,0.853195,0.989130,0.284375,320,66,0.487500
119460,2,1,1,600,0.01,0.01,0.7,0.951311,0.765060,0.680725,0.848080,0.989474,0.283133,332,61,0.481928
121110,3,2,1,600,0.01,0.01,0.5,0.956522,0.750000,0.642604,0.840764,1.000000,0.276515,264,39,0.473485
126535,4,2,1,600,0.01,0.01,0.5,0.951351,0.752137,0.633723,0.840095,1.000000,0.260684,234,34,0.491453
120510,3,3,1,600,0.01,0.01,0.2,0.960396,0.734848,0.637738,0.832618,1.000000,0.276515,264,39,0.458333
120525,3,3,1,600,0.21,0.01,0.2,0.960396,0.734848,0.637738,0.832618,1.000000,0.276515,264,39,0.458333
126335,4,3,1,600,0.01,0.01,0.2,0.955556,0.735043,0.628574,0.830918,1.000000,0.260684,234,34,0.474359


In [345]:
threshold_expt_df[selected_cols].\
query('r1_true_prec > 0.96 & r1_true_rec > 0.5 & truth_pairs_min == 1').\
sort_values('r1_macro_avg_f1', ascending=False).head(50)

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,r1_true_prec,r1_true_rec,r1_macro_avg_f1,r1_true_f1,r2_true_prec,r2_true_rec,r1_true_support,r1_false_support,recall_uplift
122702,4,1,1,86400,0.61,0.41,0.80,0.960526,0.540741,0.732335,0.691943,0.956522,0.407407,270,227,0.133333
122752,4,3,1,86400,0.61,0.41,0.80,0.960526,0.540741,0.732335,0.691943,0.956522,0.407407,270,227,0.133333
122727,4,2,1,86400,0.61,0.41,0.80,0.960526,0.540741,0.732335,0.691943,0.956522,0.407407,270,227,0.133333
12784,1,1,1,86400,0.61,0.41,0.80,0.960854,0.556701,0.732011,0.704961,0.957143,0.414433,485,367,0.142268
12814,1,1,1,86400,0.71,0.41,0.80,0.963504,0.544330,0.725604,0.695652,0.957143,0.414433,485,367,0.129897
47614,3,3,1,86400,0.71,0.41,0.90,0.962733,0.514950,0.721725,0.670996,0.955556,0.428571,301,264,0.086379
47314,3,2,1,86400,0.71,0.41,0.90,0.962733,0.514950,0.721725,0.670996,0.955556,0.428571,301,264,0.086379
47014,3,1,1,86400,0.71,0.41,0.90,0.962733,0.514950,0.721725,0.670996,0.955556,0.428571,301,264,0.086379
12844,1,1,1,86400,0.81,0.41,0.80,0.962121,0.523711,0.712941,0.678238,0.957143,0.414433,485,367,0.109278
13714,1,1,1,86400,0.71,0.41,0.85,0.961977,0.521649,0.711666,0.676471,0.957143,0.414433,485,367,0.107216


In [ ]:
threshold_expt_df[selected_cols].\
query('r1_true_prec > 0.95 & r1_true_rec > 0.5 & r1_true_support < 400 & truth_pairs_min == 1').\
sort_values('r1_macro_avg_f1', ascending=False).head(50)

In [336]:
threshold_expt_df[selected_cols].\
query('r1_true_prec > 0.95 & r1_true_rec > 0.6 & truth_pairs_min == 1').\
sort_values('r1_true_rec', ascending=False).head(50)

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,r1_true_prec,r1_true_rec,r1_macro_avg_f1,r1_true_f1,r2_true_prec,r2_true_rec,r1_true_support,r1_false_support,recall_uplift
120885,3,2,1,600,0.01,0.01,0.40,0.950226,0.795455,0.664395,0.865979,1.000000,0.276515,264,39,0.518939
121485,3,1,1,600,0.01,0.01,0.70,0.958140,0.780303,0.666283,0.860125,1.000000,0.276515,264,39,0.503788
109335,3,1,1,3600,0.01,0.01,0.70,0.953668,0.771875,0.706390,0.853195,0.989130,0.284375,320,66,0.487500
119460,2,1,1,600,0.01,0.01,0.70,0.951311,0.765060,0.680725,0.848080,0.989474,0.283133,332,61,0.481928
121110,3,2,1,600,0.01,0.01,0.50,0.956522,0.750000,0.642604,0.840764,1.000000,0.276515,264,39,0.473485
121125,3,2,1,600,0.21,0.01,0.50,0.955665,0.734848,0.631245,0.830835,1.000000,0.276515,264,39,0.458333
120525,3,3,1,600,0.21,0.01,0.20,0.960396,0.734848,0.637738,0.832618,1.000000,0.276515,264,39,0.458333
120510,3,3,1,600,0.01,0.01,0.20,0.960396,0.734848,0.637738,0.832618,1.000000,0.276515,264,39,0.458333
117435,1,1,1,600,0.01,0.01,0.70,0.950139,0.726695,0.675430,0.823529,0.979021,0.296610,472,100,0.430085
120735,3,3,1,600,0.01,0.01,0.30,0.969388,0.719697,0.639071,0.826087,1.000000,0.276515,264,39,0.443182


In [309]:
threshold_expt_df

,Unnamed: 0,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,r1_overall_support,r1_macro_avg_prec,...,num_pairs_y,pct_similar_pairs_pred,order_level_similarity_pred,lenient_order_pred,moderate_order_pred,order_level_lenient_truth,order_level_moderate_truth,order_level_combined_moderate_truth,order_level_combined_lenient_truth,order_level_simple_or_pred
0,0,1,1,1,86400,0.01,0.01,0.10,852,0.805010,...,5373.0,427.496104,631.0,40.0,210.0,57.0,305.0,608.0,519.0,680.0
1,0,1,1,1,86400,0.01,0.11,0.10,852,0.807815,...,5373.0,427.496104,631.0,40.0,210.0,57.0,305.0,592.0,498.0,680.0
2,0,1,1,1,86400,0.01,0.21,0.10,852,0.782079,...,5373.0,427.496104,631.0,40.0,210.0,57.0,305.0,557.0,451.0,680.0
3,0,1,1,1,86400,0.01,0.31,0.10,852,0.749726,...,5373.0,427.496104,631.0,40.0,210.0,57.0,305.0,513.0,400.0,680.0
4,0,1,1,1,86400,0.01,0.41,0.10,852,0.729138,...,5373.0,427.496104,631.0,40.0,210.0,57.0,305.0,485.0,359.0,680.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91780,0,3,3,2,43200,0.91,0.01,0.65,491,0.711045,...,4154.0,132.897897,55.0,21.0,113.0,31.0,174.0,323.0,272.0,148.0
91781,0,3,3,2,43200,0.91,0.11,0.65,491,0.713961,...,4154.0,132.897897,55.0,21.0,113.0,31.0,174.0,321.0,269.0,148.0
91782,0,3,3,2,43200,0.91,0.21,0.65,491,0.728538,...,4154.0,132.897897,55.0,21.0,113.0,31.0,174.0,311.0,255.0,148.0
91783,0,3,3,2,43200,0.91,0.31,0.65,491,0.744642,...,4154.0,132.897897,55.0,21.0,113.0,31.0,174.0,290.0,227.0,148.0


In [445]:
threshold_expt_df_4 = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_expt_df_4.csv')
# not sure what happened to 5
threshold_expt_df_6 = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_expt_df_6.csv')
threshold_expt_df_7 = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_expt_df_7.csv')
threshold_expt_df_8 = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_expt_df_8.csv')
threshold_expt_df_9 = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_expt_df_9.csv')

threshold_expt_df_4 = pd.concat([threshold_expt_df_4, threshold_expt_df_6, threshold_expt_df_7, threshold_expt_df_8,threshold_expt_df_9]).drop_duplicates().reset_index()
threshold_expt_df_4['hash_threshold'].fillna(14,inplace=True)
threshold_expt_df_4['recall_uplift'] = threshold_expt_df_4['r1_true_rec'] - threshold_expt_df_4['r2_true_rec']

selected_cols = ['overall_num_pairs_min','pred_pairs_min',
       'truth_pairs_min', 'duration_threshold', 'pred_threshold',
       'truth_threshold', 'hash_threshold','sim_threshold','r1_true_prec', 'r1_true_rec','r1_macro_avg_f1','r1_true_f1',
        'r2_true_prec', 'r2_true_rec','r1_true_support','r1_false_support','recall_uplift']

In [446]:
threshold_expt_df_4[selected_cols].\
query('r1_true_prec > 0.975 & r1_true_rec > 0.5 & truth_pairs_min == 1')

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,hash_threshold,sim_threshold,r1_true_prec,r1_true_rec,r1_macro_avg_f1,r1_true_f1,r2_true_prec,r2_true_rec,r1_true_support,r1_false_support,recall_uplift
2280,2,1,1,600,0.01,0.01,14.0,0.91,0.976000,0.516949,0.560699,0.675900,0.979021,0.296610,472,100,0.220339
3405,1,1,1,600,0.01,0.01,14.0,0.96,0.976096,0.519068,0.562144,0.677732,0.979021,0.296610,472,100,0.222458
7020,1,1,1,600,0.01,0.01,14.0,0.95,0.976834,0.536017,0.573704,0.692202,0.979021,0.296610,472,100,0.239407
8325,1,1,1,300,0.01,0.01,14.0,0.95,0.979424,0.536036,0.563592,0.692868,0.977778,0.297297,444,86,0.238739
8820,2,1,1,300,0.01,0.01,14.0,0.91,0.978723,0.518018,0.551332,0.677467,0.977778,0.297297,444,86,0.220721


In [447]:
threshold_expt_df_4[selected_cols].query('r1_true_prec > 0.99')

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,hash_threshold,sim_threshold,r1_true_prec,r1_true_rec,r1_macro_avg_f1,r1_true_f1,r2_true_prec,r2_true_rec,r1_true_support,r1_false_support,recall_uplift


In [448]:
threshold_expt_df_4[selected_cols].\
query('r1_true_prec > 0.985 & truth_pairs_min == 1')

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,hash_threshold,sim_threshold,r1_true_prec,r1_true_rec,r1_macro_avg_f1,r1_true_f1,r2_true_prec,r2_true_rec,r1_true_support,r1_false_support,recall_uplift
2130,1,1,1,600,0.01,0.01,14.0,1.01,0.985714,0.438559,0.513476,0.607038,0.979021,0.296610,472,100,0.141949
3420,1,1,1,600,0.01,0.01,14.0,1.01,0.985714,0.438559,0.513476,0.607038,0.979021,0.296610,472,100,0.141949
3435,1,1,1,600,0.01,0.01,14.0,1.06,0.985714,0.438559,0.513476,0.607038,0.979021,0.296610,472,100,0.141949
8250,1,2,1,300,0.01,0.01,14.0,0.91,0.985149,0.448198,0.508533,0.616099,0.977778,0.297297,444,86,0.150901
8835,2,2,1,300,0.01,0.01,14.0,0.91,0.985149,0.448198,0.508533,0.616099,0.977778,0.297297,444,86,0.150901


In [449]:
threshold_expt_df_4[selected_cols].\
query('r1_true_prec > 0.985 & r1_true_rec > 0.4 & truth_pairs_min == 1')

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,hash_threshold,sim_threshold,r1_true_prec,r1_true_rec,r1_macro_avg_f1,r1_true_f1,r2_true_prec,r2_true_rec,r1_true_support,r1_false_support,recall_uplift
2130,1,1,1,600,0.01,0.01,14.0,1.01,0.985714,0.438559,0.513476,0.607038,0.979021,0.296610,472,100,0.141949
3420,1,1,1,600,0.01,0.01,14.0,1.01,0.985714,0.438559,0.513476,0.607038,0.979021,0.296610,472,100,0.141949
3435,1,1,1,600,0.01,0.01,14.0,1.06,0.985714,0.438559,0.513476,0.607038,0.979021,0.296610,472,100,0.141949
8250,1,2,1,300,0.01,0.01,14.0,0.91,0.985149,0.448198,0.508533,0.616099,0.977778,0.297297,444,86,0.150901
8835,2,2,1,300,0.01,0.01,14.0,0.91,0.985149,0.448198,0.508533,0.616099,0.977778,0.297297,444,86,0.150901


In [450]:
threshold_expt_df_4.\
query('r1_true_prec > 0.985 & r1_true_rec > 0.4 & truth_pairs_min == 1')

,index,Unnamed: 0,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,r1_overall_support,...,order_level_similarity_pred,lenient_order_pred,moderate_order_pred,order_level_lenient_truth,order_level_moderate_truth,order_level_combined_moderate_truth,order_level_combined_lenient_truth,order_level_simple_or_pred,hash_threshold,recall_uplift
2130,2130,0,1,1,1,600,0.01,0.01,1.01,572,...,88.0,29.0,143.0,36.0,209.0,472.0,423.0,210.0,14.0,0.141949
3420,1335,0,1,1,1,600,0.01,0.01,1.01,572,...,88.0,29.0,143.0,36.0,209.0,472.0,423.0,210.0,14.0,0.141949
3435,1350,0,1,1,1,600,0.01,0.01,1.06,572,...,88.0,29.0,143.0,36.0,209.0,472.0,423.0,210.0,14.0,0.141949
8250,2175,0,1,2,1,300,0.01,0.01,0.91,530,...,91.0,27.0,135.0,34.0,194.0,444.0,398.0,202.0,14.0,0.150901
8835,2760,0,2,2,1,300,0.01,0.01,0.91,530,...,91.0,27.0,135.0,34.0,194.0,444.0,398.0,202.0,14.0,0.150901


In [451]:
threshold_expt_df_4.\
query('r1_true_prec > 0.985  & truth_pairs_min == 1')

,index,Unnamed: 0,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,sim_threshold,r1_overall_support,...,order_level_similarity_pred,lenient_order_pred,moderate_order_pred,order_level_lenient_truth,order_level_moderate_truth,order_level_combined_moderate_truth,order_level_combined_lenient_truth,order_level_simple_or_pred,hash_threshold,recall_uplift
2130,2130,0,1,1,1,600,0.01,0.01,1.01,572,...,88.0,29.0,143.0,36.0,209.0,472.0,423.0,210.0,14.0,0.141949
3420,1335,0,1,1,1,600,0.01,0.01,1.01,572,...,88.0,29.0,143.0,36.0,209.0,472.0,423.0,210.0,14.0,0.141949
3435,1350,0,1,1,1,600,0.01,0.01,1.06,572,...,88.0,29.0,143.0,36.0,209.0,472.0,423.0,210.0,14.0,0.141949
8250,2175,0,1,2,1,300,0.01,0.01,0.91,530,...,91.0,27.0,135.0,34.0,194.0,444.0,398.0,202.0,14.0,0.150901
8835,2760,0,2,2,1,300,0.01,0.01,0.91,530,...,91.0,27.0,135.0,34.0,194.0,444.0,398.0,202.0,14.0,0.150901


In [452]:
threshold_expt_df_4[selected_cols].query('hash_threshold == 12')

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,hash_threshold,sim_threshold,r1_true_prec,r1_true_rec,r1_macro_avg_f1,r1_true_f1,r2_true_prec,r2_true_rec,r1_true_support,r1_false_support,recall_uplift
11370,1,1,1,86400,0.01,0.01,12.0,1.01,0.976923,0.417763,0.577316,0.585253,0.971429,0.335526,608,244,0.082237
11371,1,1,1,86400,0.21,0.01,12.0,1.01,0.973913,0.368421,0.542130,0.534606,0.971429,0.335526,608,244,0.032895
11372,1,1,1,86400,0.41,0.01,12.0,1.01,0.972603,0.350329,0.528937,0.515115,0.971429,0.335526,608,244,0.014803
11373,1,1,1,86400,0.61,0.01,12.0,1.01,0.971831,0.340461,0.521667,0.504263,0.971429,0.335526,608,244,0.004934
11374,1,1,1,86400,0.81,0.01,12.0,1.01,0.971698,0.338816,0.520450,0.502439,0.971429,0.335526,608,244,0.003289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12010,3,3,1,600,0.01,0.01,12.0,1.01,0.980132,0.313559,0.423741,0.475120,0.979021,0.296610,472,100,0.016949
12011,3,3,1,600,0.21,0.01,12.0,1.01,0.979730,0.307203,0.418985,0.467742,0.979021,0.296610,472,100,0.010593
12012,3,3,1,600,0.41,0.01,12.0,1.01,0.979021,0.296610,0.411007,0.455285,0.979021,0.296610,472,100,0.000000
12013,3,3,1,600,0.61,0.01,12.0,1.01,0.979021,0.296610,0.411007,0.455285,0.979021,0.296610,472,100,0.000000


In [453]:
import numpy as np
np.append(np.arange(0.01, 1.1, 0.1), [0.95, 0.9875])

array([0.01  , 0.11  , 0.21  , 0.31  , 0.41  , 0.51  , 0.61  , 0.71  ,
       0.81  , 0.91  , 1.01  , 0.95  , 0.9875])

In [455]:
threshold_expt_df_v4_1 = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_expt_df_v4_1.csv')
threshold_expt_df_v4_2 = pd.read_csv(f'{INTERMEDIATE_RESULTS_PATH_T}/threshold_expt_df_v4_2.csv')
threshold_expt_df_v4 = pd.concat([threshold_expt_df_v4_1,threshold_expt_df_v4_2])

threshold_expt_df_v4 = threshold_expt_df_v4.drop_duplicates().reset_index()
threshold_expt_df_v4['recall_uplift'] = threshold_expt_df_v4['r1_true_rec'] - threshold_expt_df_v4['r2_true_rec']

selected_cols = ['overall_num_pairs_min','pred_pairs_min',
       'truth_pairs_min', 'duration_threshold', 'pred_threshold',
       'truth_threshold', 'hash_threshold','sim_threshold','inca_moderate_threshold','r1_true_prec', 'r1_true_rec','r1_macro_avg_f1','r1_true_f1',
        'r2_true_prec', 'r2_true_rec','r1_true_support','r1_false_support','r1_overall_support','recall_uplift']

threshold_expt_df_v4[selected_cols]

,overall_num_pairs_min,pred_pairs_min,truth_pairs_min,duration_threshold,pred_threshold,truth_threshold,hash_threshold,sim_threshold,inca_moderate_threshold,r1_true_prec,r1_true_rec,r1_macro_avg_f1,r1_true_f1,r2_true_prec,r2_true_rec,r1_true_support,r1_false_support,r1_overall_support,recall_uplift
0,1,1,1,600,0.01,0.01,6,1.010,0.8000,0.977528,0.368644,0.462024,0.535385,0.97561,0.338983,472,100,572,0.029661
1,1,1,1,600,0.21,0.01,6,1.010,0.8000,0.976190,0.347458,0.446726,0.512500,0.97561,0.338983,472,100,572,0.008475
2,1,1,1,600,0.41,0.01,6,1.010,0.8000,0.975758,0.341102,0.442096,0.505495,0.97561,0.338983,472,100,572,0.002119
3,1,1,1,600,0.61,0.01,6,1.010,0.8000,0.975758,0.341102,0.442096,0.505495,0.97561,0.338983,472,100,572,0.002119
4,1,1,1,600,0.81,0.01,6,1.010,0.8000,0.975758,0.341102,0.442096,0.505495,0.97561,0.338983,472,100,572,0.002119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,3,3,1,300,0.01,0.01,8,0.975,0.9875,1.000000,0.216216,0.343162,0.355556,1.00000,0.171171,444,86,530,0.045045
941,3,3,1,300,0.21,0.01,8,0.975,0.9875,1.000000,0.207207,0.335764,0.343284,1.00000,0.171171,444,86,530,0.036036
942,3,3,1,300,0.41,0.01,8,0.975,0.9875,1.000000,0.200450,0.330167,0.333959,1.00000,0.171171,444,86,530,0.029279
943,3,3,1,300,0.61,0.01,8,0.975,0.9875,1.000000,0.195946,0.326413,0.327684,1.00000,0.171171,444,86,530,0.024775


In [457]:
threshold_expt_df_v4[selected_cols][['duration_threshold','r1_overall_support']].drop_duplicates()

,duration_threshold,r1_overall_support
0,600,572
405,300,530


# Preparing prod_test_2_220725_220731

In [13]:
prod_test_2 = pd.read_csv(f'{RESULT_PATH}/Trace analysis/Production-Testing/prod_test_2_220725_220731.csv', index_col = [0])


In [14]:
def download_image(url, merchant_id, file_name, folder_name):
    
    global numberImages 
    
    ## Set up the image URL and filename
    image_url = url

    new_path = f"{DATA_PATH}/{folder_name}/{merchant_id}/"
    file_path = new_path + file_name 

    if os.path.exists(file_path):
        # print("Exists: ", file_name, '\n')
        return

    try:
        # Open the url image, set stream to True, this will return the stream content.
        r = requests.get(image_url, stream = True)

        # Check if the image was retrieved successfully
        if r.status_code == 200:
            # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
            r.raw.decode_content = True

            if not os.path.exists(new_path):
                os.makedirs(new_path)

            # Open a local file with wb ( write binary ) permission.
            with open(file_path,'wb') as f:
                shutil.copyfileobj(r.raw, f)
                numberImages += 1

            #sleep for half second to prevent getting bumped out
            time.sleep(0.5)

            #  print('Downloaded: ',file_name, '\n')
        else:
              print('Image Couldn\'t be retreived')
                
    except Exception as e:
        print('Error Type: {0}\n'.format(e.__class__.__name__))
        print('Error Message: {0}\n'.format(e))
        
# delete images that do not exist
def test_and_delete(path):
    try:
        image = Image.open(path)
    
    except:
        os.remove(path)  
        print(f'\n{path} not valid and deleted.\n')
        

In [15]:
prod_test_2 = prod_test_2.drop_duplicates()
prod_test_2['file_name_new'] = prod_test_2['img_new'].str.split('/').str[-1]
prod_test_2['item_uuid_new'] = prod_test_2['file_name_new'].str[0:36]
prod_test_2['file_name_old'] = prod_test_2['img_old'].str.split('/').str[-1]
prod_test_2['item_uuid_old'] = prod_test_2['file_name_old'].str[0:36]

In [16]:
prod_test_2

,claim_amount_new,claim_amount_old,event_timestamp_new,event_timestamp_old,img_new,img_old,order_number_new,order_number_old,saudagar_id,time_diff,file_name_new,item_uuid_new,file_name_old,item_uuid_old
17,9920,NaN,2022-07-23 16:32:11.915000+00:00,NaN,https://i.gojekapi.com/darkroom/food-merchant-...,NaN,F-1780077304,NaN,G079105360,NaN,5f9759db-38de-4c39-9597-b4749beac758_GoFood-F-...,5f9759db-38de-4c39-9597-b4749beac758,NaN,NaN
18,9920,NaN,2022-07-23 16:32:11.915000+00:00,NaN,https://i.gojekapi.com/darkroom/food-merchant-...,NaN,F-1780077304,NaN,G079105360,NaN,9d1638a4-5bdf-46b5-b3f9-7941f67f4e0d_GoFood-F-...,9d1638a4-5bdf-46b5-b3f9-7941f67f4e0d,NaN,NaN
29,20000,NaN,2022-07-23 16:48:15.566000+00:00,NaN,https://i.gojekapi.com/darkroom/food-merchant-...,NaN,F-1780104310,NaN,M307428,NaN,7c8ec54b-e346-4d61-9cb3-5602dbf6e7df_GoFood-F-...,7c8ec54b-e346-4d61-9cb3-5602dbf6e7df,NaN,NaN
57,13680,NaN,2022-07-23 14:10:07.479000+00:00,NaN,https://i.gojekapi.com/darkroom/food-merchant-...,NaN,F-1779887764,NaN,G862711746,NaN,8fb3de90-e46a-4e9e-b820-12b356907666_GoFood-F-...,8fb3de90-e46a-4e9e-b820-12b356907666,NaN,NaN
87,9520,NaN,2022-07-23 13:24:21.980000+00:00,NaN,https://i.gojekapi.com/darkroom/food-merchant-...,NaN,F-1779950902,NaN,G436050230,NaN,44ab3a12-f484-4b47-a236-60f3fbc04edf_GoFood-F-...,44ab3a12-f484-4b47-a236-60f3fbc04edf,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8940,21960,21960.0,2022-07-31 23:02:03.005000+00:00,2022-07-31 23:00:41.165000+00:00,https://i.gojekapi.com/darkroom/food-merchant-...,https://i.gojekapi.com/darkroom/food-merchant-...,F-1789510581,F-1789487368,G747062604,0 days 00:01:21.840000000,5a63e892-b8cd-48c6-8c61-c10ed501ac29_GoFood-F-...,5a63e892-b8cd-48c6-8c61-c10ed501ac29,92e3a714-61a1-4779-9507-1e7043cff5e7_GoFood-F-...,92e3a714-61a1-4779-9507-1e7043cff5e7
8945,21960,8380.0,2022-07-31 23:02:03.005000+00:00,2022-07-31 22:59:38.973000+00:00,https://i.gojekapi.com/darkroom/food-merchant-...,https://i.gojekapi.com/darkroom/food-merchant-...,F-1789510581,F-1789280667,G747062604,0 days 00:02:24.032000000,5a63e892-b8cd-48c6-8c61-c10ed501ac29_GoFood-F-...,5a63e892-b8cd-48c6-8c61-c10ed501ac29,ea8a62b3-f142-4b1f-a042-5ab272552b73_GoFood-F-...,ea8a62b3-f142-4b1f-a042-5ab272552b73
8947,21960,35360.0,2022-07-31 23:02:03.005000+00:00,2022-07-30 23:12:49.943000+00:00,https://i.gojekapi.com/darkroom/food-merchant-...,https://i.gojekapi.com/darkroom/food-merchant-...,F-1789510581,F-1788458613,G747062604,0 days 23:49:13.062000000,5a63e892-b8cd-48c6-8c61-c10ed501ac29_GoFood-F-...,5a63e892-b8cd-48c6-8c61-c10ed501ac29,4ffd2ae5-d636-4df0-8d28-a52355792786_GoFood-F-...,4ffd2ae5-d636-4df0-8d28-a52355792786
11445,10840,26680.0,2022-07-31 23:08:00.233000+00:00,2022-07-31 23:05:51.627000+00:00,https://i.gojekapi.com/darkroom/food-merchant-...,https://i.gojekapi.com/darkroom/food-merchant-...,F-1789852707,F-1789851882,G294748252,0 days 00:02:08.606000000,5127d816-af15-48a8-829e-e8d18f50f25e_GoFood-F-...,5127d816-af15-48a8-829e-e8d18f50f25e,080fcf43-4e6b-4459-9532-030cd1d046bf_GoFood-F-...,080fcf43-4e6b-4459-9532-030cd1d046bf


In [22]:
prod_test_2[['saudagar_id','file_name_old','img_old']].rename(columns = {'file_name_old': 'file_name',
                                                                       'img_old': 'img'})

prod_test_dl_2 = pd.concat([prod_test_2[['saudagar_id','file_name_new','img_new']].rename(columns = {'file_name_new': 'file_name',
                                                                                              'img_new': 'img'}),
                        prod_test_2[['saudagar_id','file_name_old','img_old']].rename(columns = {'file_name_old': 'file_name',
                                                                                              'img_old': 'img'})])

prod_test_dl_2 = prod_test_dl_2.drop_duplicates()
prod_test_dl_2 = prod_test_dl_2.dropna()

url_sel_list = prod_test_dl_2['img']
merchant_sel_list = prod_test_dl_2['saudagar_id']
file_name_sel_list = prod_test_dl_2['file_name']
folder_name = "prod_test_2"

In [23]:
numberImages = 0
n_process = 4
pool = Pool(n_process)
start = datetime.now()

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_sel_list)} outlets completed in {duration}')

pool.close()
pool.join()

Trend calculation for 6022 outlets completed in 0:00:29.639139


In [24]:
import glob

for filename in glob.glob('/home/jovyan/data-vol-1/similarity-project/data/prod_test_1/**/**.jpeg', recursive=True):
    if os.stat(filename).st_size==0:
        print(filename)
        os.remove(filename)

In [25]:
numberImages = 0
n_process = 4
pool = Pool(n_process)
start = datetime.now()

pool.starmap(download_image, zip(url_sel_list, merchant_sel_list, file_name_sel_list, repeat(folder_name)))
duration = datetime.now() - start
print(f'Trend calculation for {len(merchant_sel_list)} outlets completed in {duration}')

pool.close()
pool.join()

Trend calculation for 6022 outlets completed in 0:00:00.075137


In [ ]:
# okay download complete for prod_test_2. 